## Automatización de descarga de archivos RINEX históricos para la estación Huancayo.

Si se tuviera directamente archivos .s4 se omite esta parte del código (obtención de RINEX).

In [27]:
import os
import requests
from datetime import datetime

# ==========================================
# CONFIGURACIÓN HUANCAYO
# ==========================================

CONFIGURACION = {
    "HUANCAYO": {
        "code": "lhyo",
        "periods": [
            (2013, 1),
            (2013, 2),
            (2013, 3),
            (2013, 4),
            (2013, 5),
            (2013, 6),
            (2013, 7),
            (2013, 8),
            (2013, 11),
            (2013, 12),
            (2014, 4),
            (2014, 5),
            (2014, 6),
            (2014, 7),
            (2014, 8),
            (2014, 9),
            (2014, 10),
            (2014, 11),
            (2014, 12),
            (2015, 1),
            (2015, 2),
            (2015, 4),
            (2015, 5),
            (2015, 6),
            (2015, 7),
            (2015, 8),
            (2015, 11),
            (2015, 12),
            (2016, 3),
            (2016, 4),
            (2016, 5),
            (2016, 6),
            (2016, 7),
            (2016, 8),
            (2016, 9),
            (2016, 10),
            (2016, 11),
            (2016, 12)  
        ]
    }
}

CARPETA_RAIZ = ""

# ==========================================
# FUNCIÓN: CALCULAR DÍA DEL AÑO (DOY)
# ==========================================

def calcular_doy(year, month, day):
    """
    Calcula el día del año (Day of Year - DOY)
    
    Ejemplo: 1 septiembre 2015 = día 244 del año 2015
    """
    fecha = datetime(year, month, day)
    doy = fecha.timetuple().tm_yday
    return doy

# ==========================================
# FUNCIÓN: CONSTRUIR NOMBRE DE ARCHIVO
# ==========================================

def construir_nombre_archivo(codigo, year, month, day):
    """
    Construye el nombre del archivo RINEX según formato IGP
    
    Formato: lhyo[DOY]0.[YY]d.Z
    - DOY: Día del año (3 dígitos)
    - YY: Año (2 últimos dígitos)
    
    Ejemplo: 1 septiembre 2015 → lhyo2440.15d.Z
    """
    doy = calcular_doy(year, month, day)
    year_short = str(year)[-2:]
    
    filename = f"{codigo}{doy:03d}0.{year_short}d.Z"
    return filename

# ==========================================
# LÓGICA DE DESCARGA
# ==========================================

def descargar_dataset(nombre_estacion, config):
    codigo = config["code"]
    periodos = config["periods"]
    
    print(f"\n{'='*60}")
    print(f"📍 ESTACIÓN: {nombre_estacion} ({codigo})")
    print(f"📂 Tipo: RINEX (.d.Z)")
    print(f"{'='*60}")

    for year, month in periodos:
        # Crear estructura de carpetas: /año/2015_03
        folder_path = os.path.join(CARPETA_RAIZ, str(year), f"{year}_{month:02d}", "Observation_Navegation")
        os.makedirs(folder_path, exist_ok=True)
        
        # Calcular días del mes
        import calendar
        _, num_days = calendar.monthrange(year, month)
        
        print(f"\n>>> 📅 Procesando: {year}-{month:02d} ({num_days} días)")
        
        # URL Base (carpeta rinex, no scint)
        # https://lisn.igp.gob.pe/database/base/gps/LISN/lhyo/2015/09/rinex/
        base_url = f"https://lisn.igp.gob.pe/database/base/gps/LISN/{codigo}/{year}/{month:02d}/rinex/"
        
        # Contadores
        descargados = 0
        errores = 0
        
        for day in range(1, num_days + 1):
            # Construir nombre con DOY
            filename = construir_nombre_archivo(codigo, year, month, day)
            doy = calcular_doy(year, month, day)
            
            url = base_url + filename
            local_path = os.path.join(folder_path, filename)

            # MOSTRAR INTENTO
            print(f"   🔍 Día {day:02d} (DOY {doy:03d}): {filename}")

            # Verificar si ya existe
            if os.path.exists(local_path):
                print(f"      ⏭️  Ya existe")
                continue

            try:
                # Headers para simular navegador
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                    'Accept': '*/*',
                    'Connection': 'keep-alive'
                }
                
                r = requests.get(url, headers=headers, timeout=30)
                
                if r.status_code == 200:
                    # Guardar archivo .Z directamente (ya comprimido)
                    with open(local_path, "wb") as f:
                        f.write(r.content)
                    
                    file_size = len(r.content) / 1024  # KB
                    print(f"      ✅ Descargado ({file_size:.1f} KB)")
                    descargados += 1
                    
                elif r.status_code == 404:
                    print(f"      ❌ No disponible (404)")
                    errores += 1
                else:
                    print(f"      ⚠️  Error {r.status_code}")
                    errores += 1
                    
            except Exception as e:
                print(f"      ⚠️  Error de red: {str(e)[:50]}")
                errores += 1

        # Resumen del mes
        print(f"\n   📊 RESUMEN {year}-{month:02d}:")
        print(f"      ✅ Descargados: {descargados}")
        print(f"      ❌ No disponibles/Errores: {errores}")
        print(f"      📁 Guardados en: {folder_path}")

# ==========================================
# VERIFICACIÓN DE FORMATO (DEBUG)
# ==========================================

def verificar_formato():
    """Verifica que el cálculo de DOY sea correcto"""
    print("\n🔬 VERIFICACIÓN DE FORMATO:")
    print("="*60)
    
    # Caso conocido: 1 septiembre 2015 = DOY 244
    year, month, day = 2015, 9, 1
    doy = calcular_doy(year, month, day)
    filename = construir_nombre_archivo("lhyo", year, month, day)
    
    print(f"📅 Fecha: {day}/{month}/{year}")
    print(f"📊 DOY calculado: {doy} (esperado: 244)")
    print(f"📄 Archivo generado: {filename}")
    print(f"✅ Formato correcto: {filename == 'lhyo2440.15d.Z'}")
    print("="*60)

# ==========================================
# EJECUCIÓN
# ==========================================

if __name__ == "__main__":
    print("="*60)
    print("🚀 DESCARGADOR RINEX - ESTACIÓN HUANCAYO")
    print("="*60)
    print(f"📂 Carpeta destino: {os.path.abspath(CARPETA_RAIZ)}")
    
    # Verificar cálculo antes de descargar
    verificar_formato()
    
    # Procesar cada estación (en este caso solo Huancayo)
    for nombre, conf in CONFIGURACION.items():
        descargar_dataset(nombre, conf)

    print("\n" + "="*60)
    print("🏁 PROCESO FINALIZADO")
    print("="*60)
    print("\n💡 Nota: Los archivos .d.Z están comprimidos con compress (Unix)")
    print("   Para descomprimirlos en Linux/Mac: uncompress archivo.Z")
    print("   O puedes usar 7-Zip en Windows")

🚀 DESCARGADOR RINEX - ESTACIÓN HUANCAYO
📂 Carpeta destino: g:\a_Universidad\2025-2\InteligenciaArtificial\TareaAcademica\VersionFinal

🔬 VERIFICACIÓN DE FORMATO:
📅 Fecha: 1/9/2015
📊 DOY calculado: 244 (esperado: 244)
📄 Archivo generado: lhyo2440.15d.Z
✅ Formato correcto: True

📍 ESTACIÓN: HUANCAYO (lhyo)
📂 Tipo: RINEX (.d.Z)

>>> 📅 Procesando: 2013-01 (31 días)
   🔍 Día 01 (DOY 001): lhyo0010.13d.Z
      ❌ No disponible (404)
   🔍 Día 02 (DOY 002): lhyo0020.13d.Z
      ❌ No disponible (404)
   🔍 Día 03 (DOY 003): lhyo0030.13d.Z
      ❌ No disponible (404)
   🔍 Día 04 (DOY 004): lhyo0040.13d.Z
      ❌ No disponible (404)
   🔍 Día 05 (DOY 005): lhyo0050.13d.Z
      ❌ No disponible (404)
   🔍 Día 06 (DOY 006): lhyo0060.13d.Z
      ❌ No disponible (404)
   🔍 Día 07 (DOY 007): lhyo0070.13d.Z
      ❌ No disponible (404)
   🔍 Día 08 (DOY 008): lhyo0080.13d.Z
      ❌ No disponible (404)
   🔍 Día 09 (DOY 009): lhyo0090.13d.Z
      ❌ No disponible (404)
   🔍 Día 10 (DOY 010): lhyo0100.13d.Z
    

Descomprimir archivos descargados

In [28]:
import os
import subprocess

# ================================
# CONFIGURACIÓN
# ================================

CARPETA_RAIZ = ""   # Ruta donde guardaste tus carpetas 2015/2016

SEVEN_ZIP_PATH = r"C:\Program Files\7-Zip\7z.exe"   # Cambia según tu instalación
# En Linux: SEVEN_ZIP_PATH = "7z"

# ================================
# FUNCIÓN PRINCIPAL
# ================================

def descomprimir_Z(root_folder):
    print("="*60)
    print("🔽 DESCOMPRESIÓN DE ARCHIVOS .Z (y eliminación)")
    print("="*60)

    for ruta_actual, carpetas, archivos in os.walk(root_folder):
        for archivo in archivos:
            if archivo.endswith(".Z"):
                ruta_Z = os.path.join(ruta_actual, archivo)

                # Archivo descomprimido esperado
                ruta_salida = ruta_Z[:-2]   # elimina .Z final → lhyo2440.15d

                print(f"\n📄 Archivo encontrado: {ruta_Z}")

                if os.path.exists(ruta_salida):
                    print("   ⏭️  Ya existe descomprimido, eliminando .Z...")
                    try:
                        os.remove(ruta_Z)
                        print("   🗑️  .Z eliminado")
                    except:
                        print("   ⚠️ No se pudo eliminar el .Z")
                    continue

                # Descomprimir con 7-zip
                print("   🔧 Descomprimiendo...")
                try:
                    resultado = subprocess.run(
                        [SEVEN_ZIP_PATH, "x", ruta_Z, f"-o{ruta_actual}", "-y"],
                        check=True,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE
                    )
                    print("   ✅ Descompresión correcta")

                    # Ahora sí eliminar el .Z
                    try:
                        os.remove(ruta_Z)
                        print("   🗑️  Archivo .Z eliminado")
                    except Exception as e:
                        print(f"   ⚠️ Error eliminando .Z: {str(e)[:80]}")

                except Exception as e:
                    print(f"   ❌ Error descomprimiendo: {str(e)[:80]}")
                    continue

    print("\n🏁 PROCESO COMPLETADO")
    print("="*60)


# ================================
# EJECUCIÓN
# ================================

if __name__ == "__main__":
    carpeta = os.path.abspath(CARPETA_RAIZ)
    print(f"📂 Carpeta raíz: {carpeta}")
    descomprimir_Z(carpeta)


📂 Carpeta raíz: g:\a_Universidad\2025-2\InteligenciaArtificial\TareaAcademica\VersionFinal
🔽 DESCOMPRESIÓN DE ARCHIVOS .Z (y eliminación)

📄 Archivo encontrado: g:\a_Universidad\2025-2\InteligenciaArtificial\TareaAcademica\VersionFinal\2013\2013_04\Observation_Navegation\lhyo0960.13d.Z
   🔧 Descomprimiendo...
   ✅ Descompresión correcta
   🗑️  Archivo .Z eliminado

📄 Archivo encontrado: g:\a_Universidad\2025-2\InteligenciaArtificial\TareaAcademica\VersionFinal\2013\2013_04\Observation_Navegation\lhyo0970.13d.Z
   🔧 Descomprimiendo...
   ✅ Descompresión correcta
   🗑️  Archivo .Z eliminado

📄 Archivo encontrado: g:\a_Universidad\2025-2\InteligenciaArtificial\TareaAcademica\VersionFinal\2013\2013_04\Observation_Navegation\lhyo0980.13d.Z
   🔧 Descomprimiendo...
   ✅ Descompresión correcta
   🗑️  Archivo .Z eliminado

📄 Archivo encontrado: g:\a_Universidad\2025-2\InteligenciaArtificial\TareaAcademica\VersionFinal\2013\2013_04\Observation_Navegation\lhyo0990.13d.Z
   🔧 Descomprimiendo...
  

## Script de procesamiento por lotes para la conversión de archivos GNSS comprimidos (.d) a formato estándar (.o).

* Requiere la instalación local de RTKLIB (crx2rnx.exe) como motor de conversión.
* Este código no puede correr 2 veces, es decir, si ya hay archivo .o tienes q eliminarlos y volver a correrlos, porque sino no deja convertir (necesita confirmación de overwrite)

In [29]:
import os
import subprocess

# Ruta del ejecutable convbin (RTKLIB)
CONVBIN_PATH = r"C:\Users\User\Documents\RTKLIB_bin-master\RTKLIB_bin-master\bin\crx2rnx.exe"

# Configuración de años/meses
CONFIGURACION = {
    "HUANCAYO": {
        "code": "lhyo",
        "periods": [
            (2013, 1),
            (2013, 2),
            (2013, 3),
            (2013, 4),
            (2013, 5),
            (2013, 6),
            (2013, 7),
            (2013, 8),
            (2013, 11),
            (2013, 12),
            (2014, 4),
            (2014, 5),
            (2014, 6),
            (2014, 7),
            (2014, 8),
            (2014, 9),
            (2014, 10),
            (2014, 11),
            (2014, 12),
            (2015, 1),
            (2015, 2),
            (2015, 4),
            (2015, 5),
            (2015, 6),
            (2015, 7),
            (2015, 8),
            (2015, 11),
            (2015, 12),
            (2016, 3),
            (2016, 4),
            (2016, 5),
            (2016, 6),
            (2016, 7),
            (2016, 8),
            (2016, 9),
            (2016, 10),
            (2016, 11),
            (2016, 12)  
        ]
    }
}

# Carpeta raíz
CARPETA_RAIZ = r""   # << así como tú lo pusiste


def convertir_archivos():
    print("==========================================")
    print("   🚀 CONVERSOR RINEX (.d → .o)")
    print("==========================================\n")

    for station, cfg in CONFIGURACION.items():

        for year, month in cfg["periods"]:
            folder_name = f"{year}_{month:02d}"

            # carpeta como tú la tenías antes:
            folder_path = os.path.join(f"{year}", folder_name, "Observation_Navegation")

            print(f"\n📂 Buscando en: {folder_path}")

            if not os.path.isdir(folder_path):
                print("   ⚠️ Carpeta no existe, saltando...\n")
                continue

            # Recorrer SOLO esta carpeta
            for file in os.listdir(folder_path):

                # Archivos que terminan en .??d (ejemplo: 15d, 16d)
                if file.lower().endswith("d") and len(file.split(".")) == 2:
                    entrada = os.path.join(folder_path, file)

                    # Cambiar la extensión final d → o
                    salida = entrada[:-1] + "o"

                    print(f"🔍 Detectado: {entrada}")
                    print(f"➡️  Generando: {salida}")

                    comando = [
                        CONVBIN_PATH,
                        entrada
                    ]

                    try:
                        subprocess.run(comando, check=True)
                        print("   ✅ Conversión exitosa\n")
                    except subprocess.CalledProcessError:
                        print("   ❌ Error ejecutando crx2rnx\n")

    print("==========================================")
    print("🏁 PROCESO COMPLETADO")
    print("==========================================")


if __name__ == "__main__":
    convertir_archivos()


   🚀 CONVERSOR RINEX (.d → .o)


📂 Buscando en: 2013\2013_01\Observation_Navegation

📂 Buscando en: 2013\2013_02\Observation_Navegation

📂 Buscando en: 2013\2013_03\Observation_Navegation

📂 Buscando en: 2013\2013_04\Observation_Navegation
🔍 Detectado: 2013\2013_04\Observation_Navegation\lhyo0960.13d
➡️  Generando: 2013\2013_04\Observation_Navegation\lhyo0960.13o
   ✅ Conversión exitosa

🔍 Detectado: 2013\2013_04\Observation_Navegation\lhyo0970.13d
➡️  Generando: 2013\2013_04\Observation_Navegation\lhyo0970.13o
   ✅ Conversión exitosa

🔍 Detectado: 2013\2013_04\Observation_Navegation\lhyo0980.13d
➡️  Generando: 2013\2013_04\Observation_Navegation\lhyo0980.13o
   ✅ Conversión exitosa

🔍 Detectado: 2013\2013_04\Observation_Navegation\lhyo0990.13d
➡️  Generando: 2013\2013_04\Observation_Navegation\lhyo0990.13o
   ✅ Conversión exitosa

🔍 Detectado: 2013\2013_04\Observation_Navegation\lhyo1000.13d
➡️  Generando: 2013\2013_04\Observation_Navegation\lhyo1000.13o
   ✅ Conversión exitosa

🔍 Det

## Descargando archivos complementarios .AAn

In [30]:
import os
import requests
import calendar
import subprocess  # <--- Nuevo: Para llamar a 7-Zip
from datetime import datetime
from requests.auth import HTTPBasicAuth
from urllib.parse import urlparse

# ==========================================
# 🔐 CREDENCIALES NASA EARTHDATA
# ==========================================
EARTHDATA_USERNAME = "fabricioestrada" 
EARTHDATA_PASSWORD = "Fabricio270604&"

# ==========================================
# 🛠️ CONFIGURACIÓN DE DESCOMPRESIÓN (WINDOWS)
# ==========================================
# Si tienes 7-Zip instalado, pon la ruta aquí. Si no, déjalo vacío "".
RUTA_7ZIP = r"C:\Program Files\7-Zip\7z.exe" 

# ==========================================
# CONFIGURACIÓN
# ==========================================
CONFIGURACION = {
    "HUANCAYO": {
        "code": "lhyo",
        "periods": [
            (2013, 1),
            (2013, 2),
            (2013, 3),
            (2013, 4),
            (2013, 5),
            (2013, 6),
            (2013, 7),
            (2013, 8),
            (2013, 11),
            (2013, 12),
            (2014, 4),
            (2014, 5),
            (2014, 6),
            (2014, 7),
            (2014, 8),
            (2014, 9),
            (2014, 10),
            (2014, 11),
            (2014, 12),
            (2015, 1),
            (2015, 2),
            (2015, 4),
            (2015, 5),
            (2015, 6),
            (2015, 7),
            (2015, 8),
            (2015, 11),
            (2015, 12),
            (2016, 3),
            (2016, 4),
            (2016, 5),
            (2016, 6),
            (2016, 7),
            (2016, 8),
            (2016, 9),
            (2016, 10),
            (2016, 11),
            (2016, 12)  
        ]
    }
}

CARPETA_RAIZ = "" 

# ==========================================
# 🛠️ CLASE ESPECIAL PARA ARREGLAR LOGIN NASA
# ==========================================
class SessionWithHeaderRedirection(requests.Session):
    """
    Sesión personalizada que MANTIENE la autenticación cuando NASA
    redirige de cddis.nasa.gov a urs.earthdata.nasa.gov
    """
    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        url = prepared_request.url
        if 'Authorization' in headers:
            original_parsed = urlparse(response.request.url)
            redirect_parsed = urlparse(url)
            # Si seguimos dentro de dominios nasa.gov, mantenemos la contraseña
            if (original_parsed.hostname != redirect_parsed.hostname) and \
               redirect_parsed.hostname.endswith('.nasa.gov') and \
               original_parsed.hostname.endswith('.nasa.gov'):
                return
        return super().rebuild_auth(prepared_request, response)

# ==========================================
# FUNCIONES AUXILIARES
# ==========================================
def calcular_doy(year, month, day):
    return datetime(year, month, day).timetuple().tm_yday

def construir_nombre_archivo(year, month, day):
    doy = calcular_doy(year, month, day)
    year_short = str(year)[-2:]
    return f"brdc{doy:03d}0.{year_short}n.Z"

def crear_sesion_earthdata():
    # Usamos nuestra clase especial en lugar de requests.Session()
    session = SessionWithHeaderRedirection()
    session.auth = HTTPBasicAuth(EARTHDATA_USERNAME, EARTHDATA_PASSWORD)
    return session

def descomprimir_archivo(ruta_archivo_z):
    """
    Intenta descomprimir un archivo .Z usando 7-Zip.
    Retorna True si tuvo éxito.
    """
    if not os.path.exists(RUTA_7ZIP):
        return False # No se configuró 7-Zip o no existe

    try:
        # Comando: 7z e archivo.Z -o"carpeta_destino" -y
        # e: extract, -o: output dir, -y: yes to all overwrite
        carpeta_destino = os.path.dirname(ruta_archivo_z)
        cmd = [RUTA_7ZIP, "e", ruta_archivo_z, f"-o{carpeta_destino}", "-y"]
        
        # Ejecutar silenciosamente
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except Exception as e:
        print(f"      ⚠️  Error descomprimiendo: {e}")
        return False

# ==========================================
# LÓGICA DE DESCARGA
# ==========================================
def descargar_dataset(config, session):
    periodos = config["periods"]

    for year, month in periodos:
        # Ruta corregida según tu estructura
        folder_path = os.path.join(CARPETA_RAIZ, f"{year}", f"{year}_{month:02d}", "Observation_Navegation")
        os.makedirs(folder_path, exist_ok=True)
        
        _, num_days = calendar.monthrange(year, month)
        
        print(f"\n>>> 📅 Procesando {year}-{month:02d} ({num_days} días)")
        print(f"    📂 Destino: {folder_path}")
        
        descargados = 0
        errores = 0
        existentes = 0
        descomprimidos = 0
        
        for day in range(1, num_days + 1):
            filename_z = construir_nombre_archivo(year, month, day)
            filename_final = filename_z[:-2] # Quitar .Z (ej: brdc...n)
            
            doy = calcular_doy(year, month, day)
            year_short = str(year)[-2:]
            
            base_url = f"https://cddis.nasa.gov/archive/gnss/data/daily/{year}/{doy:03d}/{year_short}n/"
            url = base_url + filename_z
            local_path_z = os.path.join(folder_path, filename_z)
            local_path_final = os.path.join(folder_path, filename_final)

            # Verificar si ya existe el DESCOMPRIMIDO o el Z
            if os.path.exists(local_path_final):
                if os.path.getsize(local_path_final) > 1000:
                    existentes += 1
                    continue
            elif os.path.exists(local_path_z):
                # Si existe el Z pero no el final, intentamos descomprimir
                if descomprimir_archivo(local_path_z):
                    os.remove(local_path_z) # Borrar el .Z para limpiar
                    existentes += 1
                    continue
            
            print(f"   ⬇️  Descargando: {filename_z} ... ", end="")

            try:
                # stream=True es importante para archivos grandes
                r = session.get(url, stream=True, timeout=30)
                
                content_type = r.headers.get('Content-Type', '').lower()
                
                # Si recibimos HTML, es que falló el login (NASA mandó la página de error)
                if 'html' in content_type:
                    print("⛔ Error de Autenticación (Revisa credenciales)")
                    errores += 1
                    break 
                
                if r.status_code == 200:
                    with open(local_path_z, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                    
                    # Intentar descomprimir inmediatamente
                    if descomprimir_archivo(local_path_z):
                        print("✅ OK (Descomprimido)")
                        os.remove(local_path_z) # Borrar el .Z
                        descomprimidos += 1
                    else:
                        print("✅ OK (Solo .Z)")
                    
                    descargados += 1
                elif r.status_code == 404:
                    print("❌ 404 (No existe)")
                    errores += 1
                else:
                    print(f"⚠️  Estado {r.status_code}")
                    errores += 1
                    
            except Exception as e:
                print(f"⚠️  Error: {e}")
                errores += 1

        print(f"   Resumen: ✅ Nuevos: {descargados} | 📦 Descomprimidos: {descomprimidos} | ⏭️ Listos: {existentes} | ❌ Errores: {errores}")

# ==========================================
# EJECUCIÓN
# ==========================================
if __name__ == "__main__":
    print("="*60)
    print("🚀 DESCARGADOR AUTOMÁTICO NASA CDDIS (NAV BRDC)")
    print("="*60)
    
    if "TU_USUARIO" in EARTHDATA_USERNAME:
        print("\n⚠️  ALERTA: Edita el archivo y pon tu usuario/contraseña real.")
    else:
        print("🔐 Iniciando sesión segura...")
        session = crear_sesion_earthdata()
        
        # Verificar 7-Zip
        if os.path.exists(RUTA_7ZIP):
            print(f"🛠️  7-Zip detectado en: {RUTA_7ZIP} (Se activará descompresión)")
        else:
            print(f"⚠️  7-Zip NO encontrado en: {RUTA_7ZIP} (Solo se descargarán .Z)")
        
        for nombre, conf in CONFIGURACION.items():
            descargar_dataset(conf, session)
            
        print("\n🏁 PROCESO FINALIZADO")

🚀 DESCARGADOR AUTOMÁTICO NASA CDDIS (NAV BRDC)
🔐 Iniciando sesión segura...
🛠️  7-Zip detectado en: C:\Program Files\7-Zip\7z.exe (Se activará descompresión)

>>> 📅 Procesando 2013-01 (31 días)
    📂 Destino: 2013\2013_01\Observation_Navegation
   ⬇️  Descargando: brdc0010.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0020.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0030.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0040.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0050.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0060.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0070.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0080.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0090.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0100.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0110.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0120.13n.Z ... ✅ OK (Descomprimido)
   ⬇️  Descargando: brdc0130.13n.Z 

## Generador de Targets

### Generador de archivos .csv a partir de archivos .o .n 

Para la obtención de parámetros .o.pos.stat de manera parametrizada se tiene que usar la herramienta RTKLib en su versión en Linux, se deja adjuntado los scripts utilizados para sus creaciones.
* Esta etapa es importante para discriminar Cintilación de Multipath

In [ ]:
# Ejecutar en la ruta correspondiente [LINUX]
# Archivo basic.conf
# rtkpost options (2010/08/07 09:24:29, v.2.4.0)

"""
pos1-posmode       =single     # (0:single,1:dgps,2:kinematic,3:static,4:movingbase,5:fixed,6:ppp-kine,7:ppp-static)
pos1-frequency     =l1+l2      # (1:l1,2:l1+l2,3:l1+l2+l5)
pos1-soltype       =forward    # (0:forward,1:backward,2:combined)
pos1-elmask        =15         # (deg)
pos1-snrmask       =0          # (dBHz)
pos1-dynamics      =off        # (0:off,1:on)
pos1-tidecorr      =off        # (0:off,1:on)
pos1-ionoopt       =brdc       # (0:off,1:brdc,2:sbas,3:dual-freq,4:est-stec)
pos1-tropopt       =saas       # (0:off,1:saas,2:sbas,3:est-ztd,4:est-ztdgrad)
pos1-sateph        =brdc       # (0:brdc,1:precise,2:brdc+sbas,3:brdc+ssrapc,4:brdc+ssrcom)
pos1-exclsats      =           # (prn ...)
pos1-navsys        =1          # (1:gps+2:sbas+4:glo+8:gal+16:qzs+32:comp)
pos2-armode        =continuous # (0:off,1:continous,2:instantaneous,3:fix-and-hold)
pos2-gloarmode     =on         # (0:off,1:on,2:autocal)
pos2-arthres       =3
pos2-arlockcnt     =0
pos2-arelmask      =0          # (deg)
pos2-aroutcnt      =5
pos2-arminfix      =10
pos2-slipthres     =0.05       # (m)
pos2-maxage        =30         # (s)
pos2-rejionno      =30         # (m)
pos2-niter         =1
pos2-baselen       =0          # (m)
pos2-basesig       =0          # (m)
out-solformat      =llh        # (0:llh,1:xyz,2:enu,3:nmea)
out-outhead        =on         # (0:off,1:on)
out-outopt         =on         # (0:off,1:on)
out-timesys        =utc       # (0:gpst,1:utc,2:jst)
out-timeform       =hms        # (0:tow,1:hms)
out-timendec       =3
out-degform        =deg        # (0:deg,1:dms)
out-fieldsep       =
out-height         =ellipsoidal # (0:ellipsoidal,1:geodetic)
out-geoid          =internal   # (0:internal,1:egm96,2:egm08_2.5,3:egm08_1,4:gsi2000)
out-solstatic      =all     # (0:all,1:single)
out-nmeaintv1      =0          # (s)
out-nmeaintv2      =0          # (s)
out-outstat        =residual        # (0:off,1:state,2:residual)
stats-errratio     =100
stats-errphase     =0.003      # (m)
stats-errphaseel   =0.003      # (m)
stats-errphasebl   =0          # (m/10km)
stats-errdoppler   =10         # (Hz)
stats-stdbias      =30         # (m)
stats-stdiono      =0.03       # (m)
stats-stdtrop      =0.3        # (m)
stats-prnaccelh    =10          # (m/s^2)
stats-prnaccelv    =10        # (m/s^2)
stats-prnbias      =0.0001     # (m)
stats-prniono      =0.001      # (m)
stats-prntrop      =0.0001     # (m)
stats-clkstab      =5e-12      # (s/s)
ant1-postype       =llh        # (0:llh,1:xyz,2:single,3:posfile,4:rinexhead,5:rtcm)
ant1-pos1          =90          # (deg|m)
ant1-pos2          =0          # (deg|m)
ant1-pos3          =-6335367.6285          # (m|m)
ant1-anttype       =
ant1-antdele       =0          # (m)
ant1-antdeln       =0          # (m)
ant1-antdelu       =0          # (m)
ant2-postype       =llh     # (0:llh,1:xyz,2:single,3:posfile,4:rinexhead,5:rtcm)
ant2-pos1          =90          # (deg|m)
ant2-pos2          =0          # (deg|m)
ant2-pos3          =-6335367.6285          # (m|m)
ant2-anttype       =
ant2-antdele       =0          # (m)
ant2-antdeln       =0          # (m)
ant2-antdelu       =0          # (m)
misc-timeinterp    =off         # (0:off,1:on)
misc-sbasatsel     =0          # (0:all)
file-satantfile    =
file-rcvantfile    =
file-staposfile    =
file-geoidfile     =
file-dcbfile       =
file-tempdir       =
file-geexefile     =
file-solstatfile   =
file-tracefile     =
"""

In [ ]:
# Ejecutar en la ruta correspondiente [LINUX]

# Archivo Shell para parametrizar la conversión

"""
#!/bin/bash

ROOT_DIR="./"
CONF_FILE="basic.conf"

# Busca todos los Observation_Navegation del árbol
find "$ROOT_DIR" -type d -name "Observation_Navegation" | while read navdir; do
    echo "⏳ Procesando carpeta: $navdir"

    # Carpeta padre (ej: 2013/2013_09)
    parent=$(dirname "$navdir")

    # Crear carpeta de salida
    outdir="$parent/Archivos_Procesados_Az_El"
    mkdir -p "$outdir"

    # Encontrar archivos .??o
    for obs in "$navdir"/*.??o; do
        
        # Verifica existencia real
        [ -e "$obs" ] || continue

        # Extraer cuerpo base (ej: lhyo2440)
        base=$(basename "$obs" .??o)

        # Año en sufijo (ej: 13)
        year_suffix=$(echo "$obs" | sed -E 's/.*\.([0-9][0-9])o/\1/')

        # Buscar navegación del mismo DOY y año
        nav="$navdir/brdc${base:4:3}0.$year_suffix"n

        if [ ! -f "$nav" ]; then
            echo "⚠ No se encontró navegación para: $obs"
            continue
        fi

        doy=${base:4:3}
        echo "➡ Día $doy Año .$year_suffix"

        salida="$outdir/${base}.pos"
        estad="$outdir/${base}.pos.stat"

        # Ejecutar RTKLIB
        rnx2rtkp -k "$CONF_FILE" "$obs" "$nav" -o "$salida" -s "$estad"

        echo "✔ Guardado: $salida"

    done

done
"""

Transformando a csv entendible SOLO POSICION AZIMUTH Y ELEVACION:
* input: .<AA>o.pos.stat
* output: archivo.<AA>o.pos.csv

In [1]:
import csv
import os
from datetime import datetime, timedelta

# ==========================================
# ⚙️  CONFIGURACIÓN DE CARPETAS A PROCESAR
# ==========================================

# Lista de tuplas (AÑO, [MESES])
# El script buscará la estructura: AÑO/AÑO_MES/Archivos_Procesados_Az_El
TARGET_BATCHES = [
    ("2013", ["01", "02", "03", "04", "05", "06", "07", "08", "11", "12"]),              # Todos los meses menos Sept y Oct
    ("2014", ["04", "05", "06", "07", "08", "09", "10", "11", "12"]),        # Todos los meses menos Ene, Feb, Mar
    ("2015", ["01", "02", "04", "05", "06", "07", "08", "11", "12"]),        # Todos los meses menos Marzo, Sept, Oct
    ("2016", ["03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])               # Todos los meses menos Enero, Feb
]

# Directorio Raíz (donde están las carpetas 2013, 2014, etc.)
ROOT_DIR = "." 

# ==========================================
# 🔧 LÓGICA DEL SCRIPT
# ==========================================

def obtener_leap_seconds(year, month):
    """
    Determina los segundos bisiestos correctos según la fecha.
    - Antes del 1 Julio 2015: 16 segundos.
    - Desde 1 Julio 2015 hasta 31 Dic 2016: 17 segundos.
    - (2017 en adelante serían 18, pero no está en tu lista).
    """
    y = int(year)
    m = int(month)
    
    # Lógica del cambio en Julio 2015
    if y < 2015:
        return 16
    elif y == 2015:
        if m < 7: # Enero a Junio
            return 16
        else:     # Julio a Diciembre
            return 17
    elif y >= 2016:
        return 17
    
    return 16 # Default por seguridad

def gps_week_seconds_to_utc(week, seconds, leap_seconds):
    gps_epoch = datetime(1980, 1, 6, 0, 0, 0)
    elapsed = timedelta(weeks=int(week), seconds=float(seconds))
    gps_time = gps_epoch + elapsed
    utc_time = gps_time - timedelta(seconds=leap_seconds)
    return utc_time.strftime('%Y/%m/%d %H:%M:%S.%f')[:-3]

def procesar_archivo_stat(input_file, output_csv, leap_seconds):
    headers = ['TIME_UTC', 'PRN', 'AZIMUTH', 'ELEVATION']
    datos_procesados = []
    lineas_procesadas = 0

    try:
        with open(input_file, 'r') as f:
            for line in f:
                if line.startswith('$SAT'):
                    parts = line.split(',')
                    try:
                        week = parts[1]
                        tow = parts[2]
                        prn = parts[3]
                        azimuth = parts[5]
                        elevation = parts[6]
                        
                        time_utc = gps_week_seconds_to_utc(week, tow, leap_seconds)
                        datos_procesados.append([time_utc, prn, azimuth, elevation])
                        lineas_procesadas += 1
                    except IndexError:
                        continue 

        if lineas_procesadas > 0:
            with open(output_csv, 'w', newline='') as f_out:
                writer = csv.writer(f_out)
                writer.writerow(headers)
                writer.writerows(datos_procesados)
            return True 
        else:
            return False 

    except Exception as e:
        print(f"❌ Error leyendo {input_file}: {e}")
        return False

def main():
    print(f"🚀 INICIANDO PROCESO MULTI-ANUAL")
    print("-" * 60)

    total_files = 0

    for year, months in TARGET_BATCHES:
        for month in months:
            # Construir nombres de carpetas y patrones
            # Ej: folder_name = "2013_09"
            folder_name = f"{year}_{month}"
            
            # Ruta: ./2013/2013_09/Archivos_Procesados_Az_El
            target_path = os.path.join(ROOT_DIR, year, folder_name, "Archivos_Procesados_Az_El")
            
            # Patrón de archivo: .13o.pos.stat
            year_suffix = year[-2:]
            file_pattern = f".{year_suffix}o.pos.stat"
            
            # Calcular Leap Seconds dinámicamente
            current_leap = obtener_leap_seconds(year, month)

            print(f"\n📂 Analizando: {year}/{folder_name}")
            print(f"   ⚙️  Leap Seconds: {current_leap}s | Patrón: {file_pattern}")

            if not os.path.exists(target_path):
                print(f"   ⚠️  Ruta no encontrada: {target_path}")
                continue

            # Procesar archivos en esa carpeta específica
            batch_count = 0
            for file in os.listdir(target_path):
                if file.endswith(file_pattern):
                    full_input = os.path.join(target_path, file)
                    
                    # Output CSV
                    output_name = file.replace(".stat", ".csv")
                    full_output = os.path.join(target_path, output_name)
                    
                    exito = procesar_archivo_stat(full_input, full_output, current_leap)
                    if exito:
                        batch_count += 1
                        print(f"     ✅ {file} -> CSV")
                    else:
                        print(f"     ⚠️  {file} (Vacío/Error)")
            
            print(f"   ∑ Procesados en este mes: {batch_count}")
            total_files += batch_count

    print("-" * 60)
    print(f"🏁 FIN DEL PROCESO. Total archivos generados: {total_files}")

if __name__ == "__main__":
    main()

🚀 INICIANDO PROCESO MULTI-ANUAL
------------------------------------------------------------

📂 Analizando: 2013/2013_01
   ⚙️  Leap Seconds: 16s | Patrón: .13o.pos.stat
   ∑ Procesados en este mes: 0

📂 Analizando: 2013/2013_02
   ⚙️  Leap Seconds: 16s | Patrón: .13o.pos.stat
   ∑ Procesados en este mes: 0

📂 Analizando: 2013/2013_03
   ⚙️  Leap Seconds: 16s | Patrón: .13o.pos.stat
   ∑ Procesados en este mes: 0

📂 Analizando: 2013/2013_04
   ⚙️  Leap Seconds: 16s | Patrón: .13o.pos.stat
     ✅ lhyo0960.13o.pos.stat -> CSV
     ✅ lhyo0970.13o.pos.stat -> CSV
     ✅ lhyo0980.13o.pos.stat -> CSV
     ✅ lhyo0990.13o.pos.stat -> CSV
     ✅ lhyo1000.13o.pos.stat -> CSV
     ✅ lhyo1010.13o.pos.stat -> CSV
     ✅ lhyo1020.13o.pos.stat -> CSV
     ✅ lhyo1030.13o.pos.stat -> CSV
     ✅ lhyo1040.13o.pos.stat -> CSV
     ✅ lhyo1050.13o.pos.stat -> CSV
     ✅ lhyo1060.13o.pos.stat -> CSV
     ✅ lhyo1070.13o.pos.stat -> CSV
     ✅ lhyo1080.13o.pos.stat -> CSV
     ✅ lhyo1090.13o.pos.stat -> CSV
  

Convirtiendo a CSV los .xxo para matchear después por fecha

In [2]:
import csv
import os
import math
from datetime import datetime, timedelta

# ==========================================
# ⚙️  CONFIGURACIÓN DE CARPETAS A PROCESAR
# ==========================================

# AÑO, [MESES]
TARGET_BATCHES = [
    ("2013", ["01", "02", "03", "04", "05", "06", "07", "08", "11", "12"]),              # Todos los meses menos Sept y Oct
    ("2014", ["04", "05", "06", "07", "08", "09", "10", "11", "12"]),        # Todos los meses menos Ene, Feb, Mar
    ("2015", ["01", "02", "04", "05", "06", "07", "08", "11", "12"]),        # Todos los meses menos Marzo, Sept, Oct
    ("2016", ["03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])               # Todos los meses menos Enero, Feb
]

ROOT_DIR = "." 

# ==========================================
# 🔧 LÓGICA DEL SCRIPT
# ==========================================

def obtener_leap_seconds(year, month):
    y = int(year)
    m = int(month)
    if y < 2015: return 16
    elif y == 2015: return 16 if m < 7 else 17
    elif y >= 2016: return 17
    return 16

def parse_rinex_obs_file(input_file, output_csv, leap_seconds):
    """
    Parsea un archivo de observación RINEX v2.10 buscando S1 y S2.
    """
    headers_csv = ['TIME_UTC', 'PRN', 'S1', 'S2']
    datos_procesados = []
    
    # Indices por defecto (basados en tu ejemplo: L1, L2, C1, P2, S1, S2)
    # S1 es el índice 4 (5ta posición), S2 es el índice 5 (6ta posición)
    idx_s1 = 4 
    idx_s2 = 5 
    num_observables = 6 

    try:
        with open(input_file, 'r') as f:
            lines = f.readlines()

        i = 0
        header_end = False
        
        # 1. Leer Header para confirmar posición de S1 y S2 (Opcional pero recomendado)
        while i < len(lines):
            line = lines[i]
            if "TYPES OF OBSERV" in line:
                # Ejemplo: 6    L1    L2    C1    P2    S1    S2
                parts = line[:60].split()
                try:
                    num_observables = int(parts[0])
                    types = parts[1:]
                    if "S1" in types: idx_s1 = types.index("S1")
                    if "S2" in types: idx_s2 = types.index("S2")
                except:
                    pass # Si falla, usamos los defaults de tu ejemplo
            
            if "END OF HEADER" in line:
                header_end = True
                i += 1
                break
            i += 1

        if not header_end: return False

        # Calcular cuántas líneas ocupa cada satélite
        # RINEX v2 pone max 5 observaciones por línea.
        lines_per_sat = math.ceil(num_observables / 5.0)

        # 2. Leer Cuerpo del archivo (Observaciones)
        while i < len(lines):
            line = lines[i]
            
            # Detectar línea de época (Año Mes Dia Hora Min Seg ...)
            # Las líneas de época en v2 tienen el año en los primeros caracteres
            if len(line) > 30 and ("G" in line or "R" in line or "E" in line):
                try:
                    # Parsear Fecha GPS
                    yy = int(line[1:3])
                    mm = int(line[4:6])
                    dd = int(line[7:9])
                    hh = int(line[10:12])
                    min_ = int(line[13:15])
                    sec = float(line[15:26])
                    
                    # Ajustar siglo (RINEX suele usar 2 dígitos)
                    year_full = 2000 + yy if yy < 80 else 1900 + yy
                    
                    # Convertir a Objeto Datetime (Tiempo GPS)
                    gps_dt = datetime(year_full, mm, dd, hh, min_, 0) + timedelta(seconds=sec)
                    
                    # Convertir a UTC
                    utc_dt = gps_dt - timedelta(seconds=leap_seconds)
                    time_utc_str = utc_dt.strftime('%Y/%m/%d %H:%M:%S.%f')[:-3]

                    # Parsear Número de Satélites y Lista de PRNs
                    # Bandera de época (0=OK) está en col 26-28, Num Sats en 29-32
                    num_sats = int(line[29:32])
                    
                    # Obtener lista de PRNs (ej: G02G05...)
                    # Si hay mas de 12 satélites, continúan en la siguiente línea (pendiente manejar simple)
                    prn_str = line[32:].strip()
                    
                    # Si num_sats > 12, la lista de PRNs sigue en la siguiente línea
                    # (Lógica simplificada asumiendo < 12 sats por ahora como en tu ejemplo)
                    if len(prn_str) / 3 < num_sats:
                         i += 1
                         prn_str += lines[i].strip()

                    prn_list = [prn_str[k:k+3] for k in range(0, len(prn_str), 3)]
                    
                    i += 1 # Moverse a las líneas de datos

                    # Iterar por cada satélite en esta época
                    for prn in prn_list:
                        # Recopilar todas las observaciones del satélite en una sola lista
                        obs_values = []
                        
                        # Leer las líneas necesarias para este satélite
                        for _ in range(lines_per_sat):
                            if i >= len(lines): break
                            obs_line = lines[i]
                            # RINEX v2: cada dato son 16 caracteres (14 valor + 2 flags)
                            # Cortamos chunks de 16
                            for k in range(0, len(obs_line.rstrip()), 16):
                                val_str = obs_line[k:k+14].strip()
                                obs_values.append(val_str)
                            i += 1
                        
                        # Extraer S1 y S2 según los índices detectados
                        val_s1 = obs_values[idx_s1] if idx_s1 < len(obs_values) and obs_values[idx_s1] else ""
                        val_s2 = obs_values[idx_s2] if idx_s2 < len(obs_values) and obs_values[idx_s2] else ""
                        
                        datos_procesados.append([time_utc_str, prn, val_s1, val_s2])

                except Exception as e:
                    # Si falla parsear una línea de época, avanzamos una para intentar recuperar
                    # print(f"Error en linea {i}: {e}") 
                    i += 1
            else:
                i += 1

        # Escribir CSV
        if datos_procesados:
            with open(output_csv, 'w', newline='') as f_out:
                writer = csv.writer(f_out)
                writer.writerow(headers_csv)
                writer.writerows(datos_procesados)
            return True
        return False

    except Exception as e:
        print(f"❌ Error fatal leyendo {input_file}: {e}")
        return False

def main():
    print(f"🚀 INICIANDO PROCESO OBS RINEX (S1/S2)")
    print("-" * 60)

    total_files = 0

    for year, months in TARGET_BATCHES:
        for month in months:
            folder_name = f"{year}_{month}"
            
            # ⚠️ CAMBIO IMPORTANTE: Ahora buscamos en 'Observation_Navegation'
            target_path = os.path.join(ROOT_DIR, year, folder_name, "Observation_Navegation")
            
            # Patrón RINEX Observación: termina en .YYo (ej: .13o, .14o)
            year_suffix = year[-2:]
            ext_pattern = f".{year_suffix}o" 
            
            current_leap = obtener_leap_seconds(year, month)

            print(f"\n📂 Analizando: {year}/{folder_name}/Observation_Navegation")
            
            if not os.path.exists(target_path):
                print(f"   ⚠️  Ruta no encontrada: {target_path}")
                continue

            batch_count = 0
            for file in os.listdir(target_path):
                # Filtramos que termine en .13o, .14o, etc.
                if file.endswith(ext_pattern):
                    full_input = os.path.join(target_path, file)
                    
                    # Output CSV (ej: lhyo2480.13o -> lhyo2480_obs.csv)
                    output_name = file + "_obs.csv"
                    full_output = os.path.join(target_path, output_name)
                    
                    exito = parse_rinex_obs_file(full_input, full_output, current_leap)
                    if exito:
                        batch_count += 1
                        print(f"     ✅ {file} -> CSV (S1/S2 extraídos)")
                    else:
                        print(f"     ⚠️  {file} (Error o vacío)")
            
            print(f"   ∑ Procesados: {batch_count}")
            total_files += batch_count

    print("-" * 60)
    print(f"🏁 FIN. Total archivos OBS procesados: {total_files}")

if __name__ == "__main__":
    main()

🚀 INICIANDO PROCESO OBS RINEX (S1/S2)
------------------------------------------------------------

📂 Analizando: 2013/2013_01/Observation_Navegation
   ∑ Procesados: 0

📂 Analizando: 2013/2013_02/Observation_Navegation
   ∑ Procesados: 0

📂 Analizando: 2013/2013_03/Observation_Navegation
   ∑ Procesados: 0

📂 Analizando: 2013/2013_04/Observation_Navegation
     ✅ lhyo0960.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo0970.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo0980.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo0990.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1000.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1010.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1020.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1030.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1040.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1050.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1060.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1070.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1080.13o -> CSV (S1/S2 extraídos)
     ✅ lhyo1090.13o -> CSV (S1/S2 extraídos)
     ✅ lhy

Uniendo ambos CSV...

TIME_UTC es cambiado a time (TAMBIEN EN UTC)

In [3]:
import csv
import os
from datetime import datetime, timedelta

# ==========================================
# ⚙️  CONFIGURACIÓN DE CARPETAS A PROCESAR
# ==========================================

TARGET_BATCHES = [
    ("2013", ["01", "02", "03", "04", "05", "06", "07", "08", "11", "12"]),              # Todos los meses menos Sept y Oct
    ("2014", ["04", "05", "06", "07", "08", "09", "10", "11", "12"]),        # Todos los meses menos Ene, Feb, Mar
    ("2015", ["01", "02", "04", "05", "06", "07", "08", "11", "12"]),        # Todos los meses menos Marzo, Sept, Oct
    ("2016", ["03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])               # Todos los meses menos Enero, Feb
]

ROOT_DIR = "."

# ==========================================
# 🔧 LÓGICA DE FUSIÓN
# ==========================================

def redondear_tiempo_utc(time_str):
    """
    Toma un string de tiempo (ej: 2013/09/01 23:59:43.834)
    y lo redondea al SEGUNDO más cercano para corregir desfases
    de milisegundos entre RINEX y RTKLIB.
    """
    try:
        # Formato esperado del script anterior
        dt = datetime.strptime(time_str, '%Y/%m/%d %H:%M:%S.%f')
        
        # Si los microsegundos son > 500000, sumamos un segundo
        if dt.microsecond >= 500000:
            dt += timedelta(seconds=1)
        
        # Retornamos el tiempo limpio sin decimales para el match
        return dt.replace(microsecond=0).strftime('%Y/%m/%d %H:%M:%S')
    except ValueError:
        # Si ya viene sin decimales o formato raro
        return time_str.split('.')[0]

def cargar_datos_obs(archivo_obs):
    """
    Carga el archivo OBS en un diccionario para búsqueda rápida.
    Clave: (tiempo_redondeado, prn) -> Valor: (s1, s2)
    """
    datos = {}
    with open(archivo_obs, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            t_raw = row['TIME_UTC']
            prn = row['PRN']
            s1 = row['S1']
            s2 = row['S2']
            
            t_round = redondear_tiempo_utc(t_raw)
            
            # Guardamos en diccionario
            datos[(t_round, prn)] = (s1, s2)
    return datos

def fusionar_archivos(archivo_obs, archivo_pos, archivo_salida):
    """
    Combina OBS y POS basándose en Tiempo (redondeado) y PRN.
    """
    # 1. Cargar OBS en memoria (suele ser más ligero o igual)
    print("      📥 Cargando datos OBS en memoria...", end="")
    mapa_obs = cargar_datos_obs(archivo_obs)
    print(f" OK ({len(mapa_obs)} registros)")

    encabezados_finales = ['time', 'prn', 'azimut', 'elevacion', 'S1', 'S2']
    
    registros_fusionados = 0

    # 2. Leer POS línea por línea y buscar coincidencia en OBS
    with open(archivo_pos, 'r') as f_in, open(archivo_salida, 'w', newline='') as f_out:
        reader = csv.DictReader(f_in)
        writer = csv.writer(f_out)
        
        writer.writerow(encabezados_finales)
        
        for row in reader:
            t_pos_raw = row['TIME_UTC']
            prn = row['PRN']
            az = row['AZIMUTH']
            el = row['ELEVATION']
            
            # Redondear tiempo del POS para matchear con OBS
            t_pos_round = redondear_tiempo_utc(t_pos_raw)
            
            # 3. VERIFICAR MATCH (Intersección)
            clave = (t_pos_round, prn)
            
            if clave in mapa_obs:
                # ¡Bingo! Tenemos datos en ambos lados
                s1, s2 = mapa_obs[clave]
                
                # Escribir fila combinada
                writer.writerow([t_pos_round, prn, az, el, s1, s2])
                registros_fusionados += 1
            else:
                # Si no hay match (falta el PRN en obs o el tiempo no cuadra), se ignora.
                pass

    return registros_fusionados

def main():
    print("🚀 INICIANDO FUSIÓN DE DATOS (COLLAPSE)")
    print("-" * 60)
    
    total_archivos = 0

    for year, months in TARGET_BATCHES:
        for month in months:
            folder_name = f"{year}_{month}"
            
            # Rutas de entrada
            dir_obs = os.path.join(ROOT_DIR, year, folder_name, "Observation_Navegation")
            dir_pos = os.path.join(ROOT_DIR, year, folder_name, "Archivos_Procesados_Az_El")
            
            # Ruta de salida (Guardaremos los procesados en la carpeta del MES para orden)
            # Ej: ./2013/2013_09/lhyoXXXX_procesado.csv
            dir_salida = os.path.join(ROOT_DIR, year, folder_name)

            print(f"\n📂 Procesando carpeta: {folder_name}")

            if not os.path.exists(dir_obs) or not os.path.exists(dir_pos):
                print("   ⚠️  Faltan carpetas de origen. Saltando.")
                continue

            # Buscar archivos POS (Az/El) como base para iterar
            archivos_pos = [f for f in os.listdir(dir_pos) if f.endswith(".pos.csv")]

            for f_pos in archivos_pos:
                # Nombre base esperado: lhyo2440.13o.pos.csv -> Base: lhyo2440.13o
                # Estrategia: quitar ".pos.csv" del final
                base_name = f_pos.replace(".pos.csv", "")
                
                # Nombre del archivo OBS correspondiente: base_name + "_obs.csv"
                # (Según el script anterior: lhyo2440.13o_obs.csv)
                f_obs = f"{base_name}_obs.csv"
                
                path_pos = os.path.join(dir_pos, f_pos)
                path_obs = os.path.join(dir_obs, f_obs)
                
                # Nombre final colapsado
                # Limpiar extensión del año para que quede lhyo2440_procesado.csv
                # base_name es lhyo2440.13o -> split('.') -> lhyo2440
                nombre_limpio = base_name.split('.')[0] 
                f_salida = f"{nombre_limpio}_procesado.csv"
                path_salida = os.path.join(dir_salida, f_salida)

                if os.path.exists(path_obs):
                    print(f"   ⚡ Fusionando: {base_name} ...")
                    
                    matches = fusionar_archivos(path_obs, path_pos, path_salida)
                    
                    print(f"      ✅ Generado: {f_salida} (Filas: {matches})")
                    total_archivos += 1
                else:
                    print(f"      ❌ No se encontró pareja OBS para: {f_pos}")

    print("-" * 60)
    print(f"🏁 FIN DEL PROCESO. Archivos finales generados: {total_archivos}")

if __name__ == "__main__":
    main()

🚀 INICIANDO FUSIÓN DE DATOS (COLLAPSE)
------------------------------------------------------------

📂 Procesando carpeta: 2013_01

📂 Procesando carpeta: 2013_02

📂 Procesando carpeta: 2013_03

📂 Procesando carpeta: 2013_04
   ⚡ Fusionando: lhyo0960.13o ...
      📥 Cargando datos OBS en memoria... OK (23940 registros)
      ✅ Generado: lhyo0960_procesado.csv (Filas: 21303)
   ⚡ Fusionando: lhyo0970.13o ...
      📥 Cargando datos OBS en memoria... OK (74328 registros)
      ✅ Generado: lhyo0970_procesado.csv (Filas: 65330)
   ⚡ Fusionando: lhyo0980.13o ...
      📥 Cargando datos OBS en memoria... OK (48392 registros)
      ✅ Generado: lhyo0980_procesado.csv (Filas: 42807)
   ⚡ Fusionando: lhyo0990.13o ...
      📥 Cargando datos OBS en memoria... OK (77551 registros)
      ✅ Generado: lhyo0990_procesado.csv (Filas: 68470)
   ⚡ Fusionando: lhyo1000.13o ...
      📥 Cargando datos OBS en memoria... OK (77616 registros)
      ✅ Generado: lhyo1000_procesado.csv (Filas: 68480)
   ⚡ Fusionando:

### Generación de Índice de S4

In [ ]:
"""
CALCULADOR DE ÍNDICE S4 (SIN SLANT RANGE)
1. Lee archivos *_procesado.csv generados previamente.
2. Aplica la lógica de S4 por ventanas de 60 segundos.
3. Guarda en carpeta 'TargetFinal'.
"""

import pandas as pd
import numpy as np
import os
import glob

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES
# ==========================================
TARGET_BATCHES = [
    ("2013", ["01", "02", "03", "04", "05", "06", "07", "08", "11", "12"]),              # Todos los meses menos Sept y Oct
    ("2014", ["04", "05", "06", "07", "08", "09", "10", "11", "12"]),        # Todos los meses menos Ene, Feb, Mar
    ("2015", ["01", "02", "04", "05", "06", "07", "08", "11", "12"]),        # Todos los meses menos Marzo, Sept, Oct
    ("2016", ["03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])               # Todos los meses menos Enero, Feb
]

ROOT_DIR = "."

# ==========================================
# 🧠 LÓGICA DE CÁLCULO S4
# ==========================================

def calcular_s4_ventana(grupo):
    """
    Calcula el índice S4 para una ventana de 60 segundos.
    S4 = sqrt(<I²> - <I>²) / <I>
    """
    # S1 está en dB-Hz, convertir a escala lineal
    S1_dBHz = pd.to_numeric(grupo['S1'], errors='coerce').values
    S1_dBHz = S1_dBHz[~np.isnan(S1_dBHz)]
    
    if len(S1_dBHz) == 0:
        return None 

    intensidad = 10 ** (S1_dBHz / 10)
    
    # Metadatos para el retorno
    meta_time = grupo['time'].iloc[0]
    meta_prn = grupo['prn'].iloc[0]
    meta_az = grupo['azimut'].mean()
    meta_el = grupo['elevacion'].mean()
    meta_s2 = pd.to_numeric(grupo['S2'], errors='coerce').mean()
    # (Slant range eliminado)

    # Si hay menos de 3 puntos, no calculamos S4 pero devolvemos promedios
    if len(intensidad) < 3: 
        return pd.Series({
            'time': meta_time,
            'prn': meta_prn,
            'azimut': meta_az,
            'elevacion': meta_el,
            'S1_mean': np.nan,
            'S2_mean': meta_s2,
            'S4': np.nan,
            'num_muestras': len(intensidad)
        })
    
    # Calcular S4
    I_mean = np.mean(intensidad)
    I_squared_mean = np.mean(intensidad ** 2)
    
    if I_mean > 0:
        S4 = np.sqrt(max(I_squared_mean - I_mean**2, 0)) / I_mean
    else:
        S4 = np.nan
    
    return pd.Series({
        'time': meta_time,
        'prn': meta_prn,
        'azimut': meta_az,
        'elevacion': meta_el,
        'S1_mean': I_mean,
        'S2_mean': meta_s2,
        'S4': S4,
        'num_muestras': len(intensidad)
    })

def procesar_archivo_individual(archivo_path):
    try:
        df = pd.read_csv(archivo_path)
        if df.empty: return None

        # 1. Convertir tiempo y numéricos
        df['time'] = pd.to_datetime(df['time'])
        cols_num = ['azimut', 'elevacion', 'S1', 'S2']
        for c in cols_num:
            df[c] = pd.to_numeric(df[c], errors='coerce')
        
        df = df.dropna(subset=['S1', 'elevacion']) 

        # 2. Crear ventanas de 60 segundos
        df['time_window'] = df['time'].dt.floor('1min')
        
        # 3. Agrupar y Aplicar Lógica S4
        df_s4 = df.groupby(['time_window', 'prn']).apply(calcular_s4_ventana)
        
        if df_s4 is None or df_s4.empty: return None
            
        # Limpieza post-groupby
        df_s4 = df_s4.reset_index(drop=True)
        df_s4 = df_s4.rename(columns={'time': 'time_window'})
        
        return df_s4
        
    except Exception as e:
        print(f"      ❌ Error procesando {os.path.basename(archivo_path)}: {e}")
        return None

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 CALCULADOR S4 MASIVO (SIN SLANT RANGE)")
    print("="*70)

    for year, months in TARGET_BATCHES:
        for month in months:
            folder_name = f"{year}_{month}"
            
            # Ruta origen (archivos _procesado.csv)
            source_folder = os.path.join(ROOT_DIR, year, folder_name)
            # Carpeta destino
            target_final_folder = os.path.join(source_folder, "TargetFinal")
            
            print(f"\n📂 Analizando carpeta: {folder_name}")
            
            if not os.path.exists(source_folder):
                print(f"   ⚠️  No existe: {source_folder}")
                continue

            # Buscar archivos _procesado.csv
            patron = os.path.join(source_folder, "*_procesado.csv")
            archivos = sorted(glob.glob(patron))
            
            if not archivos:
                print("   ⚠️  No hay archivos '_procesado.csv'.")
                continue
            
            print(f"   📊 Encontrados: {len(archivos)} archivos.")

            os.makedirs(target_final_folder, exist_ok=True)
            resultados_mes = []

            for archivo in archivos:
                df_resultado = procesar_archivo_individual(archivo)
                if df_resultado is not None and not df_resultado.empty:
                    resultados_mes.append(df_resultado)

            # Consolidar Mes
            if resultados_mes:
                print(f"   📥 Consolidando resultados...")
                df_final = pd.concat(resultados_mes, ignore_index=True)
                df_final = df_final.sort_values(['time_window', 'prn'])
                
                output_filename = f"huancayo_{year}_{month}_S4_Calculado.csv"
                output_path = os.path.join(target_final_folder, output_filename)
                
                df_final.to_csv(output_path, index=False)
                
                print(f"   ✅ GUARDADO: {output_path}")
                print(f"      Filas totales: {len(df_final)}")
                
                s4_vals = df_final['S4'].dropna()
                if not s4_vals.empty:
                    print(f"      Eventos S4 > 0.5: {(s4_vals > 0.5).sum()}")
            else:
                print("   ❌ Sin resultados válidos este mes.")

    print("\n" + "="*70)
    print("🏁 PROCESO FINAL TERMINADO")

if __name__ == "__main__":
    main()

🚀 CALCULADOR S4 MASIVO (SIN SLANT RANGE)

📂 Analizando carpeta: 2014_01
   📊 Encontrados: 30 archivos.


C:\Users\User\AppData\Local\Temp\ipykernel_22620\2902373313.py:101: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_s4 = df.groupby(['time_window', 'prn']).apply(calcular_s4_ventana)
C:\Users\User\AppData\Local\Temp\ipykernel_22620\2902373313.py:101: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_s4 = df.groupby(['time_window', 'prn']).apply(calcular_s4_ventana)
C:\Users\User\AppData\Local\Temp\ipykernel_22620\290

   📥 Consolidando resultados...
   ✅ GUARDADO: .\2014\2014_01\TargetFinal\huancayo_2014_01_S4_Calculado.csv
      Filas totales: 319264
      Eventos S4 > 0.5: 112

📂 Analizando carpeta: 2014_02
   📊 Encontrados: 28 archivos.


C:\Users\User\AppData\Local\Temp\ipykernel_22620\2902373313.py:101: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_s4 = df.groupby(['time_window', 'prn']).apply(calcular_s4_ventana)
C:\Users\User\AppData\Local\Temp\ipykernel_22620\2902373313.py:101: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_s4 = df.groupby(['time_window', 'prn']).apply(calcular_s4_ventana)
C:\Users\User\AppData\Local\Temp\ipykernel_22620\290

   📥 Consolidando resultados...
   ✅ GUARDADO: .\2014\2014_02\TargetFinal\huancayo_2014_02_S4_Calculado.csv
      Filas totales: 269247
      Eventos S4 > 0.5: 322

📂 Analizando carpeta: 2014_03
   📊 Encontrados: 27 archivos.


C:\Users\User\AppData\Local\Temp\ipykernel_22620\2902373313.py:101: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_s4 = df.groupby(['time_window', 'prn']).apply(calcular_s4_ventana)
C:\Users\User\AppData\Local\Temp\ipykernel_22620\2902373313.py:101: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_s4 = df.groupby(['time_window', 'prn']).apply(calcular_s4_ventana)
C:\Users\User\AppData\Local\Temp\ipykernel_22620\290

   📥 Consolidando resultados...
   ✅ GUARDADO: .\2014\2014_03\TargetFinal\huancayo_2014_03_S4_Calculado.csv
      Filas totales: 227083
      Eventos S4 > 0.5: 654

🏁 PROCESO FINAL TERMINADO


### Salida del Target

Depende de todos los anterioes, parametrizado

In [ ]:
"""
CALCULADOR DE TARGET (BINARIO) DE CENTELLEO
1. Lee los archivos S4 calculados en el paso anterior.
2. Convierte a Hora Perú (UTC-5).
3. Filtra ventana nocturna (19:00 - 23:59).
4. Determina si hubo centelleo (S4 > 0.4) por día.
5. Genera columna 'FECHA_PERU' y guarda en TargetFinal.
"""

import pandas as pd
import numpy as np
import os
import glob

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES
# ==========================================
TARGET_BATCHES = [
    ("2013", ["01", "02", "03", "04", "05", "06", "07", "08", "11", "12"]),              # Todos los meses menos Sept y Oct
    ("2014", ["04", "05", "06", "07", "08", "09", "10", "11", "12"]),        # Todos los meses menos Ene, Feb, Mar
    ("2015", ["01", "02", "04", "05", "06", "07", "08", "11", "12"]),        # Todos los meses menos Marzo, Sept, Oct
    ("2016", ["03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])               # Todos los meses menos Enero, Feb
]

ROOT_DIR = "."
UMBRAL_S4 = 0.4  # Criterio para definir centelleo

# ==========================================
# 🧠 LÓGICA DE TARGET
# ==========================================

def calcular_target_diario(input_file, umbral=0.4, output_folder=None):
    print(f"\n📄 Procesando: {os.path.basename(input_file)}")
    
    # 1. CARGAR DATOS
    try:
        df = pd.read_csv(input_file)
    except Exception as e:
        print(f"   ❌ Error leyendo archivo: {e}")
        return None

    if df.empty:
        print("   ⚠️  Archivo vacío.")
        return None

    # 2. CONVERTIR UTC A HORA PERÚ
    # El archivo anterior tiene 'time_window' en UTC
    df['dt_utc'] = pd.to_datetime(df['time_window'])
    
    # Restar 5 horas
    df['DATETIME_PERU'] = df['dt_utc'] - pd.Timedelta(hours=5)
    
    # Extraer componentes para filtrado y agrupación
    df['FECHA_PERU'] = df['DATETIME_PERU'].dt.date
    df['HORA_PERU'] = df['DATETIME_PERU'].dt.hour
    
    # 3. FILTRAR VENTANA NOCTURNA (19:00 - 23:59 LT)
    # Filtramos horas mayores o iguales a 19. (19, 20, 21, 22, 23)
    df_noche = df[df['HORA_PERU'] >= 19].copy()
    
    if df_noche.empty:
        print("   ⚠️  No hay datos en horario nocturno (19-24h).")
        return None

    # 4. CALCULAR TARGET POR DÍA (FECHA_PERU)
    resultados = []
    fechas_unicas = sorted(df_noche['FECHA_PERU'].unique())
    
    for fecha in fechas_unicas:
        df_dia = df_noche[df_noche['FECHA_PERU'] == fecha]
        
        # Validar que S4 no sea NaN
        df_dia_valido = df_dia.dropna(subset=['S4'])
        
        if df_dia_valido.empty:
            continue
            
        s4_max = df_dia_valido['S4'].max()
        s4_mean = df_dia_valido['S4'].mean()
        
        # LOGICA PRINCIPAL DEL TARGET
        # Si el máximo S4 del día supera el umbral, Target = 1
        target = 1 if s4_max > umbral else 0
        
        # Estadísticas adicionales
        veces_supero = len(df_dia_valido[df_dia_valido['S4'] > umbral])
        
        # Datos del pico
        idx_max = df_dia_valido['S4'].idxmax()
        prn_pico = df_dia_valido.loc[idx_max, 'prn']
        hora_pico = df_dia_valido.loc[idx_max, 'DATETIME_PERU']

        resultados.append({
            'FECHA_PERU': fecha,
            'TARGET': target,
            'S4_MAX': round(s4_max, 4),
            'S4_MEAN': round(s4_mean, 4),
            'VECES_SUPERO_UMBRAL': veces_supero,
            'TOTAL_MEDICIONES': len(df_dia_valido),
            'PRN_PICO': prn_pico,
            'HORA_PICO': hora_pico
        })

    df_target = pd.DataFrame(resultados)
    
    # 5. GUARDAR RESULTADOS
    if not df_target.empty:
        # Generar nombres de archivo
        base_name = os.path.basename(input_file).replace('.csv', '')
        
        # Archivo Resumen (Simple)
        file_simple = os.path.join(output_folder, f"{base_name}_Target_Simple.csv")
        df_target[['FECHA_PERU', 'TARGET', 'S4_MAX']].to_csv(file_simple, index=False)
        
        # Archivo Completo (Detallado)
        file_full = os.path.join(output_folder, f"{base_name}_Target_Completo.csv")
        df_target.to_csv(file_full, index=False)
        
        print(f"   ✅ Generado: {os.path.basename(file_simple)}")
        print(f"      Días con Centelleo (S4 > {umbral}): {df_target['TARGET'].sum()} de {len(df_target)}")
        
        return df_target
    else:
        print("   ⚠️  No se generaron resultados diarios.")
        return None

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*80)
    print(f"🚀 GENERADOR DE TARGET (S4 > {UMBRAL_S4}) - HORA PERÚ")
    print("="*80)

    total_dias_analizados = 0
    total_dias_centelleo = 0

    for year, months in TARGET_BATCHES:
        for month in months:
            folder_name = f"{year}_{month}"
            
            # Buscar en la carpeta TargetFinal donde guardamos el paso anterior
            target_final_path = os.path.join(ROOT_DIR, year, folder_name, "TargetFinal")
            
            if not os.path.exists(target_final_path):
                print(f"\n⚠️  Saltando {folder_name}: No existe carpeta TargetFinal")
                continue
            
            # Buscar el archivo calculado del paso anterior
            # Patrón: huancayo_2013_09_S4_Calculado.csv
            patron = os.path.join(target_final_path, "*_S4_Calculado.csv")
            archivos = glob.glob(patron)
            
            if not archivos:
                print(f"\n⚠️  En {folder_name}: No se encontró archivo S4 Calculado.")
                continue

            print(f"\n📂 Analizando mes: {folder_name}")
            
            for archivo in archivos:
                df_res = calcular_target_diario(archivo, UMBRAL_S4, target_final_path)
                
                if df_res is not None:
                    total_dias_analizados += len(df_res)
                    total_dias_centelleo += df_res['TARGET'].sum()

    print("\n" + "="*80)
    print("🏁 ESTADÍSTICAS GLOBALES DEL PROCESO")
    print("="*80)
    print(f"Total Días Procesados: {total_dias_analizados}")
    print(f"Total Días con Centelleo (Target=1): {total_dias_centelleo}")
    if total_dias_analizados > 0:
        print(f"Porcentaje Global: {(total_dias_centelleo/total_dias_analizados)*100:.2f}%")
    print("="*80)

if __name__ == "__main__":
    main()

🚀 GENERADOR DE TARGET (S4 > 0.4) - HORA PERÚ

📂 Analizando mes: 2014_01

📄 Procesando: huancayo_2014_01_S4_Calculado.csv
   ✅ Generado: huancayo_2014_01_S4_Calculado_Target_Simple.csv
      Días con Centelleo (S4 > 0.4): 27 de 28

📂 Analizando mes: 2014_02

📄 Procesando: huancayo_2014_02_S4_Calculado.csv
   ✅ Generado: huancayo_2014_02_S4_Calculado_Target_Simple.csv
      Días con Centelleo (S4 > 0.4): 24 de 27

📂 Analizando mes: 2014_03

📄 Procesando: huancayo_2014_03_S4_Calculado.csv
   ✅ Generado: huancayo_2014_03_S4_Calculado_Target_Simple.csv
      Días con Centelleo (S4 > 0.4): 24 de 26

🏁 ESTADÍSTICAS GLOBALES DEL PROCESO
Total Días Procesados: 81
Total Días con Centelleo (Target=1): 75
Porcentaje Global: 92.59%


## Generador de Inputs

### Variables Ionosféricas

A partir del siguiente Link se obtiene los valores que serán usados como Inputs:
https://giro.uml.edu/didbase/scaled.php

1. Select Time Interval:
* Start: 2013-09-01 00:00
* Stop: 2013-10-31 23:59
2. Pick one GIRO Location:
* (JI91J) Jicamarca
3. Select Data to Download:
* foF2
* hF
* hmF2
* B0
4. Search!

Script Adaptado para Procesamiento por Lotes (GIRO Data)

In [ ]:
import pandas as pd
from datetime import timedelta
import os
import glob
from pathlib import Path

# ==========================================
# ⚙️  SECCIÓN DE CONFIGURACIÓN (PARAMETRIZADA)
# ==========================================

# Mapeo de meses de 3 letras a su número de 2 dígitos (para verificación/filtro)
MONTH_MAPPING = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'May': '05', 
    'Jun': '06', 'Jul': '07', 'Ago': '08', 'Sept': '09', 'Oct': '10', 
    'Nov': '11', 'Dic': '12'
}

# Lotes a procesar (Configuración flexible)
TARGET_BATCHES = {
    "2013": ["2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2013_Nov_Dic_DIDBase.txt"],
    "2014": ["2014_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"], 
    "2015": ["2015_Ene_Feb_DIDBase.txt", "2015_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2015_Nov_Dic_DIDBase.txt"],
    "2016": ["2016_Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"]
}

ROOT_INPUT_DIR = "Inputs"
OUTPUT_SUFFIX = "_procesado.csv"

# ==========================================
# 🔧 LÓGICA DE PROCESAMIENTO ORIGINAL (CONSERVADA)
# ==========================================

def process_giro_data(input_file):
    """
    Procesa datos ionosféricos de GIRO (JICAMARCA), aplica filtros y ajusta a hora local.
    (Función original adaptada para no manejar el output_file, ya lo hacemos en el main)
    """
    
    # 1. CARGAR EL ARCHIVO
    # Leer el archivo, saltando las líneas de encabezado (comentarios con #)
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    # Encontrar la línea donde empiezan los datos (después del encabezado de columnas)
    data_start_idx = 0
    for i, line in enumerate(lines):
        if line.startswith('#Time'):
            data_start_idx = i
            break
    
    # Cargar datos usando pandas
    df = pd.read_csv(
        input_file,
        skiprows=data_start_idx,
        delim_whitespace=True,      # Usar espacios como delimitador
        na_values=['---', '__', '/_']  # Valores que representan datos faltantes
    )
    
    # Limpiar nombres de columnas (remover # si existe)
    df.columns = df.columns.str.replace('#', '')
    
    # Remover columnas 'QD' (Quality Descriptor)
    columns_to_keep = [col for col in df.columns if col != 'QD']
    df = df[columns_to_keep].copy() # Usar copy() para evitar SettingWithCopyWarning
    
    
    # 2. CONVERTIR LA COLUMNA 'Time' A DATETIME (UTC)
    df['Time'] = pd.to_datetime(df['Time'], format='ISO8601', utc=True)
    
    
    # 3. APLICAR FILTRO DE CALIDAD (CS >= 70)
    df_filtered = df[df['CS'] >= 70].copy()
    
    
    # 4. AJUSTAR A HORA LOCAL PERÚ (LT = UTC - 5 horas)
    df_filtered['Time_LT'] = df_filtered['Time'] - timedelta(hours=5)
    
    # Reorganizar columnas y renombrar
    cols = ['Time', 'Time_LT', 'CS', 'foF2', 'hF', 'hmF2', 'B0']
    df_filtered = df_filtered[cols]
    
    df_filtered = df_filtered.rename(columns={
        'Time': 'Time_UTC',
        'Time_LT': 'Time_Local_Peru'
    })
    
    return df_filtered


# ==========================================
# 🚀 EJECUCIÓN PRINCIPAL Y PROCESAMIENTO POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 PROCESAMIENTO DE DATOS GIRO POR LOTES")
    print("="*70)

    total_archivos_procesados = 0

    if not os.path.exists(ROOT_INPUT_DIR):
        print(f"❌ Error: El directorio raíz de entrada '{ROOT_INPUT_DIR}' no existe.")
        return

    # Iterar sobre los años definidos en TARGET_BATCHES
    for year, filenames in TARGET_BATCHES.items():
        
        if not filenames:
            print(f"⚠️  Saltando año {year}: No hay archivos definidos en TARGET_BATCHES.")
            continue
            
        print(f"\n===== [AÑO {year}] Procesando {len(filenames)} archivo(s) =====")
        
        # Ruta de la carpeta del año (ej: Inputs/2013)
        year_folder = os.path.join(ROOT_INPUT_DIR, year)
        
        for filename in filenames:
            
            input_file_path = os.path.join(year_folder, filename)
            
            if not os.path.exists(input_file_path):
                print(f"   ❌ No encontrado: {filename}")
                continue
            
            print(f"   ⚡ Procesando: {filename}...")
            
            try:
                # 1. Procesar datos
                df_processed = process_giro_data(input_file_path)
                
                # 2. Definir ruta y guardar
                output_file = input_file_path.replace('.txt', OUTPUT_SUFFIX)
                df_processed.to_csv(output_file, index=False, date_format='%Y-%m-%d %H:%M:%S')
                
                print(f"      ✅ Guardado: {os.path.basename(output_file)}")
                total_archivos_procesados += 1

            except Exception as e:
                print(f"      ❌ ERROR en {filename}: {e}")

    print("\n" + "="*70)
    print(f"🏁 PROCESO DE GIRO FINALIZADO. Archivos procesados: {total_archivos_procesados}")
    print("="*70)

if __name__ == "__main__":
    main()

🚀 PROCESAMIENTO DE DATOS GIRO POR LOTES

===== [AÑO 2013] Procesando 1 archivo(s) =====
   ⚡ Procesando: 2013_Sept_Oct_DIDBase.txt...


C:\Users\User\AppData\Local\Temp\ipykernel_14480\2809495674.py:52: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


      ✅ Guardado: 2013_Sept_Oct_DIDBase_procesado.csv
⚠️  Saltando año 2014: No hay archivos definidos en TARGET_BATCHES.

===== [AÑO 2015] Procesando 2 archivo(s) =====
   ⚡ Procesando: 2015_Mar_DIDBase.txt...


C:\Users\User\AppData\Local\Temp\ipykernel_14480\2809495674.py:52: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\User\AppData\Local\Temp\ipykernel_14480\2809495674.py:52: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


      ✅ Guardado: 2015_Mar_DIDBase_procesado.csv
   ⚡ Procesando: 2015_Sept_Oct_DIDBase.txt...
      ✅ Guardado: 2015_Sept_Oct_DIDBase_procesado.csv

===== [AÑO 2016] Procesando 1 archivo(s) =====
   ⚡ Procesando: 2016_Ene_Feb_DIDBase.txt...


C:\Users\User\AppData\Local\Temp\ipykernel_14480\2809495674.py:52: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


      ✅ Guardado: 2016_Ene_Feb_DIDBase_procesado.csv

🏁 PROCESO DE GIRO FINALIZADO. Archivos procesados: 4


Creando DataSet para 1 sola ventana (no se divide):
* Fecha en Horario Perú

In [ ]:
import pandas as pd
import numpy as np
import os

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES (Parametrizada)
# ==========================================

# Usamos los mismos nombres base de los .txt originales para mantener el orden,
# pero el script buscará automáticamente la versión "_procesado.csv"
TARGET_BATCHES = {
    "2013": ["2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2013_Nov_Dic_DIDBase.txt"],
    "2014": ["2014_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"], 
    "2015": ["2015_Ene_Feb_DIDBase.txt", "2015_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2015_Nov_Dic_DIDBase.txt"],
    "2016": ["2016_Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"]
}

ROOT_INPUT_DIR = "Inputs"

# ==========================================
# 🧠 LÓGICA DE CÁLCULO (INTACTA)
# ==========================================

def calcular_inputs_1_ventana_validado(input_file, output_file=None, tolerancia_min=10):
    """
    Calcula inputs con 1 ventana validando extremos con tolerancia estricta
    
    Extremos esperados:
    - 17:00 (tolerancia: ±5 min → acepta 16:55-17:05)
    - 19:00 (tolerancia: ±5 min → acepta 18:55-19:05)
    
    Si algún extremo está FUERA de tolerancia → NaN en TODO
    
    Mv_PRECURSOR se calcula con el tiempo REAL entre extremos encontrados
    
    INPUTS EXTENDIDOS foF2 (para 1 ventana):
    - ΔfoF2_1: foF2_fin - foF2_inicio
    - slope_foF2_1: ΔfoF2_1 / Δt (MHz/min)
    - foF2_max_SW: Máximo foF2 en toda la ventana
    - foF2_min_SW: Mínimo foF2 (solo si >=3 muestras)
    - foF2_drop_SW: foF2_max - foF2_min (solo si >=3 muestras)
    """
    
    print("="*70)
    print(f"PROCESANDO: {os.path.basename(input_file)}")
    print("DATASET 1: INPUTS CON VALIDACIÓN ESTRICTA + INPUTS EXTENDIDOS foF2")
    print("Ventana: 17:00-19:00 LT")
    print(f"Tolerancia: ±{tolerancia_min} min en CADA extremo")
    print("="*70)
    
    # 1. CARGAR DATOS
    print("\n[1/4] Cargando datos...")
    df = pd.read_csv(input_file)
    df['Time_Local_Peru'] = pd.to_datetime(df['Time_Local_Peru'])
    df['FECHA'] = df['Time_Local_Peru'].dt.date
    df['HORA_DECIMAL'] = df['Time_Local_Peru'].dt.hour + df['Time_Local_Peru'].dt.minute / 60.0
    
    print(f"✓ Registros cargados: {len(df)}")
    
    # 2. DEFINIR EXTREMOS ESPERADOS
    print("\n[2/4] Definiendo extremos esperados...")
    
    hora_inicio_esperada = 17.0   # 17:00
    hora_fin_esperada = 19.0      # 19:00
    
    tolerancia_decimal = tolerancia_min / 60.0
    
    print(f"  Extremo INICIO: {hora_inicio_esperada:.2f} LT (±{tolerancia_min} min)")
    print(f"    Rango aceptable: {hora_inicio_esperada - tolerancia_decimal:.2f} - {hora_inicio_esperada + tolerancia_decimal:.2f} LT")
    print(f"  Extremo FIN: {hora_fin_esperada:.2f} LT (±{tolerancia_min} min)")
    print(f"    Rango aceptable: {hora_fin_esperada - tolerancia_decimal:.2f} - {hora_fin_esperada + tolerancia_decimal:.2f} LT")
    
    # 3. CALCULAR INPUTS
    print("\n[3/4] Calculando inputs con validación...")
    
    fechas_unicas = sorted(df['FECHA'].unique())
    resultados = []
    
    dias_rechazados_por_tolerancia = 0
    
    for fecha in fechas_unicas:
        df_dia = df[df['FECHA'] == fecha].copy()
        fila = {'FECHA': fecha}
        
        # Buscar todos los registros del día en ventana amplia
        df_ventana = df_dia[
            (df_dia['HORA_DECIMAL'] >= hora_inicio_esperada - tolerancia_decimal) &
            (df_dia['HORA_DECIMAL'] <= hora_fin_esperada + tolerancia_decimal)
        ].sort_values('HORA_DECIMAL').copy()
        
        if len(df_ventana) >= 2:
            # ✅ PASO 1: BUSCAR registro más cercano al INICIO (17:00)
            df_ventana['dist_inicio'] = abs(df_ventana['HORA_DECIMAL'] - hora_inicio_esperada)
            idx_inicio = df_ventana['dist_inicio'].idxmin()
            registro_inicio = df_ventana.loc[idx_inicio]
            
            # ✅ PASO 2: BUSCAR registro más cercano al FIN (19:00)
            df_ventana['dist_fin'] = abs(df_ventana['HORA_DECIMAL'] - hora_fin_esperada)
            idx_fin = df_ventana['dist_fin'].idxmin()
            registro_fin = df_ventana.loc[idx_fin]
            
            # ✅ PASO 3: CALCULAR deltas (en minutos)
            delta_inicio_min = (registro_inicio['HORA_DECIMAL'] - hora_inicio_esperada) * 60
            delta_fin_min = (registro_fin['HORA_DECIMAL'] - hora_fin_esperada) * 60
            
            # ✅ PASO 4: VALIDAR tolerancia ESTRICTA
            inicio_valido = abs(delta_inicio_min) <= tolerancia_min
            fin_valido = abs(delta_fin_min) <= tolerancia_min
            extremos_validos = inicio_valido and fin_valido
            
            if extremos_validos:
                # ✅✅ AMBOS EXTREMOS DENTRO DE TOLERANCIA: Calcular inputs
                
                hmF2_inicio = registro_inicio['hmF2']
                hmF2_fin = registro_fin['hmF2']
                
                # Dh_TOTAL
                Dh_TOTAL = hmF2_fin - hmF2_inicio
                
                # ⚡ Mv_PRECURSOR: Calcular con tiempo REAL (no fijo)
                tiempo_real_min = (registro_fin['Time_Local_Peru'] - 
                                   registro_inicio['Time_Local_Peru']).total_seconds() / 60.0
                Mv_PRECURSOR = Dh_TOTAL / tiempo_real_min
                
                # Datos entre extremos
                df_entre_extremos = df_ventana[
                    (df_ventana['HORA_DECIMAL'] >= registro_inicio['HORA_DECIMAL']) &
                    (df_ventana['HORA_DECIMAL'] <= registro_fin['HORA_DECIMAL'])
                ]
                
                # Mh_PRECURSOR: Máximo entre los extremos válidos
                Mh_PRECURSOR = df_entre_extremos['hmF2'].max()
                
                # ========================================
                # NUEVOS INPUTS EXTENDIDOS foF2
                # ========================================
                
                if 'foF2' in df_entre_extremos.columns:
                    foF2_validos = df_entre_extremos['foF2'].dropna()
                    num_muestras_foF2 = len(foF2_validos)
                    
                    # F_DENSITY: Promedio entre extremos
                    F_DENSITY = foF2_validos.mean() if num_muestras_foF2 > 0 else np.nan
                    
                    # ΔfoF2_1: Diferencia entre extremos
                    foF2_inicio = registro_inicio.get('foF2', np.nan)
                    foF2_fin = registro_fin.get('foF2', np.nan)
                    
                    if pd.notna(foF2_inicio) and pd.notna(foF2_fin):
                        Delta_foF2_1 = foF2_fin - foF2_inicio
                        # slope_foF2_1: Pendiente (MHz/min)
                        slope_foF2_1 = Delta_foF2_1 / tiempo_real_min if tiempo_real_min > 0 else np.nan
                    else:
                        Delta_foF2_1 = np.nan
                        slope_foF2_1 = np.nan
                    
                    # foF2_max_SW: Máximo en toda la ventana
                    foF2_max_SW = foF2_validos.max() if num_muestras_foF2 > 0 else np.nan
                    
                    # foF2_min y foF2_drop: Solo si hay >=3 muestras
                    if num_muestras_foF2 >= 3:
                        foF2_min_SW = foF2_validos.min()
                        foF2_drop_SW = foF2_max_SW - foF2_min_SW
                    else:
                        foF2_min_SW = np.nan
                        foF2_drop_SW = np.nan
                    
                else:
                    F_DENSITY = np.nan
                    Delta_foF2_1 = np.nan
                    slope_foF2_1 = np.nan
                    foF2_max_SW = np.nan
                    foF2_min_SW = np.nan
                    foF2_drop_SW = np.nan
                    num_muestras_foF2 = 0
                
                hora_inicio_real = registro_inicio['Time_Local_Peru']
                hora_fin_real = registro_fin['Time_Local_Peru']
                tiempo_real_usado = tiempo_real_min
                
            else:
                # ❌ EXTREMOS FUERA DE TOLERANCIA: TODO NaN
                Dh_TOTAL = np.nan
                Mv_PRECURSOR = np.nan
                Mh_PRECURSOR = np.nan
                F_DENSITY = np.nan
                Delta_foF2_1 = np.nan
                slope_foF2_1 = np.nan
                foF2_max_SW = np.nan
                foF2_min_SW = np.nan
                foF2_drop_SW = np.nan
                num_muestras_foF2 = 0
                
                hora_inicio_real = registro_inicio['Time_Local_Peru'] if 'Time_Local_Peru' in registro_inicio else np.nan
                hora_fin_real = registro_fin['Time_Local_Peru'] if 'Time_Local_Peru' in registro_fin else np.nan
                tiempo_real_usado = np.nan
                
                dias_rechazados_por_tolerancia += 1
            
            num_registros = len(df_entre_extremos) if extremos_validos else len(df_ventana)
            
        elif len(df_ventana) == 1:
            # ⚠️ SOLO 1 REGISTRO: No se puede calcular
            registro = df_ventana.iloc[0]
            
            delta_inicio_min = (registro['HORA_DECIMAL'] - hora_inicio_esperada) * 60
            delta_fin_min = (registro['HORA_DECIMAL'] - hora_fin_esperada) * 60
            extremos_validos = False
            
            Dh_TOTAL = np.nan
            Mv_PRECURSOR = np.nan
            Mh_PRECURSOR = registro['hmF2']
            F_DENSITY = registro.get('foF2', np.nan)
            Delta_foF2_1 = np.nan
            slope_foF2_1 = np.nan
            foF2_max_SW = registro.get('foF2', np.nan)
            foF2_min_SW = np.nan
            foF2_drop_SW = np.nan
            num_muestras_foF2 = 1 if pd.notna(registro.get('foF2', np.nan)) else 0
            
            hora_inicio_real = registro['Time_Local_Peru']
            hora_fin_real = np.nan
            tiempo_real_usado = np.nan
            num_registros = 1
            
        else:
            # ❌ SIN DATOS
            delta_inicio_min = np.nan
            delta_fin_min = np.nan
            extremos_validos = False
            
            Dh_TOTAL = np.nan
            Mv_PRECURSOR = np.nan
            Mh_PRECURSOR = np.nan
            F_DENSITY = np.nan
            Delta_foF2_1 = np.nan
            slope_foF2_1 = np.nan
            foF2_max_SW = np.nan
            foF2_min_SW = np.nan
            foF2_drop_SW = np.nan
            num_muestras_foF2 = 0
            
            hora_inicio_real = np.nan
            hora_fin_real = np.nan
            tiempo_real_usado = np.nan
            num_registros = 0
        
        # Guardar resultados
        fila['Mh_PRECURSOR'] = Mh_PRECURSOR
        fila['Mv_PRECURSOR'] = Mv_PRECURSOR
        fila['Dh_TOTAL'] = Dh_TOTAL
        fila['F_DENSITY'] = F_DENSITY
        fila['Delta_foF2_1'] = Delta_foF2_1
        fila['slope_foF2_1'] = slope_foF2_1
        fila['foF2_max_SW'] = foF2_max_SW
        fila['foF2_min_SW'] = foF2_min_SW
        fila['foF2_drop_SW'] = foF2_drop_SW
        fila['num_muestras_foF2'] = num_muestras_foF2
        fila['delta_inicio_min'] = delta_inicio_min
        fila['delta_fin_min'] = delta_fin_min
        fila['extremos_validos'] = extremos_validos
        fila['hora_inicio_real'] = hora_inicio_real
        fila['hora_fin_real'] = hora_fin_real
        fila['tiempo_real_min'] = tiempo_real_usado
        fila['num_registros'] = num_registros
        
        resultados.append(fila)
    
    df_resultado = pd.DataFrame(resultados)
    
    # 4. ESTADÍSTICAS
    print("\n[4/4] Estadísticas finales...")
    print("\n" + "="*70)
    print("RESUMEN DE VALIDACIÓN")
    print("="*70)
    
    total_dias = len(df_resultado)
    dias_validos = df_resultado['extremos_validos'].sum()
    dias_invalidos = total_dias - dias_validos
    
    print(f"\nTotal de días procesados: {total_dias}")
    print(f"Días con extremos VÁLIDOS: {dias_validos} ({dias_validos/total_dias*100:.1f}%)")
    print(f"Días RECHAZADOS por tolerancia: {dias_rechazados_por_tolerancia} ({dias_rechazados_por_tolerancia/total_dias*100:.1f}%)")
    
    # Estadísticas de deltas
    print("\n--- DELTA TIEMPO EN EXTREMOS ---")
    df_con_datos = df_resultado[df_resultado['delta_inicio_min'].notna()]
    if len(df_con_datos) > 0:
        print(f"\nDelta INICIO (minutos):")
        print(f"  Promedio: {df_con_datos['delta_inicio_min'].mean():.2f} min")
        print(f"  Rango: [{df_con_datos['delta_inicio_min'].min():.2f}, {df_con_datos['delta_inicio_min'].max():.2f}]")
        
        print(f"\nDelta FIN (minutos):")
        print(f"  Promedio: {df_con_datos['delta_fin_min'].mean():.2f} min")
        print(f"  Rango: [{df_con_datos['delta_fin_min'].min():.2f}, {df_con_datos['delta_fin_min'].max():.2f}]")
    
    # Tiempo real usado
    df_con_tiempo = df_resultado[df_resultado['tiempo_real_min'].notna()]
    if len(df_con_tiempo) > 0:
        print(f"\n--- TIEMPO REAL ENTRE EXTREMOS ---")
        print(f"  Promedio: {df_con_tiempo['tiempo_real_min'].mean():.2f} min")
        print(f"  Esperado: 120 min")
        print(f"  Diferencia: {df_con_tiempo['tiempo_real_min'].mean() - 120:.2f} min")
    
    # Estadísticas foF2
    print(f"\n--- ESTADÍSTICAS foF2 ---")
    df_con_foF2 = df_resultado[df_resultado['num_muestras_foF2'] > 0]
    if len(df_con_foF2) > 0:
        print(f"Días con datos foF2: {len(df_con_foF2)}")
        print(f"Muestras promedio por día: {df_con_foF2['num_muestras_foF2'].mean():.1f}")
        
        df_con_drop = df_resultado[df_resultado['foF2_drop_SW'].notna()]
        print(f"Días con foF2_drop calculado (>=3 muestras): {len(df_con_drop)}")
    
    # Inputs calculables
    columnas_inputs_basicos = ['Mh_PRECURSOR', 'Mv_PRECURSOR', 'Dh_TOTAL', 'F_DENSITY']
    columnas_inputs_foF2 = ['Delta_foF2_1', 'slope_foF2_1', 'foF2_max_SW']
    
    dias_completos_basicos = df_resultado[df_resultado[columnas_inputs_basicos].notna().all(axis=1)]
    dias_completos_foF2 = df_resultado[df_resultado[columnas_inputs_foF2].notna().all(axis=1)]
    
    print(f"\n✅ Días con inputs BÁSICOS completos: {len(dias_completos_basicos)} ({len(dias_completos_basicos)/total_dias*100:.1f}%)")
    print(f"✅ Días con inputs foF2 EXTENDIDOS completos: {len(dias_completos_foF2)} ({len(dias_completos_foF2)/total_dias*100:.1f}%)")
    
    # 5. GUARDAR
    if output_file is None:
        output_file = input_file.replace('.csv', '_inputs.csv')
    
    df_resultado.to_csv(output_file, index=False)
    print(f"\n💾 Archivo guardado: {output_file}")
    
    print("\n" + "="*70)
    print("✓ CÁLCULO COMPLETADO")
    print("Mv calculado con tiempo REAL entre extremos")
    print("Tolerancia ESTRICTA: ±5 min en cada extremo")
    print("Inputs foF2 extendidos incluidos")
    print("="*70)
    
    return df_resultado

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 GENERADOR DE INPUTS (GIRO) POR LOTES")
    print("="*70)

    total_archivos = 0

    if not os.path.exists(ROOT_INPUT_DIR):
        print(f"❌ Error: El directorio raíz '{ROOT_INPUT_DIR}' no existe.")
        return

    # Iterar sobre la configuración de años
    for year, filenames in TARGET_BATCHES.items():
        if not filenames:
            continue
            
        print(f"\n===== [AÑO {year}] Analizando carpeta... =====")
        year_folder = os.path.join(ROOT_INPUT_DIR, year)
        
        for base_filename in filenames:
            # Transformamos el nombre base (ej: 2013_Sep_Oct.txt) 
            # al nombre generado en el paso anterior (ej: 2013_Sep_Oct_procesado.csv)
            
            # El input para este script es el output del anterior
            input_filename = base_filename.replace(".txt", "_procesado.csv")
            input_path = os.path.join(year_folder, input_filename)
            
            # Definimos el nombre de salida para este paso
            output_filename = base_filename.replace(".txt", "_inputs.csv")
            output_path = os.path.join(year_folder, output_filename)
            
            if os.path.exists(input_path):
                # Llamar a la función de cálculo (Intacta)
                calcular_inputs_1_ventana_validado(input_path, output_path, tolerancia_min=10)
                total_archivos += 1
            else:
                print(f"⚠️  Saltando: No se encontró {input_filename}")

    print("\n" + "="*70)
    print(f"🏁 PROCESO FINALIZADO. Archivos de inputs generados: {total_archivos}")
    print("="*70)

if __name__ == "__main__":
    main()

🚀 GENERADOR DE INPUTS (GIRO) POR LOTES

===== [AÑO 2013] Analizando carpeta... =====
PROCESANDO: 2013_Sept_Oct_DIDBase_procesado.csv
DATASET 1: INPUTS CON VALIDACIÓN ESTRICTA + INPUTS EXTENDIDOS foF2
Ventana: 17:00-19:00 LT
Tolerancia: ±10 min en CADA extremo

[1/4] Cargando datos...
✓ Registros cargados: 6151

[2/4] Definiendo extremos esperados...
  Extremo INICIO: 17.00 LT (±10 min)
    Rango aceptable: 16.83 - 17.17 LT
  Extremo FIN: 19.00 LT (±10 min)
    Rango aceptable: 18.83 - 19.17 LT

[3/4] Calculando inputs con validación...

[4/4] Estadísticas finales...

RESUMEN DE VALIDACIÓN

Total de días procesados: 59
Días con extremos VÁLIDOS: 31 (52.5%)
Días RECHAZADOS por tolerancia: 26 (44.1%)

--- DELTA TIEMPO EN EXTREMOS ---

Delta INICIO (minutos):
  Promedio: 21.21 min
  Rango: [0.00, 120.00]

Delta FIN (minutos):
  Promedio: -0.86 min
  Rango: [-15.00, 0.00]

--- TIEMPO REAL ENTRE EXTREMOS ---
  Promedio: 119.84 min
  Esperado: 120 min
  Diferencia: -0.16 min

--- ESTADÍSTICAS

Creando DataSet para 2 ventanas:
* Fecha en Horario Perú

In [ ]:
import pandas as pd
import numpy as np
import os

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES (Parametrizada)
# ==========================================

# Mismos archivos base, el script buscará la versión "_procesado.csv"
TARGET_BATCHES = {
    "2013": ["2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2013_Nov_Dic_DIDBase.txt"],
    "2014": ["2014_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"], 
    "2015": ["2015_Ene_Feb_DIDBase.txt", "2015_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2015_Nov_Dic_DIDBase.txt"],
    "2016": ["2016_Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"]
}

ROOT_INPUT_DIR = "Inputs"

# ==========================================
# 🧠 LÓGICA DE CÁLCULO (INTACTA)
# ==========================================

def calcular_inputs_2_ventanas_validado(input_file, output_file=None, tolerancia_min=10):
    """
    Calcula inputs con 2 sub-ventanas validando CADA extremo
    
    Extremos a validar:
    SW1: 17:00 (±5) y 18:00 (±5)
    SW2: 18:00 (±5) y 19:00 (±5)
    
    Si una sub-ventana NO cumple tolerancia → NaN en ESA sub-ventana
    """
    
    print("="*70)
    print(f"PROCESANDO: {os.path.basename(input_file)}")
    print("DATASET 2: 2 SUB-VENTANAS + INPUTS EXTENDIDOS foF2")
    print(f"Tolerancia: ±{tolerancia_min} min en CADA extremo")
    print("="*70)
    
    # 1. CARGAR DATOS
    print("\n[1/4] Cargando datos...")
    df = pd.read_csv(input_file)
    df['Time_Local_Peru'] = pd.to_datetime(df['Time_Local_Peru'])
    df['FECHA'] = df['Time_Local_Peru'].dt.date
    df['HORA_DECIMAL'] = df['Time_Local_Peru'].dt.hour + df['Time_Local_Peru'].dt.minute / 60.0
    
    print(f"✓ Registros cargados: {len(df)}")
    
    # 2. DEFINIR SUB-VENTANAS
    print("\n[2/4] Definiendo sub-ventanas...")
    
    subventanas = [
        {'nombre': 'SW1', 'inicio': 17.0, 'fin': 18.0},
        {'nombre': 'SW2', 'inicio': 18.0, 'fin': 19.0}
    ]
    
    tolerancia_decimal = tolerancia_min / 60.0
    
    for sw in subventanas:
        print(f"  {sw['nombre']}: {sw['inicio']:.2f} - {sw['fin']:.2f} LT")
        print(f"    Extremo inicio: {sw['inicio']:.2f} ±{tolerancia_min} min")
        print(f"    Extremo fin: {sw['fin']:.2f} ±{tolerancia_min} min")
    
    # 3. CALCULAR INPUTS
    print("\n[3/4] Calculando inputs con validación...")
    
    fechas_unicas = sorted(df['FECHA'].unique())
    resultados = []
    
    for fecha in fechas_unicas:
        df_dia = df[df['FECHA'] == fecha].copy()
        fila = {'FECHA': fecha}
        
        # F_DENSITY global (promedio de toda la ventana 17:00-19:00)
        df_ventana_completa = df_dia[
            (df_dia['HORA_DECIMAL'] >= 17.0) &
            (df_dia['HORA_DECIMAL'] <= 19.0)
        ]
        
        if 'foF2' in df_ventana_completa.columns:
            foF2_validos_global = df_ventana_completa['foF2'].dropna()
            F_DENSITY_GLOBAL = foF2_validos_global.mean() if len(foF2_validos_global) > 0 else np.nan
        else:
            F_DENSITY_GLOBAL = np.nan
        
        # Procesar cada sub-ventana
        for idx, sw in enumerate(subventanas, 1):
            # Buscar en ventana amplia
            df_sw = df_dia[
                (df_dia['HORA_DECIMAL'] >= sw['inicio'] - tolerancia_decimal) &
                (df_dia['HORA_DECIMAL'] <= sw['fin'] + tolerancia_decimal)
            ].sort_values('HORA_DECIMAL').copy()

            if len(df_sw) >= 2:
                # BUSCAR extremo INICIO
                df_sw['dist_inicio'] = abs(df_sw['HORA_DECIMAL'] - sw['inicio'])
                idx_inicio = df_sw['dist_inicio'].idxmin()
                registro_inicio = df_sw.loc[idx_inicio]
                
                # BUSCAR extremo FIN
                df_sw['dist_fin'] = abs(df_sw['HORA_DECIMAL'] - sw['fin'])
                idx_fin = df_sw['dist_fin'].idxmin()
                registro_fin = df_sw.loc[idx_fin]
                
                # CALCULAR deltas
                delta_inicio_min = (registro_inicio['HORA_DECIMAL'] - sw['inicio']) * 60
                delta_fin_min = (registro_fin['HORA_DECIMAL'] - sw['fin']) * 60
                
                # VALIDAR tolerancia ESTRICTA
                inicio_valido = abs(delta_inicio_min) <= tolerancia_min
                fin_valido = abs(delta_fin_min) <= tolerancia_min
                extremos_validos = inicio_valido and fin_valido
                
                if extremos_validos:
                    # ✅ CALCULAR con extremos válidos
                    hmF2_inicio = registro_inicio['hmF2']
                    hmF2_fin = registro_fin['hmF2']
                    
                    Δh = hmF2_fin - hmF2_inicio
                    
                    # ⚡ Mv con tiempo REAL
                    tiempo_real_min = (registro_fin['Time_Local_Peru'] - 
                                       registro_inicio['Time_Local_Peru']).total_seconds() / 60.0
                    Mv = Δh / tiempo_real_min
                    
                    # Datos entre extremos
                    df_entre = df_sw[
                        (df_sw['HORA_DECIMAL'] >= registro_inicio['HORA_DECIMAL']) &
                        (df_sw['HORA_DECIMAL'] <= registro_fin['HORA_DECIMAL'])
                    ]
                    
                    # Mh entre extremos
                    Mh = df_entre['hmF2'].max()
                    
                    # ========================================
                    # INPUTS EXTENDIDOS foF2 POR SUB-VENTANA
                    # ========================================
                    
                    if 'foF2' in df_entre.columns:
                        foF2_validos = df_entre['foF2'].dropna()
                        num_muestras_foF2 = len(foF2_validos)
                        
                        # ΔfoF2: Diferencia entre extremos
                        foF2_inicio = registro_inicio.get('foF2', np.nan)
                        foF2_fin = registro_fin.get('foF2', np.nan)
                        
                        if pd.notna(foF2_inicio) and pd.notna(foF2_fin):
                            Delta_foF2 = foF2_fin - foF2_inicio
                            # slope_foF2: Pendiente (MHz/min)
                            slope_foF2 = Delta_foF2 / tiempo_real_min if tiempo_real_min > 0 else np.nan
                        else:
                            Delta_foF2 = np.nan
                            slope_foF2 = np.nan
                        
                        # foF2_max: Máximo en la sub-ventana
                        foF2_max = foF2_validos.max() if num_muestras_foF2 > 0 else np.nan
                        
                        # foF2_min y foF2_drop: Solo si hay >=3 muestras
                        if num_muestras_foF2 >= 3:
                            foF2_min = foF2_validos.min()
                            foF2_drop = foF2_max - foF2_min
                        else:
                            foF2_min = np.nan
                            foF2_drop = np.nan
                        
                    else:
                        Delta_foF2 = np.nan
                        slope_foF2 = np.nan
                        foF2_max = np.nan
                        foF2_min = np.nan
                        foF2_drop = np.nan
                        num_muestras_foF2 = 0
                    
                    tiempo_usado = tiempo_real_min
                    
                else:
                    # ❌ EXTREMOS FUERA DE TOLERANCIA: NaN
                    Δh = np.nan
                    Mv = np.nan
                    Mh = np.nan
                    Delta_foF2 = np.nan
                    slope_foF2 = np.nan
                    foF2_max = np.nan
                    foF2_min = np.nan
                    foF2_drop = np.nan
                    num_muestras_foF2 = 0
                    tiempo_usado = np.nan
                
            elif len(df_sw) == 1:
                registro = df_sw.iloc[0]
                delta_inicio_min = (registro['HORA_DECIMAL'] - sw['inicio']) * 60
                delta_fin_min = (registro['HORA_DECIMAL'] - sw['fin']) * 60
                extremos_validos = False
                
                Δh = np.nan
                Mv = np.nan
                Mh = registro['hmF2']
                Delta_foF2 = np.nan
                slope_foF2 = np.nan
                foF2_max = registro.get('foF2', np.nan)
                foF2_min = np.nan
                foF2_drop = np.nan
                num_muestras_foF2 = 1 if pd.notna(registro.get('foF2', np.nan)) else 0
                tiempo_usado = np.nan
                
            else:
                delta_inicio_min = np.nan
                delta_fin_min = np.nan
                extremos_validos = False
                
                Δh = np.nan
                Mv = np.nan
                Mh = np.nan
                Delta_foF2 = np.nan
                slope_foF2 = np.nan
                foF2_max = np.nan
                foF2_min = np.nan
                foF2_drop = np.nan
                num_muestras_foF2 = 0
                tiempo_usado = np.nan
            
            # Guardar resultados de la sub-ventana
            fila[f'Δh{idx}'] = Δh
            fila[f'Mh{idx}'] = Mh
            fila[f'Mv{idx}'] = Mv
            fila[f'Delta_foF2_{idx}'] = Delta_foF2
            fila[f'slope_foF2_{idx}'] = slope_foF2
            fila[f'foF2_max_SW{idx}'] = foF2_max
            fila[f'foF2_min_SW{idx}'] = foF2_min
            fila[f'foF2_drop_SW{idx}'] = foF2_drop
            fila[f'num_muestras_foF2_SW{idx}'] = num_muestras_foF2
            fila[f'delta_inicio_SW{idx}_min'] = delta_inicio_min
            fila[f'delta_fin_SW{idx}_min'] = delta_fin_min
            fila[f'extremos_validos_SW{idx}'] = extremos_validos
            fila[f'tiempo_real_SW{idx}_min'] = tiempo_usado
        
        fila['F_DENSITY'] = F_DENSITY_GLOBAL
        resultados.append(fila)
    
    df_resultado = pd.DataFrame(resultados)
    
    # 4. ESTADÍSTICAS
    print("\n[4/4] Estadísticas finales...")
    print("\n" + "="*70)
    print("RESUMEN DE VALIDACIÓN")
    print("="*70)
    
    total_dias = len(df_resultado)
    
    for idx in range(1, 3):
        validos = df_resultado[f'extremos_validos_SW{idx}'].sum()
        print(f"\nSW{idx}:")
        print(f"  Días con extremos VÁLIDOS: {validos} ({validos/total_dias*100:.1f}%)")
        print(f"  Días RECHAZADOS: {total_dias - validos} ({(total_dias-validos)/total_dias*100:.1f}%)")
        
        # Tiempo real usado
        df_con_tiempo = df_resultado[df_resultado[f'tiempo_real_SW{idx}_min'].notna()]
        if len(df_con_tiempo) > 0:
            tiempo_prom = df_con_tiempo[f'tiempo_real_SW{idx}_min'].mean()
            print(f"  Tiempo real promedio: {tiempo_prom:.2f} min (esperado: 60 min)")
        
        # Estadísticas foF2
        df_con_foF2 = df_resultado[df_resultado[f'num_muestras_foF2_SW{idx}'] > 0]
        if len(df_con_foF2) > 0:
            print(f"  Días con datos foF2: {len(df_con_foF2)}")
            print(f"  Muestras promedio: {df_con_foF2[f'num_muestras_foF2_SW{idx}'].mean():.1f}")
            
            df_con_drop = df_resultado[df_resultado[f'foF2_drop_SW{idx}'].notna()]
            print(f"  Días con foF2_drop (>=3 muestras): {len(df_con_drop)}")
    
    # Inputs completos
    columnas_inputs_basicos = [f'Δh{i}' for i in range(1,3)] + [f'Mh{i}' for i in range(1,3)] + \
                               [f'Mv{i}' for i in range(1,3)] + ['F_DENSITY']
    columnas_inputs_foF2 = [f'Delta_foF2_{i}' for i in range(1,3)] + \
                            [f'slope_foF2_{i}' for i in range(1,3)] + \
                            [f'foF2_max_SW{i}' for i in range(1,3)]
    
    dias_completos_basicos = df_resultado[df_resultado[columnas_inputs_basicos].notna().all(axis=1)]
    dias_completos_foF2 = df_resultado[df_resultado[columnas_inputs_foF2].notna().all(axis=1)]
    
    print(f"\n✅ Días con inputs BÁSICOS completos: {len(dias_completos_basicos)} ({len(dias_completos_basicos)/total_dias*100:.1f}%)")
    print(f"✅ Días con inputs foF2 EXTENDIDOS completos: {len(dias_completos_foF2)} ({len(dias_completos_foF2)/total_dias*100:.1f}%)")
    
    # 5. GUARDAR
    if output_file is None:
        output_file = input_file.replace('.csv', '_inputs_2ventanas_validado.csv')
    
    df_resultado.to_csv(output_file, index=False)
    print(f"\n💾 Archivo guardado: {output_file}")
    
    print("\n" + "="*70)
    print("✓ CÁLCULO COMPLETADO")
    print("Mv calculado con tiempo REAL por sub-ventana")
    print("Inputs foF2 extendidos incluidos por sub-ventana")
    print("="*70)
    
    return df_resultado

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 GENERADOR DE INPUTS DATASET 2 (2 VENTANAS) POR LOTES")
    print("="*70)

    total_archivos = 0

    if not os.path.exists(ROOT_INPUT_DIR):
        print(f"❌ Error: El directorio raíz '{ROOT_INPUT_DIR}' no existe.")
        return

    for year, filenames in TARGET_BATCHES.items():
        if not filenames:
            continue
            
        print(f"\n===== [AÑO {year}] Analizando carpeta... =====")
        year_folder = os.path.join(ROOT_INPUT_DIR, year)
        
        for base_filename in filenames:
            # Usamos el archivo _procesado.csv como entrada (series de tiempo)
            input_filename = base_filename.replace(".txt", "_procesado.csv")
            input_path = os.path.join(year_folder, input_filename)
            
            # Nombre de salida específico para este Dataset
            output_filename = base_filename.replace(".txt", "_inputs_2ventanas.csv")
            output_path = os.path.join(year_folder, output_filename)
            
            if os.path.exists(input_path):
                # Llamamos a la función con la tolerancia especificada (10 min)
                calcular_inputs_2_ventanas_validado(input_path, output_path, tolerancia_min=10)
                total_archivos += 1
            else:
                print(f"⚠️  Saltando: No se encontró {input_filename}")

    print("\n" + "="*70)
    print(f"🏁 PROCESO FINALIZADO. Archivos de inputs (2 ventanas) generados: {total_archivos}")
    print("="*70)

if __name__ == "__main__":
    main()

🚀 GENERADOR DE INPUTS DATASET 2 (2 VENTANAS) POR LOTES

===== [AÑO 2013] Analizando carpeta... =====
PROCESANDO: 2013_Sept_Oct_DIDBase_procesado.csv
DATASET 2: 2 SUB-VENTANAS + INPUTS EXTENDIDOS foF2
Tolerancia: ±10 min en CADA extremo

[1/4] Cargando datos...
✓ Registros cargados: 6151

[2/4] Definiendo sub-ventanas...
  SW1: 17.00 - 18.00 LT
    Extremo inicio: 17.00 ±10 min
    Extremo fin: 18.00 ±10 min
  SW2: 18.00 - 19.00 LT
    Extremo inicio: 18.00 ±10 min
    Extremo fin: 19.00 ±10 min

[3/4] Calculando inputs con validación...

[4/4] Estadísticas finales...

RESUMEN DE VALIDACIÓN

SW1:
  Días con extremos VÁLIDOS: 30 (50.8%)
  Días RECHAZADOS: 29 (49.2%)
  Tiempo real promedio: 60.00 min (esperado: 60 min)
  Días con datos foF2: 36
  Muestras promedio: 6.3
  Días con foF2_drop (>=3 muestras): 29

SW2:
  Días con extremos VÁLIDOS: 39 (66.1%)
  Días RECHAZADOS: 20 (33.9%)
  Tiempo real promedio: 59.87 min (esperado: 60 min)
  Días con datos foF2: 40
  Muestras promedio: 6.8
  D

Creando DataSet para 3 ventanas:
* Fecha en Horario Perú

In [ ]:
import pandas as pd
import numpy as np
import os

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES (Parametrizada)
# ==========================================

# Mismos archivos base, el script buscará la versión "_procesado.csv"
TARGET_BATCHES = {
    "2013": ["2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2013_Nov_Dic_DIDBase.txt"],
    "2014": ["2014_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"], 
    "2015": ["2015_Ene_Feb_DIDBase.txt", "2015_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2015_Nov_Dic_DIDBase.txt"],
    "2016": ["2016_Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"]
}

ROOT_INPUT_DIR = "Inputs"

# ==========================================
# 🧠 LÓGICA DE CÁLCULO (INTACTA)
# ==========================================

def calcular_inputs_3_ventanas_validado(input_file, output_file=None, tolerancia_min=10):
    """
    Calcula inputs con 3 sub-ventanas validando extremos
    
    Similar a Dataset 2 pero con 3 ventanas de 40 minutos cada una
    """
    
    print("="*70)
    print(f"PROCESANDO: {os.path.basename(input_file)}")
    print("DATASET 3: 3 SUB-VENTANAS + INPUTS EXTENDIDOS foF2")
    print(f"Tolerancia: ±{tolerancia_min} min en CADA extremo")
    print("="*70)
    
    # 1. CARGAR DATOS
    print("\n[1/4] Cargando datos...")
    df = pd.read_csv(input_file)
    df['Time_Local_Peru'] = pd.to_datetime(df['Time_Local_Peru'])
    df['FECHA'] = df['Time_Local_Peru'].dt.date
    df['HORA_DECIMAL'] = df['Time_Local_Peru'].dt.hour + df['Time_Local_Peru'].dt.minute / 60.0
    
    print(f"✓ Registros cargados: {len(df)}")
    
    # 2. DEFINIR SUB-VENTANAS
    print("\n[2/4] Definiendo sub-ventanas...")
    
    subventanas = [
        {'nombre': 'SW1', 'inicio': 17.0, 'fin': 17 + 40/60},      # 17:00-17:40
        {'nombre': 'SW2', 'inicio': 17 + 40/60, 'fin': 18 + 20/60}, # 17:40-18:20
        {'nombre': 'SW3', 'inicio': 18 + 20/60, 'fin': 19.0}        # 18:20-19:00
    ]
    
    tolerancia_decimal = tolerancia_min / 60.0
    
    for sw in subventanas:
        print(f"  {sw['nombre']}: {sw['inicio']:.2f} - {sw['fin']:.2f} LT")
        print(f"    Extremo inicio: {sw['inicio']:.2f} ±{tolerancia_min} min")
        print(f"    Extremo fin: {sw['fin']:.2f} ±{tolerancia_min} min")
    
    # 3. CALCULAR INPUTS
    print("\n[3/4] Calculando inputs con validación...")
    
    fechas_unicas = sorted(df['FECHA'].unique())
    resultados = []
    
    for fecha in fechas_unicas:
        df_dia = df[df['FECHA'] == fecha].copy()
        fila = {'FECHA': fecha}
        
        # F_DENSITY global (promedio de toda la ventana 17:00-19:00)
        df_completa = df_dia[(df_dia['HORA_DECIMAL'] >= 17.0) & (df_dia['HORA_DECIMAL'] <= 19.0)]
        if 'foF2' in df_completa.columns:
            foF2_validos_global = df_completa['foF2'].dropna()
            F_DENSITY_GLOBAL = foF2_validos_global.mean() if len(foF2_validos_global) > 0 else np.nan
        else:
            F_DENSITY_GLOBAL = np.nan
        
        # Procesar cada sub-ventana
        for idx, sw in enumerate(subventanas, 1):
            df_sw = df_dia[
                (df_dia['HORA_DECIMAL'] >= sw['inicio'] - tolerancia_decimal) &
                (df_dia['HORA_DECIMAL'] <= sw['fin'] + tolerancia_decimal)
            ].sort_values('HORA_DECIMAL').copy()
            
            if len(df_sw) >= 2:
                # BUSCAR extremo INICIO
                df_sw['dist_inicio'] = abs(df_sw['HORA_DECIMAL'] - sw['inicio'])
                idx_inicio = df_sw['dist_inicio'].idxmin()
                registro_inicio = df_sw.loc[idx_inicio]
                
                # BUSCAR extremo FIN
                df_sw['dist_fin'] = abs(df_sw['HORA_DECIMAL'] - sw['fin'])
                idx_fin = df_sw['dist_fin'].idxmin()
                registro_fin = df_sw.loc[idx_fin]
                
                # CALCULAR deltas
                delta_inicio_min = (registro_inicio['HORA_DECIMAL'] - sw['inicio']) * 60
                delta_fin_min = (registro_fin['HORA_DECIMAL'] - sw['fin']) * 60
                
                # VALIDAR tolerancia ESTRICTA
                inicio_valido = abs(delta_inicio_min) <= tolerancia_min
                fin_valido = abs(delta_fin_min) <= tolerancia_min
                extremos_validos = inicio_valido and fin_valido
                
                if extremos_validos:
                    # ✅ CALCULAR con extremos válidos
                    hmF2_inicio = registro_inicio['hmF2']
                    hmF2_fin = registro_fin['hmF2']
                    
                    Δh = hmF2_fin - hmF2_inicio
                    
                    # ⚡ Mv con tiempo REAL
                    tiempo_real_min = (registro_fin['Time_Local_Peru'] - 
                                       registro_inicio['Time_Local_Peru']).total_seconds() / 60.0
                    Mv = Δh / tiempo_real_min
                    
                    # Datos entre extremos
                    df_entre = df_sw[
                        (df_sw['HORA_DECIMAL'] >= registro_inicio['HORA_DECIMAL']) &
                        (df_sw['HORA_DECIMAL'] <= registro_fin['HORA_DECIMAL'])
                    ]
                    
                    # Mh entre extremos
                    Mh = df_entre['hmF2'].max()
                    
                    # ========================================
                    # INPUTS EXTENDIDOS foF2 POR SUB-VENTANA
                    # ========================================
                    
                    if 'foF2' in df_entre.columns:
                        foF2_validos = df_entre['foF2'].dropna()
                        num_muestras_foF2 = len(foF2_validos)
                        
                        # ΔfoF2: Diferencia entre extremos
                        foF2_inicio = registro_inicio.get('foF2', np.nan)
                        foF2_fin = registro_fin.get('foF2', np.nan)
                        
                        if pd.notna(foF2_inicio) and pd.notna(foF2_fin):
                            Delta_foF2 = foF2_fin - foF2_inicio
                            # slope_foF2: Pendiente (MHz/min)
                            slope_foF2 = Delta_foF2 / tiempo_real_min if tiempo_real_min > 0 else np.nan
                        else:
                            Delta_foF2 = np.nan
                            slope_foF2 = np.nan
                        
                        # foF2_max: Máximo en la sub-ventana
                        foF2_max = foF2_validos.max() if num_muestras_foF2 > 0 else np.nan
                        
                        # foF2_min y foF2_drop: Solo si hay >=3 muestras
                        if num_muestras_foF2 >= 3:
                            foF2_min = foF2_validos.min()
                            foF2_drop = foF2_max - foF2_min
                        else:
                            foF2_min = np.nan
                            foF2_drop = np.nan
                        
                    else:
                        Delta_foF2 = np.nan
                        slope_foF2 = np.nan
                        foF2_max = np.nan
                        foF2_min = np.nan
                        foF2_drop = np.nan
                        num_muestras_foF2 = 0
                    
                    tiempo_usado = tiempo_real_min
                    
                else:
                    # ❌ EXTREMOS FUERA DE TOLERANCIA: NaN
                    Δh = np.nan
                    Mv = np.nan
                    Mh = np.nan
                    Delta_foF2 = np.nan
                    slope_foF2 = np.nan
                    foF2_max = np.nan
                    foF2_min = np.nan
                    foF2_drop = np.nan
                    num_muestras_foF2 = 0
                    tiempo_usado = np.nan
                
            elif len(df_sw) == 1:
                registro = df_sw.iloc[0]
                delta_inicio_min = (registro['HORA_DECIMAL'] - sw['inicio']) * 60
                delta_fin_min = (registro['HORA_DECIMAL'] - sw['fin']) * 60
                extremos_validos = False
                
                Δh = np.nan
                Mv = np.nan
                Mh = registro['hmF2']
                Delta_foF2 = np.nan
                slope_foF2 = np.nan
                foF2_max = registro.get('foF2', np.nan)
                foF2_min = np.nan
                foF2_drop = np.nan
                num_muestras_foF2 = 1 if pd.notna(registro.get('foF2', np.nan)) else 0
                tiempo_usado = np.nan
                
            else:
                delta_inicio_min = np.nan
                delta_fin_min = np.nan
                extremos_validos = False
                
                Δh = np.nan
                Mv = np.nan
                Mh = np.nan
                Delta_foF2 = np.nan
                slope_foF2 = np.nan
                foF2_max = np.nan
                foF2_min = np.nan
                foF2_drop = np.nan
                num_muestras_foF2 = 0
                tiempo_usado = np.nan
            
            # Guardar resultados de la sub-ventana
            fila[f'Δh{idx}'] = Δh
            fila[f'Mh{idx}'] = Mh
            fila[f'Mv{idx}'] = Mv
            fila[f'Delta_foF2_{idx}'] = Delta_foF2
            fila[f'slope_foF2_{idx}'] = slope_foF2
            fila[f'foF2_max_SW{idx}'] = foF2_max
            fila[f'foF2_min_SW{idx}'] = foF2_min
            fila[f'foF2_drop_SW{idx}'] = foF2_drop
            fila[f'num_muestras_foF2_SW{idx}'] = num_muestras_foF2
            fila[f'delta_inicio_SW{idx}_min'] = delta_inicio_min
            fila[f'delta_fin_SW{idx}_min'] = delta_fin_min
            fila[f'extremos_validos_SW{idx}'] = extremos_validos
            fila[f'tiempo_real_SW{idx}_min'] = tiempo_usado
        
        fila['F_DENSITY'] = F_DENSITY_GLOBAL
        resultados.append(fila)
    
    df_resultado = pd.DataFrame(resultados)
    
    # 4. ESTADÍSTICAS
    print("\n[4/4] Estadísticas finales...")
    print("\n" + "="*70)
    print("RESUMEN DE VALIDACIÓN")
    print("="*70)
    
    total_dias = len(df_resultado)
    
    for idx in range(1, 4):
        validos = df_resultado[f'extremos_validos_SW{idx}'].sum()
        print(f"\nSW{idx}:")
        print(f"  Días con extremos VÁLIDOS: {validos} ({validos/total_dias*100:.1f}%)")
        print(f"  Días RECHAZADOS: {total_dias - validos} ({(total_dias-validos)/total_dias*100:.1f}%)")
        
        # Tiempo real usado
        df_tiempo = df_resultado[df_resultado[f'tiempo_real_SW{idx}_min'].notna()]
        if len(df_tiempo) > 0:
            tiempo_prom = df_tiempo[f'tiempo_real_SW{idx}_min'].mean()
            print(f"  Tiempo real promedio: {tiempo_prom:.2f} min (esperado: 40 min)")
            
        # Estadísticas foF2
        df_con_foF2 = df_resultado[df_resultado[f'num_muestras_foF2_SW{idx}'] > 0]
        if len(df_con_foF2) > 0:
            print(f"  Días con datos foF2: {len(df_con_foF2)}")
            print(f"  Muestras promedio: {df_con_foF2[f'num_muestras_foF2_SW{idx}'].mean():.1f}")
            
            df_con_drop = df_resultado[df_resultado[f'foF2_drop_SW{idx}'].notna()]
            print(f"  Días con foF2_drop (>=3 muestras): {len(df_con_drop)}")
    
    # Inputs completos
    columnas_inputs_basicos = [f'Δh{i}' for i in range(1,4)] + [f'Mh{i}' for i in range(1,4)] + \
                               [f'Mv{i}' for i in range(1,4)] + ['F_DENSITY']
    columnas_inputs_foF2 = [f'Delta_foF2_{i}' for i in range(1,4)] + \
                            [f'slope_foF2_{i}' for i in range(1,4)] + \
                            [f'foF2_max_SW{i}' for i in range(1,4)]
    
    dias_completos_basicos = df_resultado[df_resultado[columnas_inputs_basicos].notna().all(axis=1)]
    dias_completos_foF2 = df_resultado[df_resultado[columnas_inputs_foF2].notna().all(axis=1)]
    
    print(f"\n✅ Días con inputs BÁSICOS completos: {len(dias_completos_basicos)} ({len(dias_completos_basicos)/total_dias*100:.1f}%)")
    print(f"✅ Días con inputs foF2 EXTENDIDOS completos: {len(dias_completos_foF2)} ({len(dias_completos_foF2)/total_dias*100:.1f}%)")
    
    # 5. GUARDAR
    if output_file is None:
        output_file = input_file.replace('.csv', '_inputs_3ventanas_validado.csv')
    
    df_resultado.to_csv(output_file, index=False)
    print(f"\n💾 Archivo guardado: {output_file}")
    
    print("\n" + "="*70)
    print("✓ CÁLCULO COMPLETADO")
    print("Mv calculado con tiempo REAL por sub-ventana")
    print("Inputs foF2 extendidos incluidos por sub-ventana")
    print("="*70)
    
    return df_resultado

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 GENERADOR DE INPUTS DATASET 3 (3 VENTANAS) POR LOTES")
    print("="*70)

    total_archivos = 0

    if not os.path.exists(ROOT_INPUT_DIR):
        print(f"❌ Error: El directorio raíz '{ROOT_INPUT_DIR}' no existe.")
        return

    for year, filenames in TARGET_BATCHES.items():
        if not filenames:
            continue
            
        print(f"\n===== [AÑO {year}] Analizando carpeta... =====")
        year_folder = os.path.join(ROOT_INPUT_DIR, year)
        
        for base_filename in filenames:
            # Entrada: _procesado.csv
            input_filename = base_filename.replace(".txt", "_procesado.csv")
            input_path = os.path.join(year_folder, input_filename)
            
            # Salida: _inputs_3ventanas.csv
            output_filename = base_filename.replace(".txt", "_inputs_3ventanas.csv")
            output_path = os.path.join(year_folder, output_filename)
            
            if os.path.exists(input_path):
                # Llamada con tolerancia 5 min
                calcular_inputs_3_ventanas_validado(input_path, output_path, tolerancia_min=10)
                total_archivos += 1
            else:
                print(f"⚠️  Saltando: No se encontró {input_filename}")

    print("\n" + "="*70)
    print(f"🏁 PROCESO FINALIZADO. Archivos de inputs (3 ventanas) generados: {total_archivos}")
    print("="*70)

if __name__ == "__main__":
    main()

🚀 GENERADOR DE INPUTS DATASET 3 (3 VENTANAS) POR LOTES

===== [AÑO 2013] Analizando carpeta... =====
PROCESANDO: 2013_Sept_Oct_DIDBase_procesado.csv
DATASET 3: 3 SUB-VENTANAS + INPUTS EXTENDIDOS foF2
Tolerancia: ±10 min en CADA extremo

[1/4] Cargando datos...
✓ Registros cargados: 6151

[2/4] Definiendo sub-ventanas...
  SW1: 17.00 - 17.67 LT
    Extremo inicio: 17.00 ±10 min
    Extremo fin: 17.67 ±10 min
  SW2: 17.67 - 18.33 LT
    Extremo inicio: 17.67 ±10 min
    Extremo fin: 18.33 ±10 min
  SW3: 18.33 - 19.00 LT
    Extremo inicio: 18.33 ±10 min
    Extremo fin: 19.00 ±10 min

[3/4] Calculando inputs con validación...

[4/4] Estadísticas finales...

RESUMEN DE VALIDACIÓN

SW1:
  Días con extremos VÁLIDOS: 28 (47.5%)
  Días RECHAZADOS: 31 (52.5%)
  Tiempo real promedio: 43.21 min (esperado: 40 min)
  Días con datos foF2: 35
  Muestras promedio: 4.7
  Días con foF2_drop (>=3 muestras): 26

SW2:
  Días con extremos VÁLIDOS: 37 (62.7%)
  Días RECHAZADOS: 22 (37.3%)
  Tiempo real prom

### Variables Globales OMNI2:

A partir del siguiente Link se obtiene los valores que serán usados como Inputs: https://omniweb.gsfc.nasa.gov/form/dx1.html

Datos a elegir:
* By, GSE, nT
* Bz, GSE, nT
* Flow Speed, km/sec
* Flow Pressure, nPa
* Ey - Electric Field, mV/m
* Kp*10 Index
* Dst Index, nT
* Solar index F10.7

Convertir archivos a CSV

In [ ]:
import pandas as pd
from datetime import timedelta
import os

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES (OMNI2)
# ==========================================

# Define aquí los nombres EXACTOS de tus archivos de texto OMNI2.
# El script buscará en: Inputs/{AÑO}/omni2/{NOMBRE_ARCHIVO}
TARGET_BATCHES = {
    "2013": ["omni2_2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago.lst", "omni2_2013_Nov_Dic.lst"],
    "2014": ["omni2_2014_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic.lst"], 
    "2015": ["omni2_2015_Ene_Feb.lst", "omni2_2015_Abr_May_Jun_Jul_Ago.lst", "omni2_2015_Nov_Dic.lst"],
    "2016": ["omni2_2016_Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic.lst"]
}

ROOT_INPUT_DIR = "Inputs"

# Nombres de columnas según formato OMNI2 estándar (ASCII)
COLUMN_NAMES = [
    "YEAR", "DOY", "Hour",
    "BY_GSE_nT", "BZ_GSE_nT",
    "SW_Plasma_Speed_kms", "Flow_Pressure_nPa",
    "E_Electric_Field_mVm", "Kp_Index",
    "Dst_Index_nT", "F10.7_Index"
]

# ==========================================
# 🔧 LÓGICA DE PROCESAMIENTO
# ==========================================

def procesar_omni_file(input_path, output_path):
    print(f"   ⚡ Procesando: {os.path.basename(input_path)}")
    
    try:
        # 1. Cargar el archivo
        # sep='\s+' maneja múltiples espacios como delimitador
        df = pd.read_csv(
            input_path, 
            sep=r'\s+', 
            names=COLUMN_NAMES,
            header=None
        )
        
        # 2. Crear columna de fecha UTC
        # Formato: Año + Día del Año (DOY) + Hora
        df['Time_UTC'] = pd.to_datetime(df['YEAR']*1000 + df['DOY'], format='%Y%j') \
                         + pd.to_timedelta(df['Hour'], unit='h')
        
        # 3. Crear columna de hora local Perú (UTC-5)
        df['Time_Local_Peru'] = df['Time_UTC'] - timedelta(hours=5)
        
        # 4. Crear columnas de fecha desglosada para facilitar lectura
        df['FECHA'] = df['Time_Local_Peru'].dt.date
        
        # 5. Reordenar columnas (Fechas primero)
        cols_order = ['Time_UTC', 'Time_Local_Peru', 'FECHA'] + \
                     [col for col in df.columns if col not in ['Time_UTC', 'Time_Local_Peru', 'FECHA']]
        df = df[cols_order]
        
        # 6. Guardar como CSV
        df.to_csv(output_path, index=False)
        print(f"      ✅ Guardado: {os.path.basename(output_path)}")
        return True
        
    except Exception as e:
        print(f"      ❌ Error: {e}")
        return False

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 PROCESAMIENTO DE DATOS OMNI2 POR LOTES")
    print("="*70)

    total_procesados = 0

    if not os.path.exists(ROOT_INPUT_DIR):
        print(f"❌ Error: El directorio raíz '{ROOT_INPUT_DIR}' no existe.")
        return

    for year, filenames in TARGET_BATCHES.items():
        if not filenames:
            continue
            
        print(f"\n===== [AÑO {year}] Buscando en carpeta 'omni2' =====")
        
        # Ruta esperada: Inputs/2013/omni2/
        omni_folder = os.path.join(ROOT_INPUT_DIR, year, "omni2")
        
        if not os.path.exists(omni_folder):
            print(f"⚠️  No existe la carpeta: {omni_folder}")
            continue
        
        for filename in filenames:
            input_path = os.path.join(omni_folder, filename)
            
            # Nombre de salida: omni2_2013_Sept_Oct_procesado.csv
            output_filename = filename.replace('.lst', '').replace('.txt', '') + "_procesado.csv"
            output_path = os.path.join(omni_folder, output_filename)
            
            if os.path.exists(input_path):
                if procesar_omni_file(input_path, output_path):
                    total_procesados += 1
            else:
                # Intento de fallback: A veces el archivo termina en .txt aunque en la lista pusimos .lst
                input_path_alt = input_path.replace('.lst', '.txt')
                if os.path.exists(input_path_alt):
                     if procesar_omni_file(input_path_alt, output_path):
                        total_procesados += 1
                else:
                    print(f"   ⚠️  Archivo no encontrado: {filename}")

    print("\n" + "="*70)
    print(f"🏁 PROCESO FINALIZADO. Archivos OMNI2 procesados: {total_procesados}")
    print("="*70)

if __name__ == "__main__":
    main()

🚀 PROCESAMIENTO DE DATOS OMNI2 POR LOTES

===== [AÑO 2013] Buscando en carpeta 'omni2' =====
   ⚡ Procesando: omni2_2013_Sept_Oct.lst
      ✅ Guardado: omni2_2013_Sept_Oct_procesado.csv

===== [AÑO 2014] Buscando en carpeta 'omni2' =====
   ⚡ Procesando: omni2_2014_Ene_Feb_Mar.lst
      ✅ Guardado: omni2_2014_Ene_Feb_Mar_procesado.csv

===== [AÑO 2015] Buscando en carpeta 'omni2' =====
   ⚡ Procesando: omni2_2015_Mar.lst
      ✅ Guardado: omni2_2015_Mar_procesado.csv
   ⚡ Procesando: omni2_2015_Sept_Oct.lst
      ✅ Guardado: omni2_2015_Sept_Oct_procesado.csv

===== [AÑO 2016] Buscando en carpeta 'omni2' =====
   ⚡ Procesando: omni2_2016_Ene_Feb.lst
      ✅ Guardado: omni2_2016_Ene_Feb_procesado.csv

🏁 PROCESO FINALIZADO. Archivos OMNI2 procesados: 5


La FECHA se encuentra en Hora de Perú

In [ ]:
"""
CALCULADOR DE FEATURES OMNI2 POR LOTES
Entrada: Archivos *_procesado.csv (con Time_Local_Peru)
Salida: Archivos *_features.csv (una fila por día)
"""

import pandas as pd
import numpy as np
from scipy import stats
import os

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES
# ==========================================

# Usamos los mismos nombres base para mantener la coherencia.
# El script buscará automáticamente la versión "_procesado.csv"
TARGET_BATCHES = {
    "2013": ["omni2_2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago.lst", "omni2_2013_Nov_Dic.lst"],
    "2014": ["omni2_2014_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic.lst"], 
    "2015": ["omni2_2015_Ene_Feb.lst", "omni2_2015_Abr_May_Jun_Jul_Ago.lst", "omni2_2015_Nov_Dic.lst"],
    "2016": ["omni2_2016_Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic.lst"]
}

ROOT_INPUT_DIR = "Inputs"

# ==========================================
# 🧠 LÓGICA DE CÁLCULO (INTACTA)
# ==========================================

def calcular_features_omni2(input_file, output_file=None):
    """
    Calcula features de OMNI2 para cada día usando Time_Local_Peru.
    """
    
    print("="*70)
    print(f"PROCESANDO: {os.path.basename(input_file)}")
    print("Ventana principal: 16:00-19:00 LT Perú")
    print("Ventana Kp: 13:00-18:00 LT Perú (últimas 6h)")
    print("="*70)
    
    # 1. CARGAR DATOS
    print("\n[1/5] Cargando datos OMNI2...")
    try:
        df = pd.read_csv(input_file)
    except Exception as e:
        print(f"❌ Error leyendo archivo: {e}")
        return False

    # Asegurar formato de fecha
    if 'Time_Local_Peru' not in df.columns:
        print("❌ Error: Columna 'Time_Local_Peru' no encontrada.")
        return False

    df['Time_Local_Peru'] = pd.to_datetime(df['Time_Local_Peru'])
    df['FECHA'] = df['Time_Local_Peru'].dt.date
    df['HORA'] = df['Time_Local_Peru'].dt.hour
    
    print(f"✓ Registros cargados: {len(df)}")
    
    # 3. DEFINIR FUNCIONES AUXILIARES
    def calcular_slope(serie_tiempo, serie_valores):
        """Calcula pendiente lineal usando regresión"""
        if len(serie_valores) < 2:
            return np.nan
        
        valores_validos = serie_valores.dropna()
        if len(valores_validos) < 2:
            return np.nan
        
        # Crear índices temporales (0, 1, 2, 3...)
        x = np.arange(len(valores_validos))
        y = valores_validos.values
        
        try:
            slope, _, _, _, _ = stats.linregress(x, y)
            return slope
        except:
            return np.nan
    
    # 4. CALCULAR FEATURES POR DÍA
    print("\n[3/5] Calculando features por día...")
    
    fechas_unicas = sorted(df['FECHA'].unique())
    resultados = []
    
    for fecha in fechas_unicas:
        df_dia = df[df['FECHA'] == fecha].copy()
        fila = {'FECHA': fecha}
        
        # ========================================
        # VENTANA PRINCIPAL: 16:00-19:00 LT
        # ========================================
        df_ventana = df_dia[(df_dia['HORA'] >= 16) & (df_dia['HORA'] <= 19)].copy()
        num_registros_ventana = len(df_ventana)
        
        if num_registros_ventana > 0:
            # A. VARIABLES DE BZ (IMF Bz)
            if 'BZ_GSE_nT' in df_ventana.columns:
                bz_validos = df_ventana['BZ_GSE_nT'].dropna()
                fila['Bz_mean_16_19'] = bz_validos.mean() if len(bz_validos) > 0 else np.nan
                fila['Bz_min_16_19'] = bz_validos.min() if len(bz_validos) > 0 else np.nan
                fila['Bz_slope_16_19'] = calcular_slope(df_ventana['Time_Local_Peru'], df_ventana['BZ_GSE_nT'])
            else:
                fila['Bz_mean_16_19'] = np.nan
                fila['Bz_min_16_19'] = np.nan
                fila['Bz_slope_16_19'] = np.nan
            
            # B. CAMPO ELÉCTRICO INTERPLANETARIO
            if 'E_Electric_Field_mVm' in df_ventana.columns:
                e_validos = df_ventana['E_Electric_Field_mVm'].dropna()
                fila['E_mean_16_19'] = e_validos.mean() if len(e_validos) > 0 else np.nan
                fila['E_max_16_19'] = e_validos.max() if len(e_validos) > 0 else np.nan
                fila['E_slope_16_19'] = calcular_slope(df_ventana['Time_Local_Peru'], df_ventana['E_Electric_Field_mVm'])
            else:
                fila['E_mean_16_19'] = np.nan
                fila['E_max_16_19'] = np.nan
                fila['E_slope_16_19'] = np.nan
            
            # C. VELOCIDAD DEL VIENTO SOLAR
            if 'SW_Plasma_Speed_kms' in df_ventana.columns:
                sw_validos = df_ventana['SW_Plasma_Speed_kms'].dropna()
                fila['SW_speed_mean_16_19'] = sw_validos.mean() if len(sw_validos) > 0 else np.nan
                fila['SW_speed_std_16_19'] = sw_validos.std() if len(sw_validos) > 0 else np.nan
                fila['SW_speed_slope_16_19'] = calcular_slope(df_ventana['Time_Local_Peru'], df_ventana['SW_Plasma_Speed_kms'])
            else:
                fila['SW_speed_mean_16_19'] = np.nan
                fila['SW_speed_std_16_19'] = np.nan
                fila['SW_speed_slope_16_19'] = np.nan
            
            # D. DST GEOMAGNÉTICO
            if 'Dst_Index_nT' in df_ventana.columns:
                dst_validos = df_ventana['Dst_Index_nT'].dropna()
                fila['Dst_mean_16_19'] = dst_validos.mean() if len(dst_validos) > 0 else np.nan
                fila['Dst_min_16_19'] = dst_validos.min() if len(dst_validos) > 0 else np.nan
            else:
                fila['Dst_mean_16_19'] = np.nan
                fila['Dst_min_16_19'] = np.nan
            
            # G. PRESIÓN DINÁMICA
            if 'Flow_Pressure_nPa' in df_ventana.columns:
                pdyn_validos = df_ventana['Flow_Pressure_nPa'].dropna()
                fila['Pdyn_mean_16_19'] = pdyn_validos.mean() if len(pdyn_validos) > 0 else np.nan
                fila['Pdyn_max_16_19'] = pdyn_validos.max() if len(pdyn_validos) > 0 else np.nan
            else:
                fila['Pdyn_mean_16_19'] = np.nan
                fila['Pdyn_max_16_19'] = np.nan
            
            # OPCIONAL: BY
            if 'BY_GSE_nT' in df_ventana.columns:
                by_validos = df_ventana['BY_GSE_nT'].dropna()
                fila['BY_mean_16_19'] = by_validos.mean() if len(by_validos) > 0 else np.nan
            else:
                fila['BY_mean_16_19'] = np.nan
        
        else:
            # Sin datos en ventana
            for col in ['Bz_mean_16_19', 'Bz_min_16_19', 'Bz_slope_16_19', 
                        'E_mean_16_19', 'E_max_16_19', 'E_slope_16_19',
                        'SW_speed_mean_16_19', 'SW_speed_std_16_19', 'SW_speed_slope_16_19',
                        'Dst_mean_16_19', 'Dst_min_16_19', 'Pdyn_mean_16_19', 
                        'Pdyn_max_16_19', 'BY_mean_16_19']:
                fila[col] = np.nan
        
        # ========================================
        # E. KP: ÚLTIMAS 6H ANTES DE 19:00 (13:00-18:00 LT)
        # ========================================
        df_kp = df_dia[(df_dia['HORA'] >= 13) & (df_dia['HORA'] <= 18)].copy()
        
        if len(df_kp) > 0 and 'Kp_Index' in df_kp.columns:
            kp_validos = df_kp['Kp_Index'].dropna()
            fila['Kp_last_6h_mean'] = kp_validos.mean() if len(kp_validos) > 0 else np.nan
            fila['Kp_last_6h_max'] = kp_validos.max() if len(kp_validos) > 0 else np.nan
            fila['Kp_last_6h_sum'] = kp_validos.sum() if len(kp_validos) > 0 else np.nan
        else:
            fila['Kp_last_6h_mean'] = np.nan
            fila['Kp_last_6h_max'] = np.nan
            fila['Kp_last_6h_sum'] = np.nan
        
        # ========================================
        # F. F10.7 (VALOR DIARIO)
        # ========================================
        if 'F10.7_Index' in df_dia.columns:
            f107_validos = df_dia['F10.7_Index'].dropna()
            fila['F10.7_Index'] = f107_validos.iloc[0] if len(f107_validos) > 0 else np.nan
        else:
            fila['F10.7_Index'] = np.nan
        
        fila['num_registros_16_19'] = num_registros_ventana
        fila['num_registros_kp_13_18'] = len(df_kp)
        
        resultados.append(fila)
    
    df_resultado = pd.DataFrame(resultados)
    
    # 5. GUARDAR
    if output_file is None:
        output_file = input_file.replace('_procesado.csv', '_features.csv')
    
    print("\n[5/5] Guardando resultados...")
    df_resultado.to_csv(output_file, index=False)
    print(f"💾 Archivo guardado: {os.path.basename(output_file)}")
    
    return True

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 CALCULADOR DE FEATURES OMNI2 POR LOTES")
    print("="*70)

    total_procesados = 0

    if not os.path.exists(ROOT_INPUT_DIR):
        print(f"❌ Error: El directorio raíz '{ROOT_INPUT_DIR}' no existe.")
        return

    for year, filenames in TARGET_BATCHES.items():
        if not filenames:
            continue
            
        print(f"\n===== [AÑO {year}] Analizando carpeta 'omni2' =====")
        omni_folder = os.path.join(ROOT_INPUT_DIR, year, "omni2")
        
        for filename in filenames:
            # Construir el nombre del archivo de entrada esperado (_procesado.csv)
            # Ej: omni2_2013_Sept_Oct.lst -> omni2_2013_Sept_Oct_procesado.csv
            base_name = filename.replace('.lst', '').replace('.txt', '')
            input_filename = f"{base_name}_procesado.csv"
            input_path = os.path.join(omni_folder, input_filename)
            
            # Nombre de salida
            output_filename = f"{base_name}_features.csv"
            output_path = os.path.join(omni_folder, output_filename)
            
            if os.path.exists(input_path):
                if calcular_features_omni2(input_path, output_path):
                    total_procesados += 1
            else:
                print(f"⚠️  Saltando: No se encontró {input_filename}")

    print("\n" + "="*70)
    print(f"🏁 PROCESO FINALIZADO. Archivos de features generados: {total_procesados}")
    print("="*70)

if __name__ == "__main__":
    main()

🚀 CALCULADOR DE FEATURES OMNI2 POR LOTES

===== [AÑO 2013] Analizando carpeta 'omni2' =====
PROCESANDO: omni2_2013_Sept_Oct_procesado.csv
Ventana principal: 16:00-19:00 LT Perú
Ventana Kp: 13:00-18:00 LT Perú (últimas 6h)

[1/5] Cargando datos OMNI2...
✓ Registros cargados: 1464

[3/5] Calculando features por día...

[5/5] Guardando resultados...
💾 Archivo guardado: omni2_2013_Sept_Oct_features.csv

===== [AÑO 2014] Analizando carpeta 'omni2' =====
PROCESANDO: omni2_2014_Ene_Feb_Mar_procesado.csv
Ventana principal: 16:00-19:00 LT Perú
Ventana Kp: 13:00-18:00 LT Perú (últimas 6h)

[1/5] Cargando datos OMNI2...
✓ Registros cargados: 2160

[3/5] Calculando features por día...

[5/5] Guardando resultados...
💾 Archivo guardado: omni2_2014_Ene_Feb_Mar_features.csv

===== [AÑO 2015] Analizando carpeta 'omni2' =====
PROCESANDO: omni2_2015_Mar_procesado.csv
Ventana principal: 16:00-19:00 LT Perú
Ventana Kp: 13:00-18:00 LT Perú (últimas 6h)

[1/5] Cargando datos OMNI2...
✓ Registros cargados: 74

### Unión entre OMNI2 y Features antes creados

In [ ]:
"""
FUSIÓN DE DATOS IONOSFÉRICOS + OMNI2 POR LOTES (MULTIPLE DATASETS)
Combina features de ionosfera (inputs calculados para 1, 2 y 3 ventanas) 
con features magnetosféricas (OMNI2)
"""

import pandas as pd
import numpy as np
import os

# ==========================================
# ⚙️  CONFIGURACIÓN DE LOTES
# ==========================================

TARGET_BATCHES = {
    "2013": ["2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2013_Nov_Dic_DIDBase.txt"],
    "2014": ["2014_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"], 
    "2015": ["2015_Ene_Feb_DIDBase.txt", "2015_Abr_May_Jun_Jul_Ago_DIDBase.txt", "2015_Nov_Dic_DIDBase.txt"],
    "2016": ["2016_Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic_DIDBase.txt"]
}

ROOT_INPUT_DIR = "Inputs"

# Variantes de archivos de inputs que vamos a buscar y procesar
INPUT_VARIANTS = [
    "_inputs.csv",            # Dataset 1
    "_inputs_2ventanas.csv",  # Dataset 2
    "_inputs_3ventanas.csv"   # Dataset 3
]

# ==========================================
# 🧠 LÓGICA DE FUSIÓN (INTACTA)
# ==========================================

def fusionar_ionosfera_omni2(archivo_ionosfera, archivo_omni2, output_file=None):
    """
    Fusiona datos de ionosfera con OMNI2 por fecha
    """
    
    print("="*70)
    print("FUSIÓN DE DATOS IONOSFERA + OMNI2")
    print(f"Ionosfera: {os.path.basename(archivo_ionosfera)}")
    print(f"OMNI2:     {os.path.basename(archivo_omni2)}")
    print("="*70)
    
    # 1. CARGAR DATOS
    try:
        df_iono = pd.read_csv(archivo_ionosfera)
        if 'FECHA' in df_iono.columns:
            df_iono['FECHA'] = pd.to_datetime(df_iono['FECHA']).dt.date
        else:
            print("❌ Error: Columna 'FECHA' no encontrada en archivo de ionosfera.")
            return False
        
        df_omni = pd.read_csv(archivo_omni2)
        if 'FECHA' in df_omni.columns:
            df_omni['FECHA'] = pd.to_datetime(df_omni['FECHA']).dt.date
        else:
            print("❌ Error: Columna 'FECHA' no encontrada en archivo OMNI2.")
            return False
            
    except Exception as e:
        print(f"❌ Error leyendo archivos: {e}")
        return False
    
    print(f"✓ Ionosfera cargado: {len(df_iono)} días")
    print(f"✓ OMNI2 cargado: {len(df_omni)} días")
    
    # 2. VERIFICAR OVERLAP
    fechas_iono = set(df_iono['FECHA'])
    fechas_omni = set(df_omni['FECHA'])
    fechas_comunes = fechas_iono & fechas_omni
    
    print(f"📊 Fechas COMUNES: {len(fechas_comunes)}")
    
    # 3. FUSIONAR (LEFT JOIN)
    print("⚡ Fusionando datasets (LEFT JOIN)...")
    
    df_fusionado = pd.merge(
        df_iono,
        df_omni,
        on='FECHA',
        how='left',
        suffixes=('', '_omni')
    )
    
    # 4. ANÁLISIS DE COMPLETITUD (Simplificado)
    columnas_omni = [col for col in df_omni.columns if col != 'FECHA']
    filas_con_omni = df_fusionado[columnas_omni].notna().all(axis=1).sum()
    print(f"✅ Filas con features OMNI2 completas: {filas_con_omni} ({filas_con_omni/len(df_fusionado)*100:.1f}%)")
    
    # 5. GUARDAR
    if output_file:
        df_fusionado.to_csv(output_file, index=False)
        print(f"💾 Guardado: {os.path.basename(output_file)}")
    
    print("✓ FUSIÓN COMPLETADA\n")
    return True

# ==========================================
# 🚀 EJECUCIÓN POR LOTES
# ==========================================

def main():
    print("="*70)
    print("🚀 FUSIONADOR DE DATASETS (1, 2 Y 3 VENTANAS) POR LOTES")
    print("="*70)

    total_procesados = 0

    if not os.path.exists(ROOT_INPUT_DIR):
        print(f"❌ Error: El directorio raíz '{ROOT_INPUT_DIR}' no existe.")
        return

    for year, filenames in TARGET_BATCHES.items():
        if not filenames:
            continue
            
        print(f"\n===== [AÑO {year}] Fusionando archivos... =====")
        year_folder = os.path.join(ROOT_INPUT_DIR, year)
        omni_folder = os.path.join(year_folder, "omni2")
        
        for base_filename in filenames:
            # 1. Definir archivo OMNI2 (INPUT COMÚN)
            # Lógica: '2013_Sept_Oct_DIDBase.txt' -> 'omni2_2013_Sept_Oct_features.csv'
            base_clean = base_filename.replace("_DIDBase.txt", "")
            omni_filename = f"omni2_{base_clean}_features.csv"
            omni_path = os.path.join(omni_folder, omni_filename)
            
            if not os.path.exists(omni_path):
                print(f"⚠️  Falta archivo OMNI2 para {base_clean}. Saltando este grupo.")
                continue

            # 2. Iterar sobre las variantes de inputs (1, 2 y 3 ventanas)
            for variant_suffix in INPUT_VARIANTS:
                
                # Definir archivo Ionosfera
                iono_filename = base_filename.replace(".txt", variant_suffix)
                iono_path = os.path.join(year_folder, iono_filename)
                
                # Definir Salida Específica
                # Ej: _inputs.csv -> _inputs_FUSIONADO.csv
                # Ej: _inputs_2ventanas.csv -> _inputs_2ventanas_FUSIONADO.csv
                output_suffix = variant_suffix.replace(".csv", "_FUSIONADO.csv")
                output_filename = base_filename.replace(".txt", output_suffix)
                output_path = os.path.join(year_folder, output_filename)
                
                if os.path.exists(iono_path):
                    if fusionar_ionosfera_omni2(iono_path, omni_path, output_path):
                        total_procesados += 1
                else:
                    print(f"   ⚠️  No encontrado: {iono_filename} (Saltando variante)")

    print("\n" + "="*70)
    print(f"🏁 PROCESO FINALIZADO. Total archivos fusionados: {total_procesados}")
    print("="*70)

if __name__ == "__main__":
    main()

🚀 FUSIONADOR DE DATASETS (1, 2 Y 3 VENTANAS) POR LOTES

===== [AÑO 2013] Fusionando archivos... =====
FUSIÓN DE DATOS IONOSFERA + OMNI2
Ionosfera: 2013_Sept_Oct_DIDBase_inputs.csv
OMNI2:     omni2_2013_Sept_Oct_features.csv
✓ Ionosfera cargado: 59 días
✓ OMNI2 cargado: 62 días
📊 Fechas COMUNES: 59
⚡ Fusionando datasets (LEFT JOIN)...
✅ Filas con features OMNI2 completas: 58 (98.3%)
💾 Guardado: 2013_Sept_Oct_DIDBase_inputs_FUSIONADO.csv
✓ FUSIÓN COMPLETADA

FUSIÓN DE DATOS IONOSFERA + OMNI2
Ionosfera: 2013_Sept_Oct_DIDBase_inputs_2ventanas.csv
OMNI2:     omni2_2013_Sept_Oct_features.csv
✓ Ionosfera cargado: 59 días
✓ OMNI2 cargado: 62 días
📊 Fechas COMUNES: 59
⚡ Fusionando datasets (LEFT JOIN)...
✅ Filas con features OMNI2 completas: 58 (98.3%)
💾 Guardado: 2013_Sept_Oct_DIDBase_inputs_2ventanas_FUSIONADO.csv
✓ FUSIÓN COMPLETADA

FUSIÓN DE DATOS IONOSFERA + OMNI2
Ionosfera: 2013_Sept_Oct_DIDBase_inputs_3ventanas.csv
OMNI2:     omni2_2013_Sept_Oct_features.csv
✓ Ionosfera cargado: 59 días

## Fusión Final: Target + Input

In [1]:
"""
UNIÓN AUTOMÁTICA DE INPUTS Y TARGETS - VERSIÓN MANUAL POR GRUPOS
Permite procesar diferentes grupos de meses del mismo año
Configuración fácil mediante variables globales
"""

import pandas as pd
import numpy as np
import os
from pathlib import Path
import re

# ============================================================================
# 🎯 CONFIGURACIÓN GLOBAL - MODIFICAR AQUÍ
# ============================================================================

# Lista de grupos a procesar: [año, identificador_meses, directorio]
GRUPOS_A_PROCESAR = [
    # 2013
    ['2013', 'Ene_Feb_Mar_Abr_May_Jun_Jul_Ago', 'Inputs/2013'],
    ['2013', 'Sept_Oct', 'Inputs/2013'],
    ['2013', 'Nov_Dic', 'Inputs/2013'],
    
    # 2014
    ['2014', 'Ene_Feb_Mar', 'Inputs/2014'],
    ['2014', 'Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic', 'Inputs/2014'],
    
    # 2015
    ['2015', 'Ene_Feb', 'Inputs/2015'],
    ['2015', 'Mar', 'Inputs/2015'],
    ['2015', 'Abr_May_Jun_Jul_Ago', 'Inputs/2015'],
    ['2015', 'Sept_Oct', 'Inputs/2015'],
    ['2015', 'Nov_Dic', 'Inputs/2015'],
    
    # 2016
    ['2016', 'Ene_Feb', 'Inputs/2016'],
    ['2016', 'Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic', 'Inputs/2016']
]

# Nombre de la estación
ESTACION = 'huancayo'

# ============================================================================
# CÓDIGO ORIGINAL - NO MODIFICAR
# ============================================================================

# Mapeo de meses en español a números
MESES_MAP = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
    'Sept': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
}

def extraer_meses_de_nombre(nombre_archivo):
    """
    Extrae los meses del nombre del archivo de inputs
    Ej: '2013_Sept_Oct_DIDBase_inputs_FUSIONADO.csv' -> ['09', '10']
    """
    meses_encontrados = []
    for mes_str, mes_num in MESES_MAP.items():
        if mes_str in nombre_archivo:
            meses_encontrados.append(mes_num)
    return sorted(meses_encontrados)

def encontrar_archivos_target(anio, meses, estacion):
    """
    Encuentra los archivos target correspondientes a los meses especificados
    
    Parámetros:
    -----------
    anio : str
        Año a procesar
    meses : list
        Lista de meses en formato '01', '02', etc.
    estacion : str
        Nombre de la estación (por defecto 'huancayo')
    
    Retorna:
    --------
    list : Lista de rutas a los archivos target
    """
    archivos_target = []
    
    for mes in meses:
        # Construir la ruta: 2013/2013_09/TargetFinal/huancayo_2013_09_S4_Calculado_Target_Completo.csv
        ruta_target = f"{anio}/{anio}_{mes}/TargetFinal/{estacion}_{anio}_{mes}_S4_Calculado_Target_Completo.csv"
        
        if os.path.exists(ruta_target):
            archivos_target.append(ruta_target)
            print(f"  ✓ Encontrado: {ruta_target}")
        else:
            print(f"  ⚠️  NO encontrado: {ruta_target}")
    
    return archivos_target

def concatenar_targets(archivos_target):
    """
    Concatena múltiples archivos target en un solo DataFrame
    
    Parámetros:
    -----------
    archivos_target : list
        Lista de rutas a los archivos target
    
    Retorna:
    --------
    DataFrame : Targets concatenados
    """
    if not archivos_target:
        raise ValueError("No se encontraron archivos target para concatenar")
    
    print(f"\n  Concatenando {len(archivos_target)} archivo(s) target...")
    
    dfs = []
    for archivo in archivos_target:
        df = pd.read_csv(archivo)

        # ⭐ AGREGAR ESTAS LÍNEAS AQUÍ ⭐
        if 'FECHA_PERU' in df.columns:
            df = df.rename(columns={'FECHA_PERU': 'FECHA'})

        dfs.append(df)
        print(f"    - {archivo}: {len(df)} registros")
    
    df_concatenado = pd.concat(dfs, ignore_index=True)
    
    # Eliminar duplicados por FECHA (si existen)
    if 'FECHA' in df_concatenado.columns:
        df_concatenado['FECHA'] = pd.to_datetime(df_concatenado['FECHA']).dt.date
        duplicados_antes = len(df_concatenado)
        df_concatenado = df_concatenado.drop_duplicates(subset=['FECHA'], keep='first')
        duplicados_despues = len(df_concatenado)
        
        if duplicados_antes != duplicados_despues:
            print(f"    ⚠️  Se eliminaron {duplicados_antes - duplicados_despues} duplicados")
    
    df_concatenado = df_concatenado.sort_values('FECHA').reset_index(drop=True)
    
    print(f"  ✓ Total concatenado: {len(df_concatenado)} registros únicos")
    
    return df_concatenado

def unir_inputs_y_targets(df_inputs, df_targets, tipo_ventana):
    """
    Une inputs y targets (LÓGICA ORIGINAL COMPLETA)
    
    Parámetros:
    -----------
    df_inputs : DataFrame
        DataFrame con inputs precursores + OMNI2
    df_targets : DataFrame
        DataFrame con targets de centelleo S4
    tipo_ventana : str
        Tipo de ventana ('1ventana', '2ventanas', '3ventanas')
    
    Retorna:
    --------
    tuple (DataFrame, DataFrame)
        - DataFrame con todos los registros unidos
        - DataFrame solo con registros completos y válidos
    """
    
    print("\n[1/5] Preparando datos para unión...")
    
    # Convertir FECHA a tipo fecha
    df_inputs['FECHA'] = pd.to_datetime(df_inputs['FECHA']).dt.date
    df_targets['FECHA'] = pd.to_datetime(df_targets['FECHA']).dt.date
    
    # Identificar tipo de dataset (1, 2 o 3 ventanas)
    if 'Mh_PRECURSOR' in df_inputs.columns:
        tipo_dataset = "1 ventana"
        columnas_criticas_validas = ['extremos_validos']
    elif 'Mh1' in df_inputs.columns and 'Mh2' in df_inputs.columns:
        if 'Mh3' in df_inputs.columns:
            tipo_dataset = "3 ventanas"
            columnas_criticas_validas = ['extremos_validos_SW1', 'extremos_validos_SW2', 'extremos_validos_SW3']
        else:
            tipo_dataset = "2 ventanas"
            columnas_criticas_validas = ['extremos_validos_SW1', 'extremos_validos_SW2']
    else:
        tipo_dataset = "desconocido"
        columnas_criticas_validas = []
    
    print(f"✓ Tipo de dataset detectado: {tipo_dataset}")
    print(f"✓ Columnas de validación: {columnas_criticas_validas}")
    
    # Mantener TODAS las columnas de inputs (ya incluye OMNI2)
    print(f"✓ Manteniendo todas las {len(df_inputs.columns)} columnas de inputs")
    
    # Seleccionar columnas de targets
    columnas_targets = [
        'FECHA', 'TARGET', 'VECES_SUPERO_UMBRAL', 'S4_MAX', 'S4_MEAN', 
        'S4_STD', 'S4_MIN', 'TOTAL_MEDICIONES', 'PORCENTAJE_CENTELLEO', 
        'HORA_PICO', 'PRN_PICO'
    ]
    
    # Verificar qué columnas existen realmente en targets
    columnas_targets_disponibles = [col for col in columnas_targets if col in df_targets.columns]
    print(f"✓ Columnas de targets a conservar: {columnas_targets_disponibles}")
    
    df_targets_filtrado = df_targets[columnas_targets_disponibles].copy()
    
    # HACER MERGE (OUTER JOIN para mantener todos los días)
    print("\n[2/5] Uniendo datasets por FECHA (outer join)...")
    
    df_unido = pd.merge(
        df_inputs,
        df_targets_filtrado,
        on='FECHA',
        how='outer',
        indicator=True
    )
    
    df_unido = df_unido.sort_values('FECHA').reset_index(drop=True)
    print(f"✓ Dataset unido: {len(df_unido)} registros")
    print(f"✓ Columnas totales: {len(df_unido.columns)}")
    
    # ANALIZAR RESULTADO DEL MERGE
    print("\n[3/5] Analizando resultado de la unión...")
    conteo_origen = df_unido['_merge'].value_counts()
    
    print(f"\n📊 Distribución de registros:")
    print(f"  - Ambos datasets (match perfecto): {conteo_origen.get('both', 0)}")
    print(f"  - Solo en Inputs (sin target): {conteo_origen.get('left_only', 0)}")
    print(f"  - Solo en Targets (sin inputs): {conteo_origen.get('right_only', 0)}")
    
    dias_completos = df_unido[df_unido['_merge'] == 'both']
    print(f"\n✅ Días con datos COMPLETOS (inputs + target): {len(dias_completos)}")
    
    dias_sin_target = df_unido[df_unido['_merge'] == 'left_only']
    dias_sin_inputs = df_unido[df_unido['_merge'] == 'right_only']
    
    if len(dias_sin_target) > 0:
        print(f"\n⚠️  Días con INPUTS pero sin TARGET: {len(dias_sin_target)}")
        fechas_muestra = dias_sin_target['FECHA'].tolist()[:5]
        print(f"   Ejemplos: {fechas_muestra}{'...' if len(dias_sin_target) > 5 else ''}")
    
    if len(dias_sin_inputs) > 0:
        print(f"\n⚠️  Días con TARGET pero sin INPUTS: {len(dias_sin_inputs)}")
        fechas_muestra = dias_sin_inputs['FECHA'].tolist()[:5]
        print(f"   Ejemplos: {fechas_muestra}{'...' if len(dias_sin_inputs) > 5 else ''}")
    
    # Eliminar columna auxiliar '_merge'
    df_unido = df_unido.drop(columns=['_merge'])
    
    # FILTRAR REGISTROS COMPLETAMENTE VÁLIDOS
    print("\n[4/5] Filtrando registros completos y válidos...")
    
    condicion_target = df_unido['TARGET'].notna()
    
    # Detectar si tiene algún input ionosférico
    if 'Mh_PRECURSOR' in df_unido.columns:
        condicion_inputs = df_unido['Mh_PRECURSOR'].notna()
    elif 'Mh1' in df_unido.columns:
        condicion_inputs = df_unido['Mh1'].notna()
    else:
        condicion_inputs = True
    
    # Validar extremos según el tipo de dataset
    if columnas_criticas_validas:
        condiciones_extremos = [df_unido[col].eq(True) for col in columnas_criticas_validas if col in df_unido.columns]
        if condiciones_extremos:
            condicion_extremos = pd.concat(condiciones_extremos, axis=1).all(axis=1)
        else:
            condicion_extremos = True
    else:
        condicion_extremos = True
    
    df_completos = df_unido[condicion_target & condicion_inputs & condicion_extremos].copy()
    
    print(f"✓ Registros con TARGET: {condicion_target.sum()}")
    print(f"✓ Registros con INPUTS ionosféricos: {condicion_inputs.sum()}")
    if columnas_criticas_validas:
        print(f"✓ Registros con extremos VÁLIDOS: {condicion_extremos.sum()}")
    print(f"\n✅ Registros COMPLETOS y VÁLIDOS: {len(df_completos)} ({len(df_completos)/len(df_unido)*100:.1f}%)")
    
    # ANÁLISIS DE VALORES NULL
    print("\n[5/5] Análisis de valores NULL...")
    
    columnas_importantes = ['TARGET', 'Bz_mean_16_19', 'E_mean_16_19', 'Kp_last_6h_mean']
    if 'Mh_PRECURSOR' in df_unido.columns:
        columnas_importantes.extend(['Mh_PRECURSOR', 'Mv_PRECURSOR', 'Dh_TOTAL'])
    elif 'Mh1' in df_unido.columns:
        columnas_importantes.extend(['Mh1', 'Mv1', 'Mh2', 'Mv2'])
    
    columnas_a_revisar = [col for col in columnas_importantes if col in df_unido.columns]
    
    null_encontrados = False
    for col in columnas_a_revisar:
        null_count = df_unido[col].isnull().sum()
        if null_count > 0:
            if not null_encontrados:
                print("\nValores NULL en columnas clave:")
                null_encontrados = True
            porcentaje = (null_count / len(df_unido)) * 100
            print(f"  {col:30s}: {null_count:4d} ({porcentaje:5.1f}%)")
    
    if not null_encontrados:
        print("✓ No se encontraron valores NULL en columnas clave")
    
    return df_unido, df_completos

def procesar_grupo_especifico(anio, identificador_meses, directorio, estacion):
    """
    Procesa un grupo específico de meses de un año
    
    Parámetros:
    -----------
    anio : str
        Año a procesar (ej: '2013')
    identificador_meses : str
        Identificador de meses (ej: 'Sept_Oct', 'Ene_Feb_Mar')
    directorio : str
        Directorio donde están los archivos de inputs
    estacion : str
        Nombre de la estación
    
    Retorna:
    --------
    dict : Diccionario con resultados por cada tipo de ventana
    """
    
    print("\n" + "="*80)
    print(f"PROCESANDO: {anio} - {identificador_meses} - ESTACIÓN {estacion.upper()}")
    print("="*80)
    
    # Buscar archivos FUSIONADO que contengan el identificador
    archivos_inputs = {}
    
    if not os.path.exists(directorio):
        print(f"❌ Error: No existe el directorio {directorio}")
        return None
    
    for archivo in os.listdir(directorio):
        if identificador_meses in archivo and archivo.endswith('_FUSIONADO.csv'):
            if '1ventana' in archivo or (('inputs_FUSIONADO.csv' in archivo) and ('2ventanas' not in archivo) and ('3ventanas' not in archivo)):
                archivos_inputs['1ventana'] = os.path.join(directorio, archivo)
            elif '2ventanas' in archivo:
                archivos_inputs['2ventanas'] = os.path.join(directorio, archivo)
            elif '3ventanas' in archivo:
                archivos_inputs['3ventanas'] = os.path.join(directorio, archivo)
    
    if not archivos_inputs:
        print(f"❌ Error: No se encontraron archivos con identificador '{identificador_meses}' en {directorio}")
        return None
    
    print(f"\n✓ Archivos de inputs encontrados:")
    for tipo, ruta in archivos_inputs.items():
        print(f"  - {tipo}: {os.path.basename(ruta)}")
    
    # Extraer meses del nombre del archivo
    primer_archivo = list(archivos_inputs.values())[0]
    nombre_archivo = os.path.basename(primer_archivo)
    meses = extraer_meses_de_nombre(nombre_archivo)
    
    print(f"\n✓ Meses detectados: {meses}")
    
    # Buscar y concatenar archivos target
    print(f"\n✓ Buscando archivos target para {anio}, meses {meses}:")
    archivos_target = encontrar_archivos_target(anio, meses, estacion)
    
    if not archivos_target:
        print(f"❌ Error: No se encontraron archivos target")
        return None
    
    # Concatenar targets
    df_targets = concatenar_targets(archivos_target)
    
    # Procesar cada tipo de ventana
    resultados = {}
    
    for tipo_ventana, archivo_input in archivos_inputs.items():
        print("\n" + "="*80)
        print(f"PROCESANDO: {tipo_ventana.upper()}")
        print("="*80)
        
        # Cargar inputs
        print(f"\n✓ Cargando inputs desde: {os.path.basename(archivo_input)}")
        df_inputs = pd.read_csv(archivo_input)
        print(f"  - Registros: {len(df_inputs)}")
        print(f"  - Columnas: {len(df_inputs.columns)}")
        
        # Unir inputs y targets
        df_unido, df_completos = unir_inputs_y_targets(df_inputs, df_targets, tipo_ventana)
        
        # Guardar resultados
        print("\n" + "="*80)
        print("GUARDANDO RESULTADOS")
        print("="*80)
        
        # Crear directorio de salida
        dir_salida = f"Resultados/{anio}"
        os.makedirs(dir_salida, exist_ok=True)
        
        # Nombres de archivos de salida (incluyen identificador de meses)
        output_completo = f"{dir_salida}/{estacion}_{anio}_{identificador_meses}_{tipo_ventana}_dataset_completo.csv"
        output_validos = f"{dir_salida}/{estacion}_{anio}_{identificador_meses}_{tipo_ventana}_dataset_solo_completos.csv"
        
        df_unido.to_csv(output_completo, index=False)
        print(f"✓ Dataset completo guardado en: {output_completo}")
        
        df_completos.to_csv(output_validos, index=False)
        print(f"✓ Dataset solo completos guardado en: {output_validos}")
        
        # RESUMEN FINAL
        print("\n" + "="*80)
        print(f"RESUMEN FINAL - {tipo_ventana.upper()}")
        print("="*80)
        
        print(f"\nTotal de días en el dataset: {len(df_unido)}")
        print(f"Días con datos completos y válidos: {len(df_completos)} ({len(df_completos)/len(df_unido)*100:.1f}%)")
        print(f"Días incompletos o inválidos: {len(df_unido)-len(df_completos)} ({(len(df_unido)-len(df_completos))/len(df_unido)*100:.1f}%)")
        
        # Distribución del TARGET
        if len(df_completos) > 0 and 'TARGET' in df_completos.columns:
            target_counts = df_completos['TARGET'].value_counts()
            print(f"\n📊 Distribución del TARGET (solo días completos y válidos):")
            print(f"  - TARGET = 0 (Sin centelleo): {target_counts.get(0, 0)} días ({target_counts.get(0, 0)/len(df_completos)*100:.1f}%)")
            print(f"  - TARGET = 1 (Con centelleo): {target_counts.get(1, 0)} días ({target_counts.get(1, 0)/len(df_completos)*100:.1f}%)")
            
            if target_counts.get(1, 0) > 0 and target_counts.get(0, 0) > 0:
                ratio = target_counts.get(0, 0) / target_counts.get(1, 0)
                print(f"  - Ratio 0/1: {ratio:.2f}")
                if ratio > 3:
                    print(f"  ⚠️  Dataset desbalanceado (considerar class_weight o SMOTE)")
                else:
                    print(f"  ✅ Dataset relativamente balanceado")
        
        # Guardar resultados
        resultados[tipo_ventana] = {
            'df_completo': df_unido,
            'df_validos': df_completos,
            'archivo_completo': output_completo,
            'archivo_validos': output_validos
        }
    
    return resultados


# EJECUCIÓN PRINCIPAL
if __name__ == "__main__":
    
    print("\n" + "="*80)
    print("PROCESAMIENTO MANUAL POR GRUPOS DE MESES")
    print("="*80)
    print(f"\nTotal de grupos a procesar: {len(GRUPOS_A_PROCESAR)}")
    print(f"Estación: {ESTACION}")
    
    resultados_globales = {}
    
    for grupo in GRUPOS_A_PROCESAR:
        anio, identificador_meses, directorio = grupo
        
        print(f"\n{'='*80}")
        print(f"Procesando: {anio} - {identificador_meses}")
        print(f"{'='*80}")
        
        try:
            resultados = procesar_grupo_especifico(anio, identificador_meses, directorio, ESTACION)
            if resultados:
                clave = f"{anio}_{identificador_meses}"
                resultados_globales[clave] = resultados
                print(f"\n✅ Grupo {anio} - {identificador_meses} procesado exitosamente")
            else:
                print(f"\n⚠️  Grupo {anio} - {identificador_meses} no pudo ser procesado")
        except Exception as e:
            print(f"\n❌ Error procesando {anio} - {identificador_meses}: {e}")
            import traceback
            traceback.print_exc()
    
    # RESUMEN GLOBAL
    print("\n" + "="*80)
    print("RESUMEN GLOBAL DE PROCESAMIENTO")
    print("="*80)
    
    for grupo_key, resultados in resultados_globales.items():
        print(f"\n{grupo_key}:")
        for tipo_ventana, datos in resultados.items():
            n_completos = len(datos['df_completo'])
            n_validos = len(datos['df_validos'])
            print(f"  {tipo_ventana}: {n_validos}/{n_completos} registros válidos ({n_validos/n_completos*100:.1f}%)")
    
    print("\n" + "="*80)
    print("✓ PROCESAMIENTO COMPLETADO")
    print("="*80)
    
    print("\n✅ Script finalizado exitosamente")


PROCESAMIENTO MANUAL POR GRUPOS DE MESES

Total de grupos a procesar: 12
Estación: huancayo

Procesando: 2013 - Ene_Feb_Mar_Abr_May_Jun_Jul_Ago

PROCESANDO: 2013 - Ene_Feb_Mar_Abr_May_Jun_Jul_Ago - ESTACIÓN HUANCAYO

✓ Archivos de inputs encontrados:
  - 2ventanas: 2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase_inputs_2ventanas_FUSIONADO.csv
  - 3ventanas: 2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase_inputs_3ventanas_FUSIONADO.csv
  - 1ventana: 2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase_inputs_FUSIONADO.csv

✓ Meses detectados: ['01', '02', '03', '04', '05', '06', '07', '08']

✓ Buscando archivos target para 2013, meses ['01', '02', '03', '04', '05', '06', '07', '08']:
  ⚠️  NO encontrado: 2013/2013_01/TargetFinal/huancayo_2013_01_S4_Calculado_Target_Completo.csv
  ⚠️  NO encontrado: 2013/2013_02/TargetFinal/huancayo_2013_02_S4_Calculado_Target_Completo.csv
  ⚠️  NO encontrado: 2013/2013_03/TargetFinal/huancayo_2013_03_S4_Calculado_Target_Completo.csv
  ✓ Encontrado: 2013/2013_04/Targ

## Datasets completos separados por años

Te provee datasets sin valores nulos (agresivo)

In [2]:
"""
UNIÓN AUTOMÁTICA DE INPUTS Y TARGETS - VERSIÓN CON CONSOLIDACIÓN POR AÑO
1. Procesa grupos individuales de meses
2. Detecta duplicados en extremos de grupos contiguos
3. Resuelve conflictos quedándose con la fila con menos nulls
4. Genera un dataset consolidado por año
"""

import pandas as pd
import numpy as np
import os
from pathlib import Path
import re

# ============================================================================
# 🎯 CONFIGURACIÓN GLOBAL - MODIFICAR AQUÍ
# ============================================================================

GRUPOS_A_PROCESAR = [
    # 2013
    ['2013', 'Ene_Feb_Mar_Abr_May_Jun_Jul_Ago', 'Inputs/2013'],
    ['2013', 'Sept_Oct', 'Inputs/2013'],
    ['2013', 'Nov_Dic', 'Inputs/2013'],
    
    # 2014
    ['2014', 'Ene_Feb_Mar', 'Inputs/2014'],
    ['2014', 'Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic', 'Inputs/2014'],
    
    # 2015
    ['2015', 'Ene_Feb', 'Inputs/2015'],
    ['2015', 'Mar', 'Inputs/2015'],
    ['2015', 'Abr_May_Jun_Jul_Ago', 'Inputs/2015'],
    ['2015', 'Sept_Oct', 'Inputs/2015'],
    ['2015', 'Nov_Dic', 'Inputs/2015'],
    
    # 2016
    ['2016', 'Ene_Feb', 'Inputs/2016'],
    ['2016', 'Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic', 'Inputs/2016']
]

ESTACION = 'huancayo'

# ============================================================================
# CÓDIGO ORIGINAL - NO MODIFICAR
# ============================================================================

MESES_MAP = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
    'Sept': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
}

def extraer_meses_de_nombre(nombre_archivo):
    """Extrae los meses del nombre del archivo de inputs"""
    meses_encontrados = []
    for mes_str, mes_num in MESES_MAP.items():
        if mes_str in nombre_archivo:
            meses_encontrados.append(mes_num)
    return sorted(meses_encontrados)

def encontrar_archivos_target(anio, meses, estacion):
    """Encuentra los archivos target correspondientes a los meses especificados"""
    archivos_target = []
    
    for mes in meses:
        ruta_target = f"{anio}/{anio}_{mes}/TargetFinal/{estacion}_{anio}_{mes}_S4_Calculado_Target_Completo.csv"
        
        if os.path.exists(ruta_target):
            archivos_target.append(ruta_target)
            print(f"  ✓ Encontrado: {ruta_target}")
        else:
            print(f"  ⚠️  NO encontrado: {ruta_target}")
    
    return archivos_target

def concatenar_targets(archivos_target):
    """Concatena múltiples archivos target en un solo DataFrame"""
    if not archivos_target:
        raise ValueError("No se encontraron archivos target para concatenar")
    
    print(f"\n  Concatenando {len(archivos_target)} archivo(s) target...")
    
    dfs = []
    for archivo in archivos_target:
        df = pd.read_csv(archivo)

        if 'FECHA_PERU' in df.columns:
            df = df.rename(columns={'FECHA_PERU': 'FECHA'})

        dfs.append(df)
        print(f"    - {archivo}: {len(df)} registros")
    
    df_concatenado = pd.concat(dfs, ignore_index=True)
    
    if 'FECHA' in df_concatenado.columns:
        df_concatenado['FECHA'] = pd.to_datetime(df_concatenado['FECHA']).dt.date
        duplicados_antes = len(df_concatenado)
        df_concatenado = df_concatenado.drop_duplicates(subset=['FECHA'], keep='first')
        duplicados_despues = len(df_concatenado)
        
        if duplicados_antes != duplicados_despues:
            print(f"    ⚠️  Se eliminaron {duplicados_antes - duplicados_despues} duplicados")
    
    df_concatenado = df_concatenado.sort_values('FECHA').reset_index(drop=True)
    
    print(f"  ✓ Total concatenado: {len(df_concatenado)} registros únicos")
    
    return df_concatenado

def unir_inputs_y_targets(df_inputs, df_targets, tipo_ventana):
    """Une inputs y targets (LÓGICA ORIGINAL COMPLETA)"""
    
    print("\n[1/5] Preparando datos para unión...")
    
    df_inputs['FECHA'] = pd.to_datetime(df_inputs['FECHA']).dt.date
    df_targets['FECHA'] = pd.to_datetime(df_targets['FECHA']).dt.date
    
    if 'Mh_PRECURSOR' in df_inputs.columns:
        tipo_dataset = "1 ventana"
        columnas_criticas_validas = ['extremos_validos']
    elif 'Mh1' in df_inputs.columns and 'Mh2' in df_inputs.columns:
        if 'Mh3' in df_inputs.columns:
            tipo_dataset = "3 ventanas"
            columnas_criticas_validas = ['extremos_validos_SW1', 'extremos_validos_SW2', 'extremos_validos_SW3']
        else:
            tipo_dataset = "2 ventanas"
            columnas_criticas_validas = ['extremos_validos_SW1', 'extremos_validos_SW2']
    else:
        tipo_dataset = "desconocido"
        columnas_criticas_validas = []
    
    print(f"✓ Tipo de dataset detectado: {tipo_dataset}")
    print(f"✓ Columnas de validación: {columnas_criticas_validas}")
    print(f"✓ Manteniendo todas las {len(df_inputs.columns)} columnas de inputs")
    
    columnas_targets = [
        'FECHA', 'TARGET', 'VECES_SUPERO_UMBRAL', 'S4_MAX', 'S4_MEAN', 
        'S4_STD', 'S4_MIN', 'TOTAL_MEDICIONES', 'PORCENTAJE_CENTELLEO', 
        'HORA_PICO', 'PRN_PICO'
    ]
    
    columnas_targets_disponibles = [col for col in columnas_targets if col in df_targets.columns]
    print(f"✓ Columnas de targets a conservar: {columnas_targets_disponibles}")
    
    df_targets_filtrado = df_targets[columnas_targets_disponibles].copy()
    
    print("\n[2/5] Uniendo datasets por FECHA (outer join)...")
    
    df_unido = pd.merge(
        df_inputs,
        df_targets_filtrado,
        on='FECHA',
        how='outer',
        indicator=True
    )
    
    df_unido = df_unido.sort_values('FECHA').reset_index(drop=True)
    print(f"✓ Dataset unido: {len(df_unido)} registros")
    print(f"✓ Columnas totales: {len(df_unido.columns)}")
    
    print("\n[3/5] Analizando resultado de la unión...")
    conteo_origen = df_unido['_merge'].value_counts()
    
    print(f"\n📊 Distribución de registros:")
    print(f"  - Ambos datasets (match perfecto): {conteo_origen.get('both', 0)}")
    print(f"  - Solo en Inputs (sin target): {conteo_origen.get('left_only', 0)}")
    print(f"  - Solo en Targets (sin inputs): {conteo_origen.get('right_only', 0)}")
    
    dias_completos = df_unido[df_unido['_merge'] == 'both']
    print(f"\n✅ Días con datos COMPLETOS (inputs + target): {len(dias_completos)}")
    
    dias_sin_target = df_unido[df_unido['_merge'] == 'left_only']
    dias_sin_inputs = df_unido[df_unido['_merge'] == 'right_only']
    
    if len(dias_sin_target) > 0:
        print(f"\n⚠️  Días con INPUTS pero sin TARGET: {len(dias_sin_target)}")
        fechas_muestra = dias_sin_target['FECHA'].tolist()[:5]
        print(f"   Ejemplos: {fechas_muestra}{'...' if len(dias_sin_target) > 5 else ''}")
    
    if len(dias_sin_inputs) > 0:
        print(f"\n⚠️  Días con TARGET pero sin INPUTS: {len(dias_sin_inputs)}")
        fechas_muestra = dias_sin_inputs['FECHA'].tolist()[:5]
        print(f"   Ejemplos: {fechas_muestra}{'...' if len(dias_sin_inputs) > 5 else ''}")
    
    df_unido = df_unido.drop(columns=['_merge'])
    
    print("\n[4/5] Filtrando registros completos y válidos...")
    
    condicion_target = df_unido['TARGET'].notna()
    
    if 'Mh_PRECURSOR' in df_unido.columns:
        condicion_inputs = df_unido['Mh_PRECURSOR'].notna()
    elif 'Mh1' in df_unido.columns:
        condicion_inputs = df_unido['Mh1'].notna()
    else:
        condicion_inputs = True
    
    if columnas_criticas_validas:
        condiciones_extremos = [df_unido[col].eq(True) for col in columnas_criticas_validas if col in df_unido.columns]
        if condiciones_extremos:
            condicion_extremos = pd.concat(condiciones_extremos, axis=1).all(axis=1)
        else:
            condicion_extremos = True
    else:
        condicion_extremos = True
    
    df_completos = df_unido[condicion_target & condicion_inputs & condicion_extremos].copy()
    
    print(f"✓ Registros con TARGET: {condicion_target.sum()}")
    print(f"✓ Registros con INPUTS ionosféricos: {condicion_inputs.sum()}")
    if columnas_criticas_validas:
        print(f"✓ Registros con extremos VÁLIDOS: {condicion_extremos.sum()}")
    print(f"\n✅ Registros COMPLETOS y VÁLIDOS: {len(df_completos)} ({len(df_completos)/len(df_unido)*100:.1f}%)")
    
    print("\n[5/5] Análisis de valores NULL...")
    
    columnas_importantes = ['TARGET', 'Bz_mean_16_19', 'E_mean_16_19', 'Kp_last_6h_mean']
    if 'Mh_PRECURSOR' in df_unido.columns:
        columnas_importantes.extend(['Mh_PRECURSOR', 'Mv_PRECURSOR', 'Dh_TOTAL'])
    elif 'Mh1' in df_unido.columns:
        columnas_importantes.extend(['Mh1', 'Mv1', 'Mh2', 'Mv2'])
    
    columnas_a_revisar = [col for col in columnas_importantes if col in df_unido.columns]
    
    null_encontrados = False
    for col in columnas_a_revisar:
        null_count = df_unido[col].isnull().sum()
        if null_count > 0:
            if not null_encontrados:
                print("\nValores NULL en columnas clave:")
                null_encontrados = True
            porcentaje = (null_count / len(df_unido)) * 100
            print(f"  {col:30s}: {null_count:4d} ({porcentaje:5.1f}%)")
    
    if not null_encontrados:
        print("✓ No se encontraron valores NULL en columnas clave")
    
    return df_unido, df_completos

def procesar_grupo_especifico(anio, identificador_meses, directorio, estacion):
    """Procesa un grupo específico de meses de un año"""
    
    print("\n" + "="*80)
    print(f"PROCESANDO: {anio} - {identificador_meses} - ESTACIÓN {estacion.upper()}")
    print("="*80)
    
    archivos_inputs = {}
    
    if not os.path.exists(directorio):
        print(f"❌ Error: No existe el directorio {directorio}")
        return None
    
    for archivo in os.listdir(directorio):
        if identificador_meses in archivo and archivo.endswith('_FUSIONADO.csv'):
            if '1ventana' in archivo or (('inputs_FUSIONADO.csv' in archivo) and ('2ventanas' not in archivo) and ('3ventanas' not in archivo)):
                archivos_inputs['1ventana'] = os.path.join(directorio, archivo)
            elif '2ventanas' in archivo:
                archivos_inputs['2ventanas'] = os.path.join(directorio, archivo)
            elif '3ventanas' in archivo:
                archivos_inputs['3ventanas'] = os.path.join(directorio, archivo)
    
    if not archivos_inputs:
        print(f"❌ Error: No se encontraron archivos con identificador '{identificador_meses}' en {directorio}")
        return None
    
    print(f"\n✓ Archivos de inputs encontrados:")
    for tipo, ruta in archivos_inputs.items():
        print(f"  - {tipo}: {os.path.basename(ruta)}")
    
    primer_archivo = list(archivos_inputs.values())[0]
    nombre_archivo = os.path.basename(primer_archivo)
    meses = extraer_meses_de_nombre(nombre_archivo)
    
    print(f"\n✓ Meses detectados: {meses}")
    
    print(f"\n✓ Buscando archivos target para {anio}, meses {meses}:")
    archivos_target = encontrar_archivos_target(anio, meses, estacion)
    
    if not archivos_target:
        print(f"❌ Error: No se encontraron archivos target")
        return None
    
    df_targets = concatenar_targets(archivos_target)
    
    resultados = {}
    
    for tipo_ventana, archivo_input in archivos_inputs.items():
        print("\n" + "="*80)
        print(f"PROCESANDO: {tipo_ventana.upper()}")
        print("="*80)
        
        print(f"\n✓ Cargando inputs desde: {os.path.basename(archivo_input)}")
        df_inputs = pd.read_csv(archivo_input)
        print(f"  - Registros: {len(df_inputs)}")
        print(f"  - Columnas: {len(df_inputs.columns)}")
        
        df_unido, df_completos = unir_inputs_y_targets(df_inputs, df_targets, tipo_ventana)
        
        print("\n" + "="*80)
        print("GUARDANDO RESULTADOS")
        print("="*80)
        
        dir_salida = f"Resultados/{anio}"
        os.makedirs(dir_salida, exist_ok=True)
        
        output_completo = f"{dir_salida}/{estacion}_{anio}_{identificador_meses}_{tipo_ventana}_dataset_completo.csv"
        output_validos = f"{dir_salida}/{estacion}_{anio}_{identificador_meses}_{tipo_ventana}_dataset_solo_completos.csv"
        
        df_unido.to_csv(output_completo, index=False)
        print(f"✓ Dataset completo guardado en: {output_completo}")
        
        df_completos.to_csv(output_validos, index=False)
        print(f"✓ Dataset solo completos guardado en: {output_validos}")
        
        print("\n" + "="*80)
        print(f"RESUMEN FINAL - {tipo_ventana.upper()}")
        print("="*80)
        
        print(f"\nTotal de días en el dataset: {len(df_unido)}")
        print(f"Días con datos completos y válidos: {len(df_completos)} ({len(df_completos)/len(df_unido)*100:.1f}%)")
        print(f"Días incompletos o inválidos: {len(df_unido)-len(df_completos)} ({(len(df_unido)-len(df_completos))/len(df_unido)*100:.1f}%)")
        
        if len(df_completos) > 0 and 'TARGET' in df_completos.columns:
            target_counts = df_completos['TARGET'].value_counts()
            print(f"\n📊 Distribución del TARGET (solo días completos y válidos):")
            print(f"  - TARGET = 0 (Sin centelleo): {target_counts.get(0, 0)} días ({target_counts.get(0, 0)/len(df_completos)*100:.1f}%)")
            print(f"  - TARGET = 1 (Con centelleo): {target_counts.get(1, 0)} días ({target_counts.get(1, 0)/len(df_completos)*100:.1f}%)")
            
            if target_counts.get(1, 0) > 0 and target_counts.get(0, 0) > 0:
                ratio = target_counts.get(0, 0) / target_counts.get(1, 0)
                print(f"  - Ratio 0/1: {ratio:.2f}")
                if ratio > 3:
                    print(f"  ⚠️  Dataset desbalanceado (considerar class_weight o SMOTE)")
                else:
                    print(f"  ✅ Dataset relativamente balanceado")
        
        resultados[tipo_ventana] = {
            'df_completo': df_unido,
            'df_validos': df_completos,
            'archivo_completo': output_completo,
            'archivo_validos': output_validos
        }
    
    return resultados

def consolidar_por_anio(resultados_globales, estacion):
    """
    Consolida todos los grupos de cada año en un solo DataFrame
    Resuelve duplicados en extremos quedándose con la fila con menos nulls
    """
    
    print("\n" + "="*80)
    print("🔄 CONSOLIDACIÓN DE GRUPOS POR AÑO")
    print("="*80)
    
    # Agrupar por año
    grupos_por_anio = {}
    for clave_grupo in resultados_globales.keys():
        anio = clave_grupo.split('_')[0]
        if anio not in grupos_por_anio:
            grupos_por_anio[anio] = []
        grupos_por_anio[anio].append(clave_grupo)
    
    consolidados = {}
    
    for anio, grupos in sorted(grupos_por_anio.items()):
        print(f"\n{'='*80}")
        print(f"CONSOLIDANDO AÑO {anio}")
        print(f"{'='*80}")
        print(f"Grupos a consolidar: {len(grupos)}")
        for grupo in grupos:
            print(f"  - {grupo}")
        
        # Procesar por tipo de ventana
        for tipo_ventana in ['1ventana', '2ventanas', '3ventanas']:
            print(f"\n--- Consolidando {tipo_ventana.upper()} ---")
            
            dfs_a_unir = []
            
            # Recopilar todos los DataFrames de este tipo de ventana
            for clave_grupo in grupos:
                if clave_grupo in resultados_globales:
                    resultados = resultados_globales[clave_grupo]
                    if tipo_ventana in resultados:
                        df = resultados[tipo_ventana]['df_validos'].copy()
                        if len(df) > 0:
                            dfs_a_unir.append(df)
                            print(f"  ✓ {clave_grupo}: {len(df)} registros")
            
            if not dfs_a_unir:
                print(f"  ⚠️  No hay datos para consolidar en {tipo_ventana}")
                continue
            
            # Concatenar todos los DataFrames
            df_consolidado = pd.concat(dfs_a_unir, ignore_index=True)
            df_consolidado = df_consolidado.sort_values('FECHA').reset_index(drop=True)
            
            print(f"\n  📊 Antes de resolver duplicados: {len(df_consolidado)} registros")
            
            # 🔍 DETECTAR Y RESOLVER DUPLICADOS
            duplicados = df_consolidado.duplicated(subset=['FECHA'], keep=False)
            
            if duplicados.sum() > 0:
                print(f"  ⚠️  Encontrados {duplicados.sum()} registros duplicados en {duplicados.sum()//2} fechas")
                
                fechas_duplicadas = df_consolidado[duplicados]['FECHA'].unique()
                print(f"  🔧 Resolviendo duplicados por cantidad de nulls...")
                
                filas_a_mantener = []
                
                for fecha in fechas_duplicadas:
                    filas_fecha = df_consolidado[df_consolidado['FECHA'] == fecha]
                    
                    # Contar nulls por fila
                    nulls_por_fila = filas_fecha.isnull().sum(axis=1)
                    
                    # Quedarse con la fila con MENOS nulls
                    idx_mejor = nulls_por_fila.idxmin()
                    fila_mejor = filas_fecha.loc[idx_mejor]
                    
                    filas_a_mantener.append(idx_mejor)
                    
                    nulls_minimo = nulls_por_fila.min()
                    nulls_maximo = nulls_por_fila.max()
                    print(f"    • {fecha}: Seleccionada fila con {nulls_minimo} nulls (descartada con {nulls_maximo} nulls)")
                
                # Crear DataFrame final: no duplicados + mejores de duplicados
                mask_no_duplicados = ~duplicados
                df_no_duplicados = df_consolidado[mask_no_duplicados]
                df_mejores_duplicados = df_consolidado.loc[filas_a_mantener]
                
                df_consolidado = pd.concat([df_no_duplicados, df_mejores_duplicados], ignore_index=True)
                df_consolidado = df_consolidado.sort_values('FECHA').reset_index(drop=True)
                
                print(f"  ✅ Después de resolver duplicados: {len(df_consolidado)} registros únicos")
            else:
                print(f"  ✓ No se encontraron duplicados")
            
            # Guardar consolidado
            dir_salida = f"Resultados/{anio}"
            os.makedirs(dir_salida, exist_ok=True)
            
            archivo_consolidado = f"{dir_salida}/{estacion}_{anio}_{tipo_ventana}_CONSOLIDADO.csv"
            df_consolidado.to_csv(archivo_consolidado, index=False)
            
            print(f"\n  💾 Guardado: {archivo_consolidado}")
            print(f"  📊 Total registros: {len(df_consolidado)}")
            
            # Estadísticas del consolidado
            if 'TARGET' in df_consolidado.columns:
                target_counts = df_consolidado['TARGET'].value_counts()
                print(f"\n  📈 Distribución TARGET:")
                print(f"    - TARGET = 0: {target_counts.get(0, 0)} días ({target_counts.get(0, 0)/len(df_consolidado)*100:.1f}%)")
                print(f"    - TARGET = 1: {target_counts.get(1, 0)} días ({target_counts.get(1, 0)/len(df_consolidado)*100:.1f}%)")
            
            # Guardar en diccionario
            if anio not in consolidados:
                consolidados[anio] = {}
            consolidados[anio][tipo_ventana] = {
                'df': df_consolidado,
                'archivo': archivo_consolidado
            }
    
    return consolidados


# EJECUCIÓN PRINCIPAL
if __name__ == "__main__":
    
    print("\n" + "="*80)
    print("PROCESAMIENTO MANUAL POR GRUPOS DE MESES")
    print("="*80)
    print(f"\nTotal de grupos a procesar: {len(GRUPOS_A_PROCESAR)}")
    print(f"Estación: {ESTACION}")
    
    resultados_globales = {}
    
    # PASO 1: Procesar cada grupo individualmente
    for grupo in GRUPOS_A_PROCESAR:
        anio, identificador_meses, directorio = grupo
        
        print(f"\n{'='*80}")
        print(f"Procesando: {anio} - {identificador_meses}")
        print(f"{'='*80}")
        
        try:
            resultados = procesar_grupo_especifico(anio, identificador_meses, directorio, ESTACION)
            if resultados:
                clave = f"{anio}_{identificador_meses}"
                resultados_globales[clave] = resultados
                print(f"\n✅ Grupo {anio} - {identificador_meses} procesado exitosamente")
            else:
                print(f"\n⚠️  Grupo {anio} - {identificador_meses} no pudo ser procesado")
        except Exception as e:
            print(f"\n❌ Error procesando {anio} - {identificador_meses}: {e}")
            import traceback
            traceback.print_exc()
    
    # PASO 2: Consolidar por año
    print("\n" + "="*80)
    print("INICIANDO CONSOLIDACIÓN POR AÑO")
    print("="*80)
    
    consolidados = consolidar_por_anio(resultados_globales, ESTACION)
    
    # RESUMEN GLOBAL
    print("\n" + "="*80)
    print("RESUMEN GLOBAL DE PROCESAMIENTO")
    print("="*80)
    
    print("\n📂 ARCHIVOS INDIVIDUALES POR GRUPO:")
    for grupo_key, resultados in resultados_globales.items():
        print(f"\n{grupo_key}:")
        for tipo_ventana, datos in resultados.items():
            n_completos = len(datos['df_completo'])
            n_validos = len(datos['df_validos'])
            print(f"  {tipo_ventana}: {n_validos}/{n_completos} registros válidos ({n_validos/n_completos*100:.1f}%)")
    
    print("\n" + "="*80)
    print("📊 DATASETS CONSOLIDADOS POR AÑO:")
    print("="*80)
    
    for anio in sorted(consolidados.keys()):
        print(f"\n{anio}:")
        for tipo_ventana in ['1ventana', '2ventanas', '3ventanas']:
            if tipo_ventana in consolidados[anio]:
                datos = consolidados[anio][tipo_ventana]
                print(f"  {tipo_ventana}: {len(datos['df'])} registros")
                print(f"    → {datos['archivo']}")
    
    print("\n" + "="*80)
    print("✓ PROCESAMIENTO COMPLETADO")
    print("="*80)
    
    print("\n✅ Script finalizado exitosamente")


PROCESAMIENTO MANUAL POR GRUPOS DE MESES

Total de grupos a procesar: 12
Estación: huancayo

Procesando: 2013 - Ene_Feb_Mar_Abr_May_Jun_Jul_Ago

PROCESANDO: 2013 - Ene_Feb_Mar_Abr_May_Jun_Jul_Ago - ESTACIÓN HUANCAYO

✓ Archivos de inputs encontrados:
  - 2ventanas: 2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase_inputs_2ventanas_FUSIONADO.csv
  - 3ventanas: 2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase_inputs_3ventanas_FUSIONADO.csv
  - 1ventana: 2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_DIDBase_inputs_FUSIONADO.csv

✓ Meses detectados: ['01', '02', '03', '04', '05', '06', '07', '08']

✓ Buscando archivos target para 2013, meses ['01', '02', '03', '04', '05', '06', '07', '08']:
  ⚠️  NO encontrado: 2013/2013_01/TargetFinal/huancayo_2013_01_S4_Calculado_Target_Completo.csv
  ⚠️  NO encontrado: 2013/2013_02/TargetFinal/huancayo_2013_02_S4_Calculado_Target_Completo.csv
  ⚠️  NO encontrado: 2013/2013_03/TargetFinal/huancayo_2013_03_S4_Calculado_Target_Completo.csv
  ✓ Encontrado: 2013/2013_04/Targ

Te provee datasets con algunos valores nulos (menos agresivo)

In [3]:
"""
CONSOLIDADOR DE DATASETS COMPLETOS POR AÑO
Lee los archivos _completo.csv generados previamente
Resuelve duplicados en extremos por cantidad de nulls
Genera UN archivo consolidado por año y tipo de ventana
"""

import pandas as pd
import numpy as np
import os
from pathlib import Path
import re

# ============================================================================
# 🎯 CONFIGURACIÓN GLOBAL - MODIFICAR AQUÍ
# ============================================================================

# Lista de grupos a consolidar: [año, identificador_meses]
GRUPOS_A_CONSOLIDAR = [
    # 2013
    ['2013', 'Ene_Feb_Mar_Abr_May_Jun_Jul_Ago'],
    ['2013', 'Sept_Oct'],
    ['2013', 'Nov_Dic'],
    
    # 2014
    ['2014', 'Ene_Feb_Mar'],
    ['2014', 'Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic'],
    
    # 2015
    ['2015', 'Ene_Feb'],
    ['2015', 'Mar'],
    ['2015', 'Abr_May_Jun_Jul_Ago'],
    ['2015', 'Sept_Oct'],
    ['2015', 'Nov_Dic'],
    
    # 2016
    ['2016', 'Ene_Feb'],
    ['2016', 'Mar_Abr_May_Jun_Jul_Ago_Sept_Oct_Nov_Dic']
]

ESTACION = 'huancayo'
DIRECTORIO_BASE = 'Resultados'

# ============================================================================
# CÓDIGO PRINCIPAL
# ============================================================================

def consolidar_datasets_completos():
    """
    Consolida todos los archivos _completo.csv por año y tipo de ventana
    Resuelve duplicados quedándose con la fila con menos nulls
    """
    
    print("\n" + "="*80)
    print("🔄 CONSOLIDACIÓN DE DATASETS COMPLETOS POR AÑO")
    print("="*80)
    
    # Agrupar por año
    grupos_por_anio = {}
    for grupo in GRUPOS_A_CONSOLIDAR:
        anio, identificador_meses = grupo
        if anio not in grupos_por_anio:
            grupos_por_anio[anio] = []
        grupos_por_anio[anio].append(identificador_meses)
    
    consolidados = {}
    
    for anio, identificadores in sorted(grupos_por_anio.items()):
        print(f"\n{'='*80}")
        print(f"CONSOLIDANDO AÑO {anio}")
        print(f"{'='*80}")
        print(f"Grupos a consolidar: {len(identificadores)}")
        for ident in identificadores:
            print(f"  - {ident}")
        
        # Procesar por tipo de ventana
        for tipo_ventana in ['1ventana', '2ventanas', '3ventanas']:
            print(f"\n--- Consolidando {tipo_ventana.upper()} ---")
            
            archivos_encontrados = []
            dfs_a_unir = []
            
            # Buscar y cargar archivos
            for identificador in identificadores:
                archivo = f"{DIRECTORIO_BASE}/{anio}/{ESTACION}_{anio}_{identificador}_{tipo_ventana}_dataset_completo.csv"
                
                if os.path.exists(archivo):
                    df = pd.read_csv(archivo)
                    
                    # Convertir FECHA a datetime.date
                    if 'FECHA' in df.columns:
                        df['FECHA'] = pd.to_datetime(df['FECHA']).dt.date
                    
                    archivos_encontrados.append(archivo)
                    dfs_a_unir.append(df)
                    print(f"  ✓ {os.path.basename(archivo)}: {len(df)} registros")
                else:
                    print(f"  ⚠️  No encontrado: {os.path.basename(archivo)}")
            
            if not dfs_a_unir:
                print(f"  ❌ No hay archivos para consolidar en {tipo_ventana}")
                continue
            
            # Concatenar todos los DataFrames
            df_consolidado = pd.concat(dfs_a_unir, ignore_index=True)
            df_consolidado = df_consolidado.sort_values('FECHA').reset_index(drop=True)
            
            print(f"\n  📊 Antes de resolver duplicados: {len(df_consolidado)} registros")
            
            # 🔍 DETECTAR Y RESOLVER DUPLICADOS
            duplicados = df_consolidado.duplicated(subset=['FECHA'], keep=False)
            
            if duplicados.sum() > 0:
                print(f"  ⚠️  Encontrados {duplicados.sum()} registros duplicados en {duplicados.sum()//2} fechas")
                
                fechas_duplicadas = df_consolidado[duplicados]['FECHA'].unique()
                print(f"  🔧 Resolviendo duplicados por cantidad de nulls...")
                
                filas_a_mantener = []
                
                for fecha in fechas_duplicadas:
                    filas_fecha = df_consolidado[df_consolidado['FECHA'] == fecha]
                    
                    # Contar nulls por fila
                    nulls_por_fila = filas_fecha.isnull().sum(axis=1)
                    
                    # Quedarse con la fila con MENOS nulls
                    idx_mejor = nulls_por_fila.idxmin()
                    
                    filas_a_mantener.append(idx_mejor)
                    
                    nulls_minimo = nulls_por_fila.min()
                    nulls_maximo = nulls_por_fila.max()
                    print(f"    • {fecha}: Seleccionada fila con {nulls_minimo} nulls (descartada con {nulls_maximo} nulls)")
                
                # Crear DataFrame final: no duplicados + mejores de duplicados
                mask_no_duplicados = ~duplicados
                df_no_duplicados = df_consolidado[mask_no_duplicados]
                df_mejores_duplicados = df_consolidado.loc[filas_a_mantener]
                
                df_consolidado = pd.concat([df_no_duplicados, df_mejores_duplicados], ignore_index=True)
                df_consolidado = df_consolidado.sort_values('FECHA').reset_index(drop=True)
                
                print(f"  ✅ Después de resolver duplicados: {len(df_consolidado)} registros únicos")
            else:
                print(f"  ✓ No se encontraron duplicados")
            
            # Guardar consolidado
            dir_salida = f"{DIRECTORIO_BASE}/{anio}"
            os.makedirs(dir_salida, exist_ok=True)
            
            archivo_consolidado = f"{dir_salida}/{ESTACION}_{anio}_{tipo_ventana}_CONSOLIDADO_COMPLETO.csv"
            df_consolidado.to_csv(archivo_consolidado, index=False)
            
            print(f"\n  💾 Guardado: {archivo_consolidado}")
            print(f"  📊 Total registros: {len(df_consolidado)}")
            
            # Estadísticas del consolidado
            if 'TARGET' in df_consolidado.columns:
                total_con_target = df_consolidado['TARGET'].notna().sum()
                total_sin_target = df_consolidado['TARGET'].isna().sum()
                
                print(f"\n  📈 Análisis de TARGET:")
                print(f"    - Registros CON target: {total_con_target} ({total_con_target/len(df_consolidado)*100:.1f}%)")
                print(f"    - Registros SIN target: {total_sin_target} ({total_sin_target/len(df_consolidado)*100:.1f}%)")
                
                if total_con_target > 0:
                    target_counts = df_consolidado['TARGET'].value_counts()
                    print(f"\n    Distribución del TARGET (solo registros con target):")
                    print(f"    - TARGET = 0: {target_counts.get(0, 0)} días ({target_counts.get(0, 0)/total_con_target*100:.1f}%)")
                    print(f"    - TARGET = 1: {target_counts.get(1, 0)} días ({target_counts.get(1, 0)/total_con_target*100:.1f}%)")
            
            # Análisis de valores NULL en columnas importantes
            columnas_importantes = ['TARGET', 'Bz_mean_16_19', 'E_mean_16_19', 'Kp_last_6h_mean']
            if 'Mh_PRECURSOR' in df_consolidado.columns:
                columnas_importantes.extend(['Mh_PRECURSOR', 'Mv_PRECURSOR', 'Dh_TOTAL'])
            elif 'Mh1' in df_consolidado.columns:
                columnas_importantes.extend(['Mh1', 'Mv1', 'Mh2', 'Mv2'])
            
            columnas_a_revisar = [col for col in columnas_importantes if col in df_consolidado.columns]
            
            print(f"\n  🔍 Valores NULL en columnas clave:")
            for col in columnas_a_revisar:
                null_count = df_consolidado[col].isnull().sum()
                porcentaje = (null_count / len(df_consolidado)) * 100
                print(f"    {col:30s}: {null_count:4d} ({porcentaje:5.1f}%)")
            
            # Guardar en diccionario
            if anio not in consolidados:
                consolidados[anio] = {}
            consolidados[anio][tipo_ventana] = {
                'df': df_consolidado,
                'archivo': archivo_consolidado,
                'archivos_origen': archivos_encontrados
            }
    
    return consolidados


def generar_reporte_final(consolidados):
    """
    Genera un reporte final con estadísticas de todos los años consolidados
    """
    
    print("\n" + "="*80)
    print("📊 REPORTE FINAL DE CONSOLIDACIÓN")
    print("="*80)
    
    for anio in sorted(consolidados.keys()):
        print(f"\n{'='*60}")
        print(f"AÑO {anio}")
        print(f"{'='*60}")
        
        for tipo_ventana in ['1ventana', '2ventanas', '3ventanas']:
            if tipo_ventana in consolidados[anio]:
                datos = consolidados[anio][tipo_ventana]
                df = datos['df']
                
                print(f"\n{tipo_ventana.upper()}:")
                print(f"  📁 Archivo: {os.path.basename(datos['archivo'])}")
                print(f"  📊 Total registros: {len(df)}")
                print(f"  🗂️  Archivos origen: {len(datos['archivos_origen'])}")
                
                if 'TARGET' in df.columns:
                    con_target = df['TARGET'].notna().sum()
                    sin_target = df['TARGET'].isna().sum()
                    print(f"  ✅ Con TARGET: {con_target} ({con_target/len(df)*100:.1f}%)")
                    print(f"  ❌ Sin TARGET: {sin_target} ({sin_target/len(df)*100:.1f}%)")
    
    print("\n" + "="*80)
    print("✓ CONSOLIDACIÓN COMPLETADA")
    print("="*80)


# EJECUCIÓN PRINCIPAL
if __name__ == "__main__":
    
    print("\n" + "="*80)
    print("CONSOLIDADOR DE DATASETS COMPLETOS")
    print("="*80)
    print(f"\nEstación: {ESTACION}")
    print(f"Directorio base: {DIRECTORIO_BASE}")
    print(f"Total de grupos configurados: {len(GRUPOS_A_CONSOLIDAR)}")
    
    # Consolidar datasets
    consolidados = consolidar_datasets_completos()
    
    # Generar reporte final
    if consolidados:
        generar_reporte_final(consolidados)
    else:
        print("\n⚠️  No se generaron archivos consolidados")
    
    print("\n✅ Script finalizado exitosamente")


CONSOLIDADOR DE DATASETS COMPLETOS

Estación: huancayo
Directorio base: Resultados
Total de grupos configurados: 12

🔄 CONSOLIDACIÓN DE DATASETS COMPLETOS POR AÑO

CONSOLIDANDO AÑO 2013
Grupos a consolidar: 3
  - Ene_Feb_Mar_Abr_May_Jun_Jul_Ago
  - Sept_Oct
  - Nov_Dic

--- Consolidando 1VENTANA ---
  ✓ huancayo_2013_Ene_Feb_Mar_Abr_May_Jun_Jul_Ago_1ventana_dataset_completo.csv: 233 registros
  ✓ huancayo_2013_Sept_Oct_1ventana_dataset_completo.csv: 61 registros
  ✓ huancayo_2013_Nov_Dic_1ventana_dataset_completo.csv: 62 registros

  📊 Antes de resolver duplicados: 356 registros
  ⚠️  Encontrados 2 registros duplicados en 1 fechas
  🔧 Resolviendo duplicados por cantidad de nulls...
    • 2013-08-31: Seleccionada fila con 15 nulls (descartada con 17 nulls)
  ✅ Después de resolver duplicados: 355 registros únicos

  💾 Guardado: Resultados/2013/huancayo_2013_1ventana_CONSOLIDADO_COMPLETO.csv
  📊 Total registros: 355

  📈 Análisis de TARGET:
    - Registros CON target: 252 (71.0%)
    - R